## 步驟0：資料預處理與重採樣

In [ ]:
from typing import List, Dict, Any, Optional, Tuple

In [ ]:
train_start = '2014-11-01'
test_start = '2023-01-01'
test_end = '2026-01-01'

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 讀取NAS100資料
def load_nas100_data(file_path):
    """載入並預處理NAS100資料"""
    df = pd.read_csv(file_path)
    
    # 轉換時間格式
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # 使用BidClose作為主要價格
    df['Close'] = df['BidClose']
    df['High'] = df['BidHigh'] 
    df['Low'] = df['BidLow']
    df['Open'] = df['BidOpen']
    
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

# 資料重採樣
def resample_data(df, interval='1H'):
    """將15分鐘資料重採樣為指定間隔"""
    resampled = df.resample(interval).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()
    
    return resampled

# 執行資料載入和重採樣
nas100_raw = load_nas100_data('equity_prices/NAS100.csv')

# # 僅保留 2015-11-01 之後
nas100_raw = nas100_raw.loc[nas100_raw.index >= train_start]
nas100_raw = nas100_raw.loc[nas100_raw.index <= test_end]
nas100_raw

# nas100_hourly = resample_data(nas100_raw, '1H')

# print(f"原始資料筆數: {len(nas100_raw)}")
# print(f"重採樣後筆數: {len(nas100_hourly)}")
# print(f"資料期間: {nas100_hourly.index[0]} 至 {nas100_hourly.index[-1]}")

## 步驟1：資料雜訊過濾

In [ ]:
# =====================================================
# 1️⃣ 雜訊過濾 Class
# =====================================================
class NoiseFilter:
    """
    雜訊過濾器：計算波動率、CUSUM 過濾等
    
    CUSUM Filter 說明：
    ==================
    CUSUM (Cumulative Sum) Filter 是一種品質控制方法，用於檢測測量值
    相對於目標值的均值偏移。
    
    用途：
    -----
    1. 過濾市場雜訊，只保留顯著的價格變動事件
    2. 避免在價格在閾值附近徘徊時觸發多個事件（這是 Bollinger Bands 等
       常見市場信號的缺點）
    3. 需要完整的運行長度 h 才會觸發事件，使事件更具意義
    
    數學原理：
    ---------
    對於觀測值 {y_t}_{t=1,...,T}，定義累積和：
    
    S^+_t = max{0, S^+_{t-1} + y_t - E_{t-1}[y_t]},  S^+_0 = 0
    S^-_t = min{0, S^-_{t-1} + y_t - E_{t-1}[y_t]},  S^-_0 = 0
    S_t = max{S^+_t, -S^-_t}
    
    其中 E_{t-1}[y_t] = y_{t-1}（使用前一期值作為期望值）
    
    當 S_t ≥ h（閾值）時，觸發事件並重置 S_t = 0。
    
    對稱 CUSUM Filter 的優勢：
    -------------------------
    - 同時捕捉上漲和下跌的累積偏差
    - 零底線機制：當 y_t ≤ E_{t-1}[y_t] - S_{t-1} 時，S_t = 0
    - 避免在閾值附近反覆觸發事件
    - 需要完整的運行長度才會觸發，使事件更具統計意義
    
    參考文獻：
    --------
    - Lam and Yam (1997): 提出交替買賣信號的投資策略
    - Fama and Blume (1966): Filter trading strategy
    """
    
    def __init__(self, vol_span: int = 100):
        """
        Parameters:
        -----------
        vol_span : int
            日波動率計算的 EWM span
        """
        self.vol_span = vol_span
        self.daily_vol = None
        self.total_bars = None
        self.filtered_bars = None
        
    def calculate_daily_vol(self, close: pd.Series) -> pd.Series:
        """
        計算日波動率
        
        使用指數加權移動標準差 (EWM) 計算波動率，對近期數據給予更高權重。
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
            
        Returns:
        --------
        daily_vol : pd.Series
            日波動率序列
        """
        # 記錄總 bar 數
        self.total_bars = len(close)
        
        # 計算日收益率
        df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
        df0 = df0[df0 > 0]
        df0 = pd.Series(close.index[df0 - 1], 
                        index=close.index[close.shape[0] - df0.shape[0]:])
        df0 = close.loc[df0.index] / close.loc[df0.values].values - 1
        
        # 使用 EWM 計算標準差（波動率）
        df0 = df0.ewm(span=self.vol_span).std()
        
        self.daily_vol = df0
        
        # 統計資訊
        valid_vol_count = df0.notna().sum()
        vol_stats = self.get_vol_stats()
        
        print("=" * 60)
        print("📊 波動率計算統計")
        print("=" * 60)
        print(f"總 K 線數 (Total Bars): {self.total_bars:,}")
        print(f"有效波動率數據數: {valid_vol_count:,}")
        print(f"波動率計算時間窗口 (vol_span): {self.vol_span} 期")
        print(f"  (註: EWM 使用指數加權，不是固定窗口)")
        print(f"\n波動率統計:")
        if vol_stats:
            print(f"  平均值 (Mean): {vol_stats['mean']:.6f}")
            print(f"  標準差 (Std): {vol_stats['std']:.6f}")
            print(f"  最小值 (Min): {vol_stats['min']:.6f}")
            print(f"  最大值 (Max): {vol_stats['max']:.6f}")
            print(f"  中位數 (Median): {vol_stats['median']:.6f}")
        print("=" * 60)
        
        return df0
    
    def cusum_filter(self, close: pd.Series, threshold: float) -> pd.DatetimeIndex:
        """
        對稱 CUSUM 過濾器 (Symmetric CUSUM Filter)
        
        實現原理：
        ---------
        對價格序列的差分（diff = y_t - y_{t-1}）應用對稱 CUSUM 過濾：
        
        S^+_t = max{0, S^+_{t-1} + diff_t}  (上漲累積)
        S^-_t = min{0, S^-_{t-1} + diff_t}  (下跌累積)
        
        當 S^-_t < -h 或 S^+_t > h 時：
        - 觸發事件（記錄時間點）
        - 重置對應的累積和為 0
        
        這樣設計的優勢：
        --------------
        1. 避免在閾值附近反覆觸發（需要完整的運行長度 h）
        2. 同時捕捉上漲和下跌的累積偏差
        3. 零底線機制確保只捕捉單方向的持續偏差
        
        Parameters:
        -----------
        close : pd.Series
            價格序列
        threshold : float
            CUSUM 閾值 (h)
            通常使用波動率的平均值或標準差
            
        Returns:
        --------
        tEvents : pd.DatetimeIndex
            過濾後的事件時間點
            
        參考實現：
        ---------
        基於 Snippet 2.4 (Advances in Financial Machine Learning, Chapter 2)
        """
        # 記錄總 bar 數
        self.total_bars = len(close)
        
        tEvents = []
        sPos, sNeg = 0, 0  # S^+_0 = 0, S^-_0 = 0
        diff = close.diff()  # y_t - y_{t-1}
        
        # 遍歷每個時間點（從第二個開始，因為第一個 diff 是 NaN）
        for i in diff.index[1:]:
            # 更新累積和
            # S^+_t = max{0, S^+_{t-1} + diff_t}
            sPos = max(0, sPos + diff.loc[i])
            # S^-_t = min{0, S^-_{t-1} + diff_t}
            sNeg = min(0, sNeg + diff.loc[i])
            
            # 檢查是否觸發事件
            if sNeg < -threshold:
                # 下跌累積超過閾值：觸發事件並重置
                sNeg = 0
                tEvents.append(i)
            elif sPos > threshold:
                # 上漲累積超過閾值：觸發事件並重置
                sPos = 0
                tEvents.append(i)
        
        filtered_events = pd.DatetimeIndex(tEvents)
        self.filtered_bars = len(filtered_events)
        
        # 統計資訊
        filter_rate = self.filtered_bars / self.total_bars if self.total_bars > 0 else 0
        
        print("=" * 60)
        print("📊 CUSUM 過濾統計")
        print("=" * 60)
        print(f"總 K 線數 (Total Bars): {self.total_bars:,}")
        print(f"篩選後 K 線數 (Filtered Bars): {self.filtered_bars:,}")
        print(f"過濾比例: {filter_rate:.4%}")
        print(f"CUSUM 閾值 (Threshold h): {threshold:.6f}")
        print(f"\n說明:")
        print(f"  - 只保留價格累積變動超過閾值 {threshold:.6f} 的時間點")
        print(f"  - 避免在閾值附近反覆觸發事件")
        print(f"  - 需要完整的運行長度才會觸發，使事件更具統計意義")
        print("=" * 60)
        
        return filtered_events
    
    def get_vol_stats(self) -> dict:
        """取得波動率統計資訊"""
        if self.daily_vol is None:
            return None
        
        return {
            'mean': self.daily_vol.mean(),
            'std': self.daily_vol.std(),
            'min': self.daily_vol.min(),
            'max': self.daily_vol.max(),
            'median': self.daily_vol.median()
        }
    
    def plot_volatility(self):
        """繪製波動率圖"""
        import matplotlib.pyplot as plt
        
        if self.daily_vol is None:
            print("⚠️ 請先執行 calculate_daily_vol()")
            return
        
        fig, ax = plt.subplots(figsize=(15, 5))
        self.daily_vol.plot(ax=ax, title='Daily Volatility', linewidth=1)
        ax.axhline(self.daily_vol.mean(), color='red', linestyle='--', 
                   label=f'Mean: {self.daily_vol.mean():.4f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

In [ ]:
# 1. 雜訊過濾
noise_filter = NoiseFilter(vol_span=500)
daily_vol = noise_filter.calculate_daily_vol(nas100_raw['Close'])

# 2. CUSUM 篩選數據
# 使用波動率平均值作為閾值
# cusum_threshold = daily_vol.mean()
cusum_threshold = 0.001
cusum_events = noise_filter.cusum_filter(nas100_raw['Close'], cusum_threshold)
df_filtered = nas100_raw.loc[cusum_events].copy()  # ✅ 過濾後的數據
print(f"過濾後的資料筆數: {len(df_filtered)}")
noise_filter.plot_volatility()
df_filtered

## 步驟2 ：策略訊號計算

In [ ]:
# =====================================================
# 2️⃣ L1-OLD 策略 Class
# =====================================================
class L1Strategy:
    """L1-OLD 突破策略"""
    
    def __init__(self, entry_param: float = 0.5, rolling_window: int = 460):
        """
        Parameters:
        -----------
        entry_param : float
            突破參數（預設 0.5）
        rolling_window : int
            計算 HL115 和突破價的滾動窗口（預設 460）
        """
        self.entry_param = entry_param
        self.rolling_window = rolling_window
        self.signals = None
        
    def calculate_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        計算策略指標
        
        Parameters:
        -----------
        df : pd.DataFrame
            需包含 'High', 'Low', 'Open', 'Close' 欄位
            
        Returns:
        --------
        df : pd.DataFrame
            新增 'HL115', '突破signal價', 'signal', 'side' 欄位
        """
        df = df.copy()
        
        # 計算 HL115（過去N根K線的價格範圍）
        df['HL115'] = (
            df['High'].shift(1).rolling(self.rolling_window, 
                                           min_periods=self.rolling_window).max() - 
            df['Low'].shift(1).rolling(self.rolling_window, 
                                         min_periods=self.rolling_window).min()
        )
        df['HL115%'] = df['HL115'] / df['Low'].shift(self.rolling_window+1)
        
        # 計算突破signal價
        df['突破signal價'] = (
            df['Low'].shift(1).rolling(self.rolling_window).min() + 
            (self.entry_param * df['HL115'])
        )
        df['突破signal價%'] = df['突破signal價']/df['Open']
        
        # 生成突破信號
        df['signal'] = False
        df['signal'] = (
            (df['signal'].shift(1) == False) & 
            (df['High'] > df['突破signal價']) & 
            (df['Open'] < df['突破signal價'])
        )
        
        # 轉換為 side（做多方向）
        df['side'] = np.nan
        df.loc[df['signal'] == True, 'side'] = 1.0
        df['side'] = df['side'].ffill()
        
        self.signals = df
        
        # 統計資訊
        total_bars = len(df)
        signal_count = df['signal'].sum()
        signal_rate = signal_count / total_bars if total_bars > 0 else 0
        
        print("=" * 60)
        print("📊 L1-OLD 策略統計")
        print("=" * 60)
        print(f"總 K 線數 (Bars): {total_bars:,}")
        print(f"信號數量 (Signals): {signal_count:,}")
        print(f"信號比例: {signal_rate:.4%}")
        print(f"策略參數: entry_param={self.entry_param}, rolling_window={self.rolling_window}")
        print("=" * 60)
        
        return df
    
    def get_signal_events(self) -> pd.DatetimeIndex:
        """取得信號發生的時間點"""
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return None
        
        return self.signals[self.signals['signal'] == True].index
    
    def get_signal_stats(self) -> dict:
        """取得信號統計資訊"""
        if self.signals is None:
            return None
        
        signal_count = self.signals['signal'].sum()
        total_bars = len(self.signals)
        
        return {
            'total_signals': signal_count,
            'signal_rate': signal_count / total_bars,
            'avg_HL115': self.signals['HL115'].mean(),
            'avg_price_to_signal': (self.signals['Close'] / self.signals['突破signal價']).mean()
        }
    
    def plot_strategy(self, start_idx: Optional[int] = None, end_idx: Optional[int] = None):
        """
        繪製策略圖
        
        Parameters:
        -----------
        start_idx : int, optional
            起始索引
        end_idx : int, optional
            結束索引
        """
        import matplotlib.pyplot as plt
        
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return
        
        df = self.signals.iloc[start_idx:end_idx] if start_idx or end_idx else self.signals
        signal_events = df[df['signal'] == True].index
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 10))
        
        # 1. 價格與突破信號
        axes[0].plot(df.index, df['Close'], label='Close Price', alpha=0.7, linewidth=1)
        axes[0].plot(df.index, df['突破signal價'], label='Breakout Signal', 
                    linestyle='--', alpha=0.7, color='orange')
        axes[0].scatter(signal_events, df.loc[signal_events, 'Close'], 
                       color='red', s=50, label='Entry Signal', zorder=5, marker='^')
        axes[0].set_title(f'L1-OLD Strategy (entry_param={self.entry_param})', fontsize=12)
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. HL115 波動指標
        axes[1].plot(df.index, df['HL115'], label='HL115', color='orange', linewidth=1)
        axes[1].set_title(f'Price Range Indicator (window={self.rolling_window})', fontsize=12)
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# =====================================================
# 3️⃣ S1 策略 Class (做空)
# =====================================================
class S1Strategy:
    """S1 做空策略"""
    
    def __init__(self, entry_param: float = 0.5, entry_param2: float = 0.4, rolling_window: int = 460):
        self.entry_param = entry_param
        self.entry_param2 = entry_param2
        self.rolling_window = rolling_window
        self.signals = None
        
    def calculate_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        # 計算 HL115
        df['HL115'] = (
            df['High'].shift(1).rolling(self.rolling_window, min_periods=self.rolling_window).max() - 
            df['Low'].shift(1).rolling(self.rolling_window, min_periods=self.rolling_window).min()
        )
        
        # 計算 lowpoint
        prev_low_min = df['Low'].shift(1).rolling(self.rolling_window).min()
        prev_high_max = df['High'].shift(1).rolling(self.rolling_window).max()
        
        # 避免除以零
        denom = prev_high_max - prev_low_min
        denom = denom.replace(0, np.nan)
        
        df['lowpoint'] = (df['Low'] - prev_low_min) / denom
        
        # 計算 avg_lowpoint
        df['avg_lowpoint'] = df['lowpoint'].shift(1).rolling(self.rolling_window).mean()
        
        # 計算 突破signal價
        df['突破signal價'] = prev_low_min + (self.entry_param * df['HL115'])
        
        # 生成信號
        # 條件: Low < 突破價 & Open > 突破價 & avg_lowpoint < entry_param2
        condition = (
            (df['Low'] < df['突破signal價']) & 
            (df['Open'] > df['突破signal價'])  
            # (df['avg_lowpoint'] < self.entry_param2)
        )
        
        df['signal'] = condition
        df.loc[df.index[:115], 'signal'] = False
        
        # 轉換為 side (-1 做空)
        df['side'] = np.nan
        df.loc[df['signal'] == True, 'side'] = -1.0
        df['side'] = df['side'].ffill()
        
        self.signals = df
        
        # 統計
        total_bars = len(df)
        signal_count = df['signal'].sum()
        signal_rate = signal_count / total_bars if total_bars > 0 else 0
        
        print("=" * 60)
        print("📊 S1 策略統計 (Short)")
        print("=" * 60)
        print(f"總 K 線數: {total_bars:,}")
        print(f"信號數量: {signal_count:,}")
        print(f"信號比例: {signal_rate:.4%}")
        print(f"策略參數: entry_param={self.entry_param}, entry_param2={self.entry_param2}, rolling_window={self.rolling_window}")
        print("=" * 60)
        
        return df
    
    def get_signal_events(self) -> pd.DatetimeIndex:
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return None
        return self.signals[self.signals['signal'] == True].index
    
    def plot_strategy(self, start_idx: Optional[int] = None, end_idx: Optional[int] = None):
        import matplotlib.pyplot as plt
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return
        
        df = self.signals.iloc[start_idx:end_idx] if start_idx or end_idx else self.signals
        signal_events = df[df['signal'] == True].index
        
        fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
        
        # 1. 價格與突破信號
        axes[0].plot(df.index, df['Close'], label='Close Price', alpha=0.7, linewidth=1)
        axes[0].plot(df.index, df['突破signal價'], label='Breakout Signal', linestyle='--', alpha=0.7, color='orange')
        axes[0].scatter(signal_events, df.loc[signal_events, 'Close'], color='green', s=50, label='Short Signal', zorder=5, marker='v')
        axes[0].set_title(f'S1 Strategy (Short)', fontsize=12)
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. lowpoint & avg_lowpoint
        axes[1].plot(df.index, df['lowpoint'], label='lowpoint', alpha=0.5, linewidth=0.5)
        axes[1].plot(df.index, df['avg_lowpoint'], label='avg_lowpoint', color='red', linewidth=1)
        axes[1].axhline(y=self.entry_param2, color='gray', linestyle='--', label='Threshold')
        axes[1].set_title('Lowpoint Indicator', fontsize=12)
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # 3. HL115
        axes[2].plot(df.index, df['HL115'], label='HL115', color='orange', linewidth=1)
        axes[2].set_title('HL115', fontsize=12)
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()


In [ ]:


# 3. 在篩選後的數據上計算策略信號
print("\n" + "=" * 60)
print("📈 在篩選後的數據上計算策略信號")
print("=" * 60)

strategy = S1Strategy(entry_param=0.5, rolling_window=500)
df = strategy.calculate_indicators(df_filtered)  # ✅ 使用 df_filtered
tEvents = strategy.get_signal_events()  # 策略信號的事件點
strategy.plot_strategy()

## 步驟3：Meta Labeling 目標變數生成與止盈止損設定

In [ ]:
# =====================================================
# 3️⃣ Meta-Labeling Class
# =====================================================
class MetaLabeling:
    """
    Meta-Labeling：Triple Barrier 和標籤生成
    
    Triple Barrier Method 說明：
    ============================
    Triple Barrier Method 是一種標籤方法，根據三個屏障中第一個被觸及的
    屏障來標記觀測值。
    
    用途：
    -----
    1. 動態設定止盈止損：根據估計的波動率（已實現或隱含）動態調整
    2. 避免目標過高或過低：考慮當前波動率，設定合理的止盈止損水平
    3. 路徑依賴標籤：考慮從 [t_i,0, t_i,0 + h] 的完整價格路徑
    
    三個屏障：
    --------
    1. 上方水平屏障（Profit Taking）：止盈限制
    2. 下方水平屏障（Stop Loss）：止損限制
    3. 垂直屏障（Vertical Barrier）：時間到期限制（持有期）
    
    屏障配置：
    --------
    用三元組 [pt, sl, t1] 表示，其中：
    - 0 表示屏障未啟用
    - 1 表示屏障啟用
    
    常用配置：
    - [1,1,1]: 標準設定（止盈、止損、時間到期）
    - [0,1,1]: 只有止損和時間到期
    - [1,1,0]: 只有止盈和止損（無時間限制）
    
    數學原理：
    ---------
    對於每個事件時間點 t_i,0：
    
    1. 上方屏障（止盈）：
       PT = t_i,0 + pt × σ_t_i,0
       其中 pt 是止盈倍數，σ_t_i,0 是目標波動率
    
    2. 下方屏障（止損）：
       SL = t_i,0 - sl × σ_t_i,0
       其中 sl 是止損倍數
    
    3. 垂直屏障（時間到期）：
       t_i,1 = t_i,0 + h
       其中 h 是持有期（期數）
    
    4. 第一個觸及的屏障：
       t_i,1 = min{PT_touch, SL_touch, t_i,0 + h}
    
    5. 標籤生成：
       - bin = 1:  觸及上方屏障（成功）
       - bin = -1: 觸及下方屏障（失敗）
       - bin = 0:  觸及垂直屏障（時間到期）
    
    路徑依賴性：
    ----------
    為了標記觀測值，必須考慮從 t_i,0 到 t_i,0 + h 的完整價格路徑。
    這使得標籤是路徑依賴的，而非僅依賴終點價格。
    
    動態閾值：
    --------
    使用 getDailyVol() 計算日波動率，作為動態調整止盈止損的基礎：
    
    σ_t = EWM_std(returns, span=span0)
    
    其中 returns 是日收益率序列。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 3
    - Snippet 3.1: getDailyVol - 計算日波動率
    - Snippet 3.2: applyPtSlOnT1 - Triple Barrier 標籤方法
    - Snippet 3.3: getEvents - 取得第一個屏障觸及時間
    - Snippet 3.4: getVerticalBarriers - 設定垂直屏障
    - Snippet 3.5: getBins - 生成標籤
    """
    
    def __init__(self, ptSl: List[float] = [1, 2], numPeriods: int = 100, 
                 minRet: float = 0.01, min_label_pct: float = 0.05):
        """
        Parameters:
        -----------
        ptSl : list
            [止盈倍數, 止損倍數]
            - ptSl[0]: 上方屏障的倍數（Profit Taking）
            - ptSl[1]: 下方屏障的倍數（Stop Loss）
            - 如果為 0，則該屏障不啟用
        numPeriods : int
            垂直屏障期數（例如：100 根 K 線）
            對應論文中的 h（持有期）
        minRet : float
            最小波動率閾值
            只有當 trgt > minRet 時才會生成事件
        min_label_pct : float
            最小標籤比例（用於 dropLabels）
            低於此比例的標籤會被移除
        """
        self.ptSl = ptSl
        self.numPeriods = numPeriods
        self.minRet = minRet
        self.min_label_pct = min_label_pct
        self.events = None
        self.bins = None
        
    def get_vertical_barriers(self, close: pd.Series, tEvents: pd.DatetimeIndex) -> pd.Series:
        """
        設定垂直時間屏障（以期數為單位）
        
        實現原理（基於 Snippet 3.4）：
        ----------------------------
        對每個事件時間點 t_i,0，找到 numPeriods 期後的時間點。
        
        公式：
        t_i,1 = close.index[position(t_i,0) + numPeriods]
        
        如果超出數據範圍，則使用最後一個時間點。
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點
            
        Returns:
        --------
        t1 : pd.Series
            垂直屏障時間點（index=事件起點, value=屏障時間點）
        """
        t1 = []
        close_index = close.index
        
        for tEvent in tEvents:
            # 找到 tEvent 在 close 中的位置
            try:
                event_pos = close_index.get_loc(tEvent)
                # 計算 numPeriods 期後的位置
                barrier_pos = event_pos + self.numPeriods
                
                # 確保不超過數據範圍
                if barrier_pos < len(close_index):
                    t1.append(close_index[barrier_pos])
                else:
                    # 如果超出範圍，使用最後一個時間點
                    t1.append(close_index[-1])
            except KeyError:
                # 如果找不到該時間點，跳過
                continue
        
        if len(t1) == 0:
            return pd.Series(dtype='datetime64[ns]')
        
        # 確保 t1 的索引與 tEvents 對應
        t1_series = pd.Series(t1, index=tEvents[:len(t1)])
        return t1_series
    
    def apply_pt_sl_on_t1(self, close: pd.Series, events: pd.DataFrame, 
                          molecule: pd.DatetimeIndex) -> pd.DataFrame:
        """
        套用止盈止損（基於 Snippet 3.2）
        
        實現原理：
        ---------
        對每個事件，計算從事件開始到垂直屏障期間的價格路徑：
        
        path_prices = close[loc:t1]
        path_returns = (path_prices / close[loc] - 1) × side
        
        然後找出：
        - 最早觸及止盈的時間：path_returns > pt × trgt
        - 最早觸及止損的時間：path_returns < -sl × trgt
        
        公式：
        - PT_level = pt × trgt
        - SL_level = -sl × trgt
        - path_returns = (close[t] / close[loc] - 1) × side
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        events : pd.DataFrame
            事件 DataFrame（包含 t1, trgt, side）
        molecule : pd.DatetimeIndex
            要處理的事件子集（用於並行計算）
            
        Returns:
        --------
        out : pd.DataFrame
            包含每個屏障觸及時間的 DataFrame
        """
        events_ = events.loc[molecule]
        out = events_[['t1']].copy(deep=True)
        
        # 計算止盈水平
        if self.ptSl[0] > 0:
            pt = self.ptSl[0] * events_['trgt']  # PT = pt × σ
        else:
            pt = pd.Series(index=events.index)  # 未啟用
        
        # 計算止損水平
        if self.ptSl[1] > 0:
            sl = -self.ptSl[1] * events_['trgt']  # SL = -sl × σ
        else:
            sl = pd.Series(index=events.index)  # 未啟用
        
        # 對每個事件計算路徑
        for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
            # 取得價格路徑
            df0 = close[loc:t1]
            # 計算路徑報酬（考慮方向）
            df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']
            
            # 找出最早觸及止損的時間
            out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
            # 找出最早觸及止盈的時間
            out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
        
        return out
    
    def get_events(self, close: pd.Series, tEvents: pd.DatetimeIndex, 
                   trgt: pd.Series, side: pd.Series) -> pd.DataFrame:
        """
        生成事件並套用 Triple Barrier（基於 Snippet 3.3）
        
        實現流程：
        ---------
        1. 過濾目標：只保留 trgt > minRet 的事件
        2. 設定垂直屏障：計算每個事件的時間到期點
        3. 套用 Triple Barrier：計算止盈止損觸及時間
        4. 找出第一個觸及的屏障：t1 = min{pt_touch, sl_touch, vertical_barrier}
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點（來自 CUSUM 過濾或策略信號）
        trgt : pd.Series
            目標波動率（daily_vol，用於動態調整止盈止損）
        side : pd.Series
            方向（1 為做多，-1 為做空）
            
        Returns:
        --------
        events : pd.DataFrame
            包含 t1, trgt, side, pt, sl 的事件 DataFrame
            - t1: 第一個屏障觸及時間
            - trgt: 目標波動率
            - side: 方向
            - pt: 止盈倍數
            - sl: 止損倍數
        """
        # 1. 過濾目標（只保留波動率足夠大的事件）
        trgt = trgt.loc[tEvents]
        trgt = trgt[trgt > self.minRet]
        
        # 2. 設定垂直屏障
        t1 = self.get_vertical_barriers(close, tEvents)
        
        # 3. 取得方向
        side_ = side.loc[trgt.index]
        
        # 4. 組合事件物件
        events = pd.concat({
            't1': t1.loc[trgt.index], 
            'trgt': trgt, 
            'side': side_
        }, axis=1)
        events = events.dropna(subset=['trgt'])
        
        # 5. 計算止盈止損觸及時間
        df0 = self.apply_pt_sl_on_t1(close, events, events.index)
        
        # 6. 找出第一個觸及的屏障（最早的時間）
        events['t1'] = df0.dropna(how='all').min(axis=1)
        
        # 7. 儲存屏障配置
        events['pt'] = self.ptSl[0]
        events['sl'] = self.ptSl[1]
        
        self.events = events
        return events
    
    def get_bins(self, close: pd.Series) -> pd.DataFrame:
        """
        生成標籤（基於 Snippet 3.5，並擴展為支援 Meta-Labeling）
        
        實現原理：
        ---------
        1. 對齊價格：取得事件開始和結束時間的所有價格點
        2. 計算實際報酬：
           ret = (close[t1] / close[t0] - 1) × side
        3. 判斷觸及的屏障：
           - 如果 ret > 0 且 ret > pt × trgt → bin = 1（觸及止盈）
           - 如果 ret < 0 且 ret < -sl × trgt → bin = -1（觸及止損）
           - 否則 → bin = 0（觸及垂直屏障）
        
        標籤定義：
        --------
        - bin = 1:  成功（觸及止盈屏障）
        - bin = 0:  時間到期（觸及垂直屏障）
        - bin = -1: 虧損（觸及止損屏障）
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
            
        Returns:
        --------
        bins : pd.DataFrame
            包含 ret, trgt, bin, side 的標籤 DataFrame
        """
        if self.events is None:
            print("⚠️ 請先執行 get_events()")
            return None
        
        events_ = self.events.dropna(subset=['t1'])
        px = events_.index.union(events_['t1'].values).drop_duplicates()
        px = close.reindex(px, method='bfill')
        
        out = pd.DataFrame(index=events_.index)
        out['ret'] = px.loc[events_['t1'].values].values / px.loc[events_.index] - 1
        out['ret'] *= events_['side']
        out['trgt'] = events_['trgt']
        
        # 判斷觸及哪個屏障
        out['bin'] = 0  # 預設為時間到期
        for date_time, values in out.iterrows():
            ret = values['ret']
            target = values['trgt']
            pt_level = ret > target * self.events.loc[date_time, 'pt']
            sl_level = ret < -target * self.events.loc[date_time, 'sl']
            
            if ret > 0.0 and pt_level:
                # 觸及止盈屏障 → bin = 1 (成功)
                out.loc[date_time, 'bin'] = 1
            elif ret < 0.0 and sl_level:
                # 觸及止損屏障 → bin = -1 (虧損)
                out.loc[date_time, 'bin'] = -1
            else:
                # 時間到期（垂直屏障）→ bin = 0
                out.loc[date_time, 'bin'] = 0
        
        # 如果有 side（Meta-Labeling），保留 side 資訊
        if 'side' in events_:
            out['side'] = events_['side']
        
        self.bins = out
        return out
    
    def drop_rare_labels(self) -> pd.DataFrame:
        """
        移除稀有標籤（基於 Snippet 3.8）
        
        目的：
        -----
        移除樣本數過少的標籤類別，避免模型學習偏斜。
        
        流程：
        -----
        1. 計算每個標籤的比例
        2. 如果最小比例 < min_label_pct，移除該標籤
        3. 重複直到所有標籤比例都 >= min_label_pct
        """
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return None
        
        bins = self.bins.copy()
        
        while True:
            df0 = bins['bin'].value_counts(normalize=True)
            if df0.min() > self.min_label_pct or df0.shape[0] < 3:
                break
            print(f"Dropped label {df0.idxmin()}, percentage: {df0.min():.4f}")
            bins = bins[bins['bin'] != df0.idxmin()]
        
        self.bins = bins
        self.events = self.events.loc[bins.index]
        
        return bins
    
    def get_label_stats(self) -> dict:
        """取得標籤統計資訊"""
        if self.bins is None:
            return None
        
        label_counts = self.bins['bin'].value_counts()
        
        return {
            'total_labels': len(self.bins),
            'label_distribution': label_counts.to_dict(),
            'avg_return': self.bins['ret'].mean(),
            'win_rate': (self.bins['bin'] == 1).sum() / len(self.bins),
            'loss_rate': (self.bins['bin'] == -1).sum() / len(self.bins),
            'timeout_rate': (self.bins['bin'] == 0).sum() / len(self.bins)
        }
    
    def plot_labels(self):
        """繪製標籤分布圖"""
        import matplotlib.pyplot as plt
        
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # 標籤分布
        label_counts = self.bins['bin'].value_counts().sort_index()
        colors = {-1: 'red', 0: 'gray', 1: 'green'}
        label_colors = [colors.get(idx, 'steelblue') for idx in label_counts.index]
        
        label_counts.plot(kind='bar', ax=axes[0], color=label_colors)
        axes[0].set_title('Label Distribution', fontsize=12)
        axes[0].set_xlabel('Label (-1: Loss, 0: Timeout, 1: Win)')
        axes[0].set_ylabel('Count')
        axes[0].grid(True, alpha=0.3, axis='y')
        
        # 回報分布
        self.bins['ret'].hist(bins=50, ax=axes[1], alpha=0.7, color='steelblue')
        axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Return')
        axes[1].set_title('Return Distribution', fontsize=12)
        axes[1].set_xlabel('Return')
        axes[1].set_ylabel('Frequency')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [ ]:
# 3. Meta-Labeling（使用原始完整數據）
meta = MetaLabeling(ptSl=[0.25, 0.25], numPeriods=32)
events = meta.get_events(
    nas100_raw['Close'],  # ✅ 原始完整數據
    tEvents,              # 策略信號的事件點
    daily_vol,            # 基於原始數據計算
    df['side']            # 策略信號的方向（來自過濾後的數據）
)
bins = meta.get_bins(nas100_raw['Close'])  # ✅ 原始完整數據
meta.plot_labels()
meta.drop_rare_labels()
meta.get_label_stats()

## 步驟4 ： 樣本權重平衡

In [ ]:
# =====================================================
# 4️⃣ 樣本權重 Class（並發度、唯一性、時間衰減、類別平衡）
# =====================================================
class SampleWeight:
    """
    樣本權重計算：並發度、唯一性、時間衰減、類別平衡
    
    Chapter 4: Sample Weights 說明
    ===============================
    
    問題背景：
    --------
    在金融應用中，觀測值不是由獨立同分布（IID）過程生成的。
    當兩個標籤 y_i 和 y_j 的時間區間有重疊時（t_i,1 > t_j,0），
    它們會依賴共同的報酬 r_{t_j,0, min{t_i,1, t_j,1}}。
    
    這導致標籤序列 {y_i}_{i=1,...,I} 不是 IID，違反了大多數 ML 算法的假設。
    
    解決方案：
    --------
    通過設計採樣和加權方案來糾正重疊結果的不當影響：
    1. 計算並發度（Concurrency）：每個時間點有多少標籤同時存活
    2. 計算唯一性（Uniqueness）：每個標籤的平均唯一性
    3. 應用時間衰減（Time Decay）：讓舊樣本權重降低
    4. 類別平衡（Class Balance）：平衡不同類別的樣本權重
    5. 組合權重：最終權重 = 唯一性 × 時間衰減 × 類別平衡（可選：× 報酬歸因）
    
    並發度（Concurrency）：
    ---------------------
    定義：兩個標籤 y_i 和 y_j 在時間 t 是並發的，當它們都依賴至少一個
    共同的報酬 r_{t-1,t} = p_t / p_{t-1} - 1。
    
    計算方式：
    對每個時間點 t = 1,...,T，形成二元陣列 {1_{t,i}}_{i=1,...,I}：
    - 1_{t,i} = 1：如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊
    - 1_{t,i} = 0：否則
    
    並發度：c_t = Σ_{i=1}^I 1_{t,i}
    
    唯一性（Uniqueness）：
    --------------------
    定義：標籤 i 在時間 t 的唯一性為 u_{t,i} = 1_{t,i} / c_t
    
    平均唯一性：ū_i = (Σ_{t=1}^T 1_{t,i})^{-1} × (Σ_{t=1}^T u_{t,i})
    
    也可以解釋為：標籤 i 存活期間的並發度倒數的調和平均。
    
    報酬歸因（Return Attribution）：
    -----------------------------
    當標籤是報酬符號的函數時（{-1,1} 或 {0,1}），樣本權重可以定義為：
    
    ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
    
    其中 r_{t-1,t} 是對數報酬，c_t 是時間 t 的並發度。
    
    然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
    
    時間衰減（Time Decay）：
    ----------------------
    市場是適應性系統，舊樣本不如新樣本相關。
    
    線性時間衰減：
    d[x] = max{0, a + bx}
    
    邊界條件：
    1. d[Σ_{i=1}^I ū_i] = 1（最新樣本權重為 1）
    2. d[0] = c（最舊樣本權重為 c，c ∈ [0,1]）
    
    參數 c 的意義：
    - c = 1：無時間衰減
    - 0 < c < 1：線性衰減，所有樣本權重 > 0
    - c = 0：最舊樣本權重為 0
    - c < 0：最舊的 |c|×T 部分樣本權重為 0
    
    注意：時間衰減是基於累積唯一性，而非時間順序，因為在存在冗餘
    觀測值的情況下，按時間順序衰減會使權重降低太快。
    
    類別平衡（Class Balance）：
    ------------------------
    當標籤分布不平衡時（例如：上漲標籤多於下跌標籤），需要對少數類別
    給予更高的權重，以平衡模型訓練。
    
    計算方式：
    w_class = n_samples / (n_classes × n_class_samples)
    
    這樣可以讓少數類別的樣本權重更高，平衡類別分布。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 4
    - Snippet 4.1: mpNumCoEvents - 計算並發度
    - Snippet 4.2: mpSampleTW - 計算平均唯一性
    - Snippet 4.10: mpSampleW - 報酬歸因權重
    - Snippet 4.11: getTimeDecay - 時間衰減因子
    """
    
    def __init__(self):
        self.num_co_events = None
        self.uniqueness = None
        self.time_decay = None
        self.class_balance_weights = None
        self.sample_weights = None
        
    def compute_num_co_events(self, close_idx: pd.DatetimeIndex, 
                              t1: pd.Series) -> pd.Series:
        """
        計算每個時間點的並發事件數（Concurrency）
        基於 Snippet 4.1: mpNumCoEvents
        
        實現原理：
        ---------
        對每個時間點 t，計算有多少個事件的存活期間 [t_i,0, t_i,1] 
        與時間點 t 重疊。
        
        公式：
        c_t = Σ_{i=1}^I 1_{t,i}
        
        其中 1_{t,i} = 1 如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊，否則為 0。
        
        參數:
        -----------
        close_idx : pd.DatetimeIndex
            價格數據的索引
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
            
        Returns:
        --------
        num_co_events : pd.Series
            每個時間點的並發事件數
        """
        t1 = t1.fillna(close_idx[-1])  # 未結束事件仍要計算
        
        # 初始化計數序列
        iloc = close_idx.searchsorted(np.array([t1.index[0], t1.max()]))
        count = pd.Series(0, index=close_idx[iloc[0]:iloc[1]+1])
        
        # 對每個事件，在其存活期間 +1
        for tIn, tOut in t1.items():
            count.loc[tIn:tOut] += 1.0
        
        self.num_co_events = count
        
        print("=" * 60)
        print("📊 並發度統計")
        print("=" * 60)
        print(f"總時間點數: {len(count):,}")
        print(f"平均並發事件數: {count.mean():.2f}")
        print(f"最大並發事件數: {count.max():.0f}")
        print(f"\n說明:")
        print(f"  - 並發度衡量每個時間點有多少個標籤同時存活")
        print(f"  - 高並發度表示標籤之間重疊多，資訊冗餘")
        print(f"  - 低並發度表示標籤相對獨立")
        print(f"並發度分布:")
        print(count.describe())
        print("=" * 60)
        
        return count

    def compute_uniqueness(self, t1: pd.Series, 
                          num_co_events: pd.Series) -> pd.Series:
        """
        計算每個事件的唯一性（Uniqueness）
        基於 Snippet 4.2: mpSampleTW
        
        實現原理：
        ---------
        對每個事件 i，計算其在存活期間的平均唯一性：
        
        u_{t,i} = 1_{t,i} / c_t  （時間 t 的唯一性）
        ū_i = (1 / T_i) × Σ_{t=t_i,0}^{t_i,1} u_{t,i}
        
        其中 T_i 是事件 i 的存活期數。
        
        也可以解釋為：事件存活期間的並發度倒數的調和平均。
        
        公式：
        ū_i = mean(1 / c_t) for t ∈ [t_i,0, t_i,1]
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        num_co_events : pd.Series
            每個時間點的並發事件數序列
            
        Returns:
        --------
        uniqueness : pd.Series
            每個事件的唯一性（1 / 平均並發度）
        """
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算事件存活期間的平均並發度倒數
            avg_co_events = num_co_events.loc[tIn:tOut].mean()
            wght.loc[tIn] = 1.0 / avg_co_events if avg_co_events > 0 else 0
        
        self.uniqueness = wght
        
        print("=" * 60)
        print("📊 唯一性統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均唯一性: {wght.mean():.4f}")
        print(f"\n說明:")
        print(f"  - 唯一性衡量每個標籤的非重疊程度")
        print(f"  - 唯一性 = 1 / 平均並發度")
        print(f"  - 唯一性越高，表示該標籤越獨立，權重應該越大")
        print(f"  - 唯一性越低，表示該標籤與其他標籤重疊多，權重應該越小")
        print(f"唯一性分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def compute_return_attribution(self, 
                                  close: pd.Series,
                                  t1: pd.Series,
                                  num_co_events: pd.Series) -> pd.Series:
        """
        計算報酬歸因權重（Return Attribution）
        基於 Snippet 4.10: mpSampleW
        
        實現原理：
        ---------
        對每個事件，計算其存活期間的歸因報酬：
        
        ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
        
        其中：
        - r_{t-1,t} = log(close_t / close_{t-1}) 是對數報酬
        - c_t 是時間 t 的並發度
        
        然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
        
        這樣設計的意義：
        - 將報酬按並發度分配，避免重疊事件重複計算報酬
        - 絕對報酬大的事件權重更高
        - 標準化後權重總和為 I（樣本數）
        
        參數:
        -----------
        close : pd.Series
            價格序列
        t1 : pd.Series
            事件結束時間
        num_co_events : pd.Series
            並發度序列
            
        Returns:
        --------
        return_weights : pd.Series
            報酬歸因權重
        """
        ret = np.log(close).diff()  # 對數報酬
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算歸因報酬：報酬 / 並發度
            attributed_ret = (ret.loc[tIn:tOut] / 
                            num_co_events.loc[tIn:tOut]).sum()
            wght.loc[tIn] = abs(attributed_ret)
        
        # 標準化：權重總和 = 樣本數
        wght = wght * len(wght) / wght.sum()
        
        print("=" * 60)
        print("📊 報酬歸因權重統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均權重: {wght.mean():.4f}")
        print(f"權重總和: {wght.sum():.0f} (應等於樣本數)")
        print(f"\n說明:")
        print(f"  - 根據歸因的絕對報酬來加權")
        print(f"  - 報酬按並發度分配，避免重疊事件重複計算")
        print(f"  - 絕對報酬大的事件權重更高")
        print(f"權重分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def compute_class_balance_weight(self, labels: pd.Series) -> pd.Series:
        """
        計算類別平衡權重
        
        實現原理：
        ---------
        對每個類別，計算其權重為：
        w_class = n_samples / (n_classes × n_class_samples)
        
        這樣可以讓少數類別的樣本權重更高，平衡類別分布。
        
        使用 sklearn 的 compute_sample_weight('balanced', labels) 實現。
        
        參數:
        -----------
        labels : pd.Series
            標籤序列（index 必須與樣本權重的 index 一致）
            
        Returns:
        --------
        class_weights : pd.Series
            每個樣本的類別平衡權重
        """
        from sklearn.utils.class_weight import compute_sample_weight
        
        # 計算類別平衡權重
        class_weights = compute_sample_weight('balanced', labels)
        class_weights_series = pd.Series(class_weights, index=labels.index)
        
        self.class_balance_weights = class_weights_series
        
        print("=" * 60)
        print("📊 類別平衡權重統計")
        print("=" * 60)
        print(f"標籤分布:")
        label_counts = labels.value_counts().sort_index()
        print(label_counts)
        print(f"\n類別平衡權重:")
        for label in sorted(labels.unique()):
            mask = labels == label
            avg_weight = class_weights_series[mask].mean()
            count = mask.sum()
            pct = count / len(labels) * 100
            print(f"  標籤 {label}: 樣本數 = {count:,} ({pct:.2f}%), 平均權重 = {avg_weight:.4f}")
        print(f"\n說明:")
        print(f"  - 類別平衡權重用於平衡不同類別的樣本分布")
        print(f"  - 少數類別的樣本會獲得更高的權重")
        print(f"  - 多數類別的樣本會獲得較低的權重")
        print(f"  - 權重計算公式: w = n_samples / (n_classes × n_class_samples)")
        print("=" * 60)
        
        return class_weights_series

    def get_time_decay_linear(self, tW: pd.Series, 
                              clf_last_w: float = 1.0) -> pd.Series:
        """
        線性時間衰減（基於 Snippet 4.11: getTimeDecay）
        
        實現原理：
        ---------
        應用分段線性衰減到觀測的唯一性（tW）：
        
        d[x] = max{0, a + bx}
        
        邊界條件：
        1. d[Σ_{i=1}^I ū_i] = 1  （最新樣本權重為 1）
        2. d[0] = clf_last_w    （最舊樣本權重為 clf_last_w）
        
        求解：
        a = 1 - b × Σ_{i=1}^I ū_i
        b = (1 - clf_last_w) / Σ_{i=1}^I ū_i  （當 clf_last_w >= 0）
        
        注意：時間衰減是基於累積唯一性 x ∈ [0, Σ_{i=1}^I ū_i]，
        而非時間順序，因為在存在冗餘觀測值的情況下，按時間順序
        衰減會使權重降低太快。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        clf_last_w : float
            最舊樣本的權重（預設 1.0，表示不衰減）
            - c = 1: 無時間衰減
            - 0 < c < 1: 線性衰減，所有樣本權重 > 0
            - c = 0: 最舊樣本權重為 0
            - c < 0: 最舊的 |c|×T 部分樣本權重為 0
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clfW = tW.sort_index().cumsum()
        
        if clfW.iloc[-1] > 0:
            if clf_last_w >= 0:
                # 線性衰減：d[0] = clf_last_w, d[Σū] = 1
                slope = (1.0 - clf_last_w) / clfW.iloc[-1]
            else:
                # 當 clf_last_w < 0 時的特殊處理
                slope = 1 / ((clf_last_w + 1) * clfW.iloc[-1])
        else:
            slope = 0
        
        const = 1.0 - slope * clfW.iloc[-1]
        clfW = const + slope * clfW
        clfW[clfW < 0] = 0
        
        self.time_decay = clfW
        
        print("=" * 60)
        print("📊 線性時間衰減")
        print("=" * 60)
        print(f"時間衰減參數: const={const:.4f}, slope={slope:.6f}")
        print(f"最舊樣本權重 (c): {clf_last_w:.4f}")
        print(f"最新樣本權重: 1.0")
        print(f"平均時間衰減權重: {clfW.mean():.4f}")
        print(f"\n說明:")
        if clf_last_w == 1.0:
            print(f"  - c = 1: 無時間衰減")
        elif 0 < clf_last_w < 1:
            print(f"  - 0 < c < 1: 線性衰減，所有樣本權重 > 0")
        elif clf_last_w == 0:
            print(f"  - c = 0: 最舊樣本權重為 0")
        elif clf_last_w < 0:
            print(f"  - c < 0: 最舊的 |c|×T 部分樣本權重為 0")
        print(f"  - 衰減基於累積唯一性，而非時間順序")
        print("=" * 60)
        
        return clfW
    
    def get_time_decay_exp(self, tW: pd.Series, 
                          decay_rate: float = 1.0,
                          percent_of_zero_wts: float = 0.0) -> pd.Series:
        """
        指數時間衰減
        
        實現原理：
        ---------
        使用指數函數進行時間衰減：
        
        d[x] = exp((decay_rate - 1) × (Σū - x))
        
        其中 x 是累積唯一性。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        decay_rate : float
            衰減率（>1 表示衰減更快，<1 表示衰減更慢）
        percent_of_zero_wts : float
            最舊樣本中設為 0 的比例
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clf_w = tW.sort_index().cumsum()
        last_value = clf_w.iloc[-1]
        
        # 計算指數衰減權重
        out_wts = []
        zero_threshold = int(round(len(clf_w) * percent_of_zero_wts))
        
        for i in range(len(clf_w)):
            if i < zero_threshold:
                out_wts.append(0.0)
            else:
                # 指數衰減：exp((decay_rate - 1) * (last_value - current_value))
                decay_factor = np.exp((decay_rate - 1.0) * (last_value - clf_w.iloc[i]))
                out_wts.append(decay_factor)
        
        time_decay = pd.Series(out_wts, index=clf_w.index)
        self.time_decay = time_decay
        
        print("=" * 60)
        print("📊 指數時間衰減統計")
        print("=" * 60)
        print(f"衰減率 (decay_rate): {decay_rate}")
        print(f"零權重樣本比例: {percent_of_zero_wts:.2%}")
        print(f"平均時間衰減權重: {time_decay.mean():.4f}")
        print(f"時間衰減分布:")
        print(time_decay.describe())
        print("=" * 60)
        
        return time_decay
    
    def compute_sample_weights(self, 
                              t1: pd.Series,
                              close_idx: pd.DatetimeIndex,
                              labels: Optional[pd.Series] = None,
                              close: Optional[pd.Series] = None,
                              use_uniqueness: bool = True,
                              use_time_decay: bool = True,
                              use_return_attribution: bool = False,
                              use_class_balance: bool = False,
                              time_decay_type: str = 'linear',
                              time_decay_params: dict = None) -> pd.Series:
        """
        計算最終樣本權重（可選擇使用哪些機制）
        
        權重組合方式：
        ------------
        1. 基礎權重 = 唯一性（如果啟用）
        2. 可選：× 報酬歸因權重（如果啟用）
        3. 可選：× 時間衰減（如果啟用）
        4. 可選：× 類別平衡權重（如果啟用）
        
        最終權重 = uniqueness × (return_attribution) × (time_decay) × (class_balance)
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        close_idx : pd.DatetimeIndex
            價格數據的索引
        labels : pd.Series, optional
            標籤序列（用於計算類別平衡權重，index 必須與 t1 一致）
        close : pd.Series, optional
            價格序列（用於計算報酬歸因）
        use_uniqueness : bool
            是否使用唯一性權重（預設 True）
        use_time_decay : bool
            是否使用時間衰減（預設 True）
        use_return_attribution : bool
            是否使用報酬歸因權重（預設 False）
        use_class_balance : bool
            是否使用類別平衡權重（預設 False）
        time_decay_type : str
            時間衰減類型：'linear' 或 'exp'（預設 'linear'）
        time_decay_params : dict, optional
            時間衰減參數
            - linear: {'clf_last_w': 0.5}
            - exp: {'decay_rate': 1.2, 'percent_of_zero_wts': 0.0}
            
        Returns:
        --------
        sample_weights : pd.Series
            最終樣本權重（如果都不使用，返回全為 1 的權重）
        """
        print("=" * 60)
        print("📊 樣本權重計算配置")
        print("=" * 60)
        print(f"使用唯一性: {use_uniqueness}")
        print(f"使用報酬歸因: {use_return_attribution}")
        print(f"使用時間衰減: {use_time_decay}")
        print(f"使用類別平衡: {use_class_balance}")
        if use_time_decay:
            print(f"時間衰減類型: {time_decay_type}")
        print("=" * 60)
        
        # 如果都不使用，返回均勻權重
        if not use_uniqueness and not use_time_decay and not use_return_attribution and not use_class_balance:
            sample_weights = pd.Series(1.0, index=t1.index)
            self.sample_weights = sample_weights
            
            print("=" * 60)
            print("📊 樣本權重統計（均勻權重）")
            print("=" * 60)
            print(f"總樣本數: {len(sample_weights):,}")
            print(f"所有樣本權重: 1.0")
            print("=" * 60)
            return sample_weights
        
        # 計算並發度（唯一性和報酬歸因都需要）
        if use_uniqueness or use_return_attribution:
            num_co_events = self.compute_num_co_events(close_idx, t1)
        else:
            num_co_events = None
        
        # 計算唯一性（如果需要）
        if use_uniqueness:
            uniqueness = self.compute_uniqueness(t1, num_co_events)
        else:
            uniqueness = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用唯一性，設為 1.0")
        
        # 計算報酬歸因（如果需要）
        if use_return_attribution:
            if close is None:
                raise ValueError("使用報酬歸因時需要提供 close 參數")
            return_weights = self.compute_return_attribution(close, t1, num_co_events)
        else:
            return_weights = pd.Series(1.0, index=t1.index)
        
        # 計算時間衰減（如果需要）
        if use_time_decay:
            if time_decay_params is None:
                time_decay_params = {}
            
            if time_decay_type == 'linear':
                clf_last_w = time_decay_params.get('clf_last_w', 1.0)
                time_decay = self.get_time_decay_linear(uniqueness, clf_last_w)
            elif time_decay_type == 'exp':
                decay_rate = time_decay_params.get('decay_rate', 1.0)
                percent_of_zero_wts = time_decay_params.get('percent_of_zero_wts', 0.0)
                time_decay = self.get_time_decay_exp(uniqueness, decay_rate, percent_of_zero_wts)
            else:
                raise ValueError(f"不支援的時間衰減類型: {time_decay_type}")
        else:
            time_decay = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用時間衰減，設為 1.0")
        
        # 計算類別平衡權重（如果需要）
        if use_class_balance:
            if labels is None:
                raise ValueError("使用類別平衡時需要提供 labels 參數")
            # 確保 labels 的 index 與 t1 一致
            if not labels.index.equals(t1.index):
                # 嘗試對齊
                common_idx = labels.index.intersection(t1.index)
                if len(common_idx) == 0:
                    raise ValueError("labels 和 t1 的 index 沒有交集")
                labels = labels.loc[common_idx]
                t1 = t1.loc[common_idx]
                uniqueness = uniqueness.loc[common_idx]
                return_weights = return_weights.loc[common_idx]
                time_decay = time_decay.loc[common_idx]
            class_balance_weights = self.compute_class_balance_weight(labels)
        else:
            class_balance_weights = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用類別平衡，設為 1.0")
        
        # 計算最終權重（所有權重相乘）
        common_idx = (uniqueness.index
                     .intersection(time_decay.index)
                     .intersection(return_weights.index)
                     .intersection(class_balance_weights.index))
        
        sample_weights = (uniqueness.loc[common_idx] * 
                          return_weights.loc[common_idx] * 
                          time_decay.loc[common_idx] *
                          class_balance_weights.loc[common_idx])
        
        self.sample_weights = sample_weights
        
        print("=" * 60)
        print("📊 樣本權重統計（最終）")
        print("=" * 60)
        print(f"總樣本數: {len(sample_weights):,}")
        print(f"平均權重: {sample_weights.mean():.4f}")
        print(f"權重分布:")
        print(sample_weights.describe())
        print("=" * 60)
        
        return sample_weights
    
    def plot_concurrency_vs_volatility(self, daily_vol: pd.Series):
        """
        繪製並發度 vs 波動率的關係圖
        
        參數:
        -----------
        daily_vol : pd.Series
            日波動率序列
        """
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        if self.num_co_events is None:
            print("⚠️ 請先執行 compute_num_co_events()")
            return
        
        # 對齊數據（使用原始時間點）
        to_plot = pd.DataFrame({
            'conc_events': self.num_co_events,
            'daily_vol': daily_vol
        }).dropna()
        
        if len(to_plot) == 0:
            print("⚠️ 對齊後無有效數據")
            return
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 10))
        
        # 1. 時間序列圖
        ax1 = axes[0]
        ax1_twin = ax1.twinx()
        
        line1 = ax1.plot(to_plot.index, to_plot['conc_events'], 
                        label='Concurrent Events', color='blue', linewidth=1.5)
        line2 = ax1_twin.plot(to_plot.index, to_plot['daily_vol'], 
                            label='Daily Volatility', color='red', linewidth=1.5, alpha=0.7)
        
        ax1.set_xlabel('Time', fontsize=12)
        ax1.set_ylabel('Concurrent Events', color='blue', fontsize=12)
        ax1_twin.set_ylabel('Daily Volatility', color='red', fontsize=12)
        ax1.set_title('Concurrent Events vs Daily Volatility', fontsize=14)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1_twin.tick_params(axis='y', labelcolor='red')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='upper left')
        ax1_twin.legend(loc='upper right')
        
        # 2. 散點圖
        sns.regplot(x=to_plot['conc_events'], y=to_plot['daily_vol'], 
                ax=axes[1], scatter_kws={'alpha': 0.6, 's': 50})
        axes[1].set_xlabel('Concurrent Events', fontsize=12)
        axes[1].set_ylabel('Daily Volatility', fontsize=12)
        axes[1].set_title('Concurrent Events vs Daily Volatility (Scatter Plot)', fontsize=14)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 打印相關性
        correlation = to_plot['conc_events'].corr(to_plot['daily_vol'])
        print(f"\n📊 並發度與波動率的相關係數: {correlation:.4f}")
    
    def plot_weights_distribution(self):
        """繪製權重分布圖"""
        import matplotlib.pyplot as plt
        
        if self.sample_weights is None:
            print("⚠️ 請先執行 compute_sample_weights()")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. 唯一性分布
        if self.uniqueness is not None:
            self.uniqueness.hist(bins=50, ax=axes[0, 0], alpha=0.7, color='blue')
            axes[0, 0].set_title('Uniqueness Distribution')
            axes[0, 0].set_xlabel('Uniqueness')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].grid(True, alpha=0.3)
        
        # 2. 時間衰減分布
        if self.time_decay is not None:
            self.time_decay.hist(bins=50, ax=axes[0, 1], alpha=0.7, color='green')
            axes[0, 1].set_title('Time Decay Distribution')
            axes[0, 1].set_xlabel('Time Decay Weight')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].grid(True, alpha=0.3)
        
        # 3. 樣本權重分布
        self.sample_weights.hist(bins=50, ax=axes[1, 0], alpha=0.7, color='orange')
        axes[1, 0].set_title('Sample Weights Distribution')
        axes[1, 0].set_xlabel('Sample Weight')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. 權重隨時間變化
        if len(self.sample_weights) > 0:
            sorted_weights = self.sample_weights.sort_index()
            axes[1, 1].plot(sorted_weights.index, sorted_weights.values, 
                           linewidth=1, alpha=0.7, color='purple')
            axes[1, 1].set_title('Sample Weights Over Time')
            axes[1, 1].set_xlabel('Time')
            axes[1, 1].set_ylabel('Sample Weight')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def get_uniqueness_autocorr(self, lag: int = 1) -> float:
        """
        計算唯一性的序列相關性（AR(1)）
        
        參數:
        -----------
        lag : int
            滯後期數（預設 1）
            
        Returns:
        --------
        autocorr : float
            序列相關性
        """
        if self.uniqueness is None:
            print("⚠️ 請先執行 compute_uniqueness()")
            return None
        
        autocorr = self.uniqueness.autocorr(lag=lag)
        
        print("=" * 60)
        print("📊 唯一性序列相關性")
        print("=" * 60)
        print(f"AR({lag}) 自相關係數: {autocorr:.4f}")
        
        # 統計顯著性檢驗（簡化版）
        n = len(self.uniqueness)
        critical_value = 1.96 / np.sqrt(n)  # 95% 信心水準
        
        if abs(autocorr) > critical_value:
            print(f"✅ 統計顯著 (|{autocorr:.4f}| > {critical_value:.4f})")
            print(f"\n解釋:")
            print(f"  - 唯一性存在序列相關性，表示市場狀態具有持續性")
            print(f"  - 這在金融數據中是常見且預期的現象")
            print(f"  - 時間衰減機制有助於降低舊數據的影響")
        else:
            print(f"❌ 統計不顯著 (|{autocorr:.4f}| <= {critical_value:.4f})")
        print("=" * 60)
        
        return autocorr

In [ ]:
# =====================================================
# 完整範例：樣本權重計算（所有選項）
# =====================================================

sample_weight = SampleWeight()

# =====================================================
# 選項 A: 標準配置（唯一性 + 線性時間衰減）
# =====================================================
final_weights_A = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=True,
    use_time_decay=True,
    time_decay_type='linear',
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    time_decay_params={'clf_last_w': 0.5}
)

# =====================================================
# 選項 B: 完整配置（唯一性 + 報酬歸因 + 時間衰減）
# =====================================================
final_weights_B = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    close=nas100_raw['Close'],
    use_uniqueness=True,
    use_time_decay=True,
    use_return_attribution=True,  # 加入報酬歸因
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    time_decay_type='linear',
    time_decay_params={'clf_last_w': 0.5}
)

# =====================================================
# 選項 C: 指數時間衰減
# =====================================================
final_weights_C = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=False,
    use_time_decay=False,
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    time_decay_type='linear',
    time_decay_params={'clf_last_w': 0.5}
)

# =====================================================
# 選項 D: 只使用唯一性（最簡單）
# =====================================================
final_weights_D = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=True,
    use_time_decay=False,
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    use_return_attribution=False
)

# =====================================================
# 選項 E: 只使用報酬歸因
# =====================================================
final_weights_E = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    close=nas100_raw['Close'],
    use_uniqueness=False,
    use_time_decay=False,
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    use_return_attribution=True
)

# =====================================================
# 選項 F: 均勻權重（基準對照）
# =====================================================
final_weights_F = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=False,
    use_time_decay=False,
    labels=bins['bin'],  # ✅ 加入 labels 參數
    use_class_balance=True,  # ✅ 啟用類別平衡
    use_return_attribution=False
)

# =====================================================
# 比較不同配置的權重統計
# =====================================================
print("\n" + "="*60)
print("📊 不同配置的權重比較")
print("="*60)

comparison = pd.DataFrame({
    'A: 唯一性+線性衰減': final_weights_A.describe(),
    'B: 唯一性+報酬歸因+線性衰減': final_weights_B.describe(),
    'C: 唯一性+指數衰減': final_weights_C.describe(),
    'D: 只唯一性': final_weights_D.describe(),
    'E: 只報酬歸因': final_weights_E.describe(),
    'F: 均勻權重': final_weights_F.describe()
})

print(comparison)

# =====================================================
# 視覺化比較
# =====================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 權重分布比較
axes[0, 0].hist(final_weights_A, bins=50, alpha=0.5, label='A: Uniqueness+Linear', color='blue')
axes[0, 0].hist(final_weights_B, bins=50, alpha=0.5, label='B: 完整配置', color='red')
axes[0, 0].hist(final_weights_D, bins=50, alpha=0.5, label='D: 只唯一性', color='green')
axes[0, 0].set_title('Weight Distribution Comparison')
axes[0, 0].set_xlabel('Sample Weight')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 權重隨時間變化
sorted_A = final_weights_A.sort_index()
axes[0, 1].plot(sorted_A.index, sorted_A.values, label='A: 唯一性+線性', alpha=0.7)
axes[0, 1].set_title('Weights Over Time')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Weight')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 權重分位數比較
weights_list = [final_weights_A, final_weights_B, final_weights_D]
labels_list = ['A: 唯一性+線性', 'B: 完整配置', 'D: 只唯一性']
percentiles = [25, 50, 75, 90, 95]

for i, (w, label) in enumerate(zip(weights_list, labels_list)):
    values = [w.quantile(p/100) for p in percentiles]
    axes[1, 0].plot(percentiles, values, marker='o', label=label)

axes[1, 0].set_title('Weight Percentiles Comparison')
axes[1, 0].set_xlabel('Percentile')
axes[1, 0].set_ylabel('Weight Value')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 權重統計摘要
stats_data = {
    'Mean': [w.mean() for w in weights_list],
    'Std': [w.std() for w in weights_list],
    'Min': [w.min() for w in weights_list],
    'Max': [w.max() for w in weights_list]
}
stats_df = pd.DataFrame(stats_data, index=labels_list)
stats_df.plot(kind='bar', ax=axes[1, 1], alpha=0.7)
axes[1, 1].set_title('Weight Statistics Comparison')
axes[1, 1].set_ylabel('Value')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# =====================================================
# 最終選擇（根據需求選擇一個）
# =====================================================
# 推薦：使用選項 A 或 B
final_weights = final_weights_B  # 或 final_weights_B

print("\n" + "="*60)
print("✅ 最終選擇的權重配置")
print("="*60)
print(f"配置: 唯一性 + 線性時間衰減")
print(f"樣本數: {len(final_weights):,}")
print(f"平均權重: {final_weights.mean():.4f}")
print(f"權重範圍: [{final_weights.min():.4f}, {final_weights.max():.4f}]")
print("="*60)

## 步驟5 ： 分階差平穩價格特徵提取

主商品

In [ ]:
# =====================================================
# 完整代碼：為所有商品產生分數階差分特徵
# =====================================================

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 2. 修正後的 FractionalDiff class（確保保留所有 index）
# =====================================================

class FractionalDiff:
    """分數階差分特徵生成"""
    
    def __init__(self):
        self.optimal_d = None
        self.fracdiff_series = None
        self.features = None
        
    def get_weights(self, d: float, size: int) -> np.ndarray:
        """計算權重（擴展窗口）"""
        w = [1.0]
        for k in range(1, size):
            w_ = -w[-1] / k * (d - k + 1)
            w.append(float(w_))
        return np.array(w[::-1]).reshape(-1, 1)
    
    def get_weights_FFD(self, d: float, thres: float = 1e-5) -> np.ndarray:
        """計算 FFD 權重（固定窗口）"""
        w = [1.0]
        k = 1
        while True:
            w_ = -w[-1] / k * (d - k + 1)
            if abs(w_) < thres:
                break
            w.append(float(w_))
            k += 1
            if k > 10000:
                break
        return np.array(w[::-1]).reshape(-1, 1)
    
    def frac_diff_FFD(self, series: pd.Series, d: float, thres: float = 1e-5) -> pd.Series:
        """
        分數階差分（固定窗口方法，FFD）
        
        重要：保留所有原始 index，只在前 width 個點設為 NaN
        """
        # 1. 計算權重（固定窗口）
        w = self.get_weights_FFD(d, thres)
        width = len(w) - 1
        
        if width < 1:
            return pd.Series(index=series.index, dtype=float)
        
        # 2. 應用權重到數值
        # 重要：先填充 NaN，但保留原始 index
        series_filled = series.fillna(method='ffill')
        
        # 建立結果 Series，使用原始 series 的完整 index
        df_ = pd.Series(index=series.index, dtype=float)
        
        # 3. 從 width 開始計算（前 width 個點會是 NaN）
        valid_count = 0
        for iloc1 in range(width, len(series_filled)):
            if iloc1 >= len(series.index):
                break
                
            loc0 = series_filled.index[iloc1 - width]
            loc1 = series_filled.index[iloc1]
            
            # 檢查原始 series 在 loc1 是否有有效值
            if loc1 not in series.index or not np.isfinite(series.loc[loc1]):
                continue
            
            # 使用固定窗口：[X_{t-width}, X_{t-width+1}, ..., X_t]
            window_data = series_filled.loc[loc0:loc1].values
            
            if len(window_data) == len(w):
                result = np.dot(w.flatten(), window_data)
                df_[loc1] = result
                valid_count += 1
        
        return df_
    
    def frac_diff(self, series: pd.Series, d: float, thres: float = 0.01) -> pd.Series:
        """分數階差分（擴展窗口）"""
        w = self.get_weights(d, series.shape[0])
        w_ = np.cumsum(np.abs(w))
        w_ /= w_[-1]
        skip = w_[w_ > thres].shape[0]
        
        series_filled = series.fillna(method='ffill')
        df_ = pd.Series(index=series.index, dtype=float)
        
        for iloc in range(skip, series_filled.shape[0]):
            loc = series_filled.index[iloc]
            if not np.isfinite(series.loc[loc]):
                continue
            
            window_data = series_filled.loc[:loc].values
            w_subset = w[-(iloc + 1):, :].flatten()
            
            if len(window_data) == len(w_subset):
                df_[loc] = np.dot(w_subset, window_data)
        
        return df_
    
    def find_optimal_d(self, series: pd.Series, 
                      d_range: Tuple[float, float] = (0, 1),
                      d_step: float = 0.05,
                      method: str = 'FFD',
                      thres: float = 1e-5,
                      target_pvalue: float = 0.05,
                      min_corr: float = 0.5) -> dict:
        """找出最適合的 d 值"""
        from statsmodels.tsa.stattools import adfuller
        from scipy.stats import jarque_bera
        
        results = []
        d_values = np.arange(d_range[0], d_range[1] + d_step, d_step)
        
        for d in d_values:
            try:
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                jb_stat, jb_pvalue = jarque_bera(fracdiff_series)[:2]
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'jb_stat': jb_stat,
                    'jb_pvalue': jb_pvalue,
                    'is_stationary': p_value < target_pvalue,
                    'meets_corr': corr >= min_corr,
                    'sample_size': len(fracdiff_series)
                })
                
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            return None
        
        valid_results = results_df[
            (results_df['is_stationary']) & 
            (results_df['meets_corr'])
        ]
        
        if len(valid_results) > 0:
            optimal = valid_results.loc[valid_results['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        else:
            optimal = results_df.loc[results_df['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        
        return {
            'optimal_d': self.optimal_d,
            'results': results_df,
            'optimal_stats': optimal.to_dict()
        }

    def find_min_d_adf(self, series: pd.Series,
                    d_range: Tuple[float, float] = (0, 1),
                    n_points: int = 11,
                    method: str = 'FFD',
                    thres: float = 0.01,
                    target_pvalue: float = 0.1) -> dict:
        """
        尋找最小 d 值（基於 Snippet 5.4）
        
        這個方法與 find_optimal_d 類似，但專門用於尋找滿足平穩性條件的最小 d 值。
        這是 AFML 書中 Snippet 5.4 的實現。
        
        參數:
        -----------
        series : pd.Series
            價格序列
        d_range : tuple
            d 值搜尋範圍 (min, max)
        n_points : int
            測試的 d 值數量（對應 np.linspace(d_range[0], d_range[1], n_points)）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值（FFD 方法）或累積權重閾值（expanding 方法）
        target_pvalue : float
            目標 p-value（ADF 檢驗）
            
        Returns:
        --------
        result : dict
            包含以下鍵值：
            - 'min_d': 最小 d 值（如果找到）
            - 'results': DataFrame，包含所有測試結果
            - 'min_d_stats': 最小 d 值的統計資訊
        """
        from statsmodels.tsa.stattools import adfuller
        
        results = []
        d_values = np.linspace(d_range[0], d_range[1], n_points)
        
        print(f"🔍 搜尋最小 d 值 (範圍: {d_range}, 測試點數: {n_points})")
        print(f"   方法: {method}, 閾值: {thres}, 目標 p-value: {target_pvalue}")
        
        for i, d in enumerate(d_values):
            try:
                # 計算分數階差分
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                # ADF 檢驗
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                # 計算與原始序列的相關性
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                is_stationary = p_value < target_pvalue
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'is_stationary': is_stationary,
                    'sample_size': len(fracdiff_series)
                })
                
                if (i + 1) % 5 == 0 or i == len(d_values) - 1:
                    print(f"   進度: {i+1}/{len(d_values)} (d={d:.3f}, p={p_value:.4f}, stationary={is_stationary})")
                    
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            print("❌ 未找到任何有效結果")
            return None
        
        # 尋找滿足平穩性條件的最小 d 值
        stationary_results = results_df[results_df['is_stationary'] == True]
        
        if len(stationary_results) > 0:
            min_d_row = stationary_results.loc[stationary_results['d'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n✅ 找到最小 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f}")
            print(f"   相關性: {min_d_row['correlation']:.4f}")
        else:
            # 如果沒有滿足條件的，選擇 p-value 最小的
            min_d_row = results_df.loc[results_df['p_value'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n⚠️ 未找到滿足平穩性條件的 d 值")
            print(f"   使用 p-value 最小的 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f} (目標: < {target_pvalue})")
        
        return {
            'min_d': min_d,
            'results': results_df,
            'min_d_stats': min_d_row.to_dict()
        }

    def _plot_d_search_results(self, result: dict):
        """
        繪製 d 值搜尋結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 或 find_optimal_d 的返回結果
        """
        import matplotlib.pyplot as plt
        
        if result is None or 'results' not in result:
            print("⚠️ 無效的結果")
            return
        
        results_df = result['results']
        min_d = result.get('min_d') or result.get('optimal_d')
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. ADF p-value vs d
        axes[0, 0].plot(results_df['d'], results_df['p_value'], 'o-', linewidth=2, markersize=6)
        if min_d is not None:
            min_d_pvalue = results_df[results_df['d'] == min_d]['p_value'].values[0]
            axes[0, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 0].scatter([min_d], [min_d_pvalue], color='red', s=100, zorder=5)
        axes[0, 0].axhline(0.05, color='gray', linestyle=':', alpha=0.5, label='p=0.05')
        axes[0, 0].axhline(0.1, color='gray', linestyle=':', alpha=0.5, label='p=0.1')
        axes[0, 0].set_xlabel('d', fontsize=12)
        axes[0, 0].set_ylabel('ADF p-value', fontsize=12)
        axes[0, 0].set_title('ADF Test p-value vs d', fontsize=14)
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. ADF Statistic vs d
        axes[0, 1].plot(results_df['d'], results_df['adf_stat'], 'o-', linewidth=2, markersize=6, color='green')
        if min_d is not None:
            min_d_stat = results_df[results_df['d'] == min_d]['adf_stat'].values[0]
            axes[0, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 1].scatter([min_d], [min_d_stat], color='red', s=100, zorder=5)
        axes[0, 1].axhline(0, color='gray', linestyle=':', alpha=0.5)
        axes[0, 1].set_xlabel('d', fontsize=12)
        axes[0, 1].set_ylabel('ADF Statistic', fontsize=12)
        axes[0, 1].set_title('ADF Test Statistic vs d', fontsize=14)
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Correlation vs d
        axes[1, 0].plot(results_df['d'], results_df['correlation'], 'o-', linewidth=2, markersize=6, color='orange')
        if min_d is not None:
            min_d_corr = results_df[results_df['d'] == min_d]['correlation'].values[0]
            axes[1, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 0].scatter([min_d], [min_d_corr], color='red', s=100, zorder=5)
        axes[1, 0].set_xlabel('d', fontsize=12)
        axes[1, 0].set_ylabel('Correlation with Original', fontsize=12)
        axes[1, 0].set_title('Correlation vs d', fontsize=14)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Stationarity Status
        if 'is_stationary' in results_df.columns:
            colors = ['red' if not stat else 'green' for stat in results_df['is_stationary']]
            axes[1, 1].scatter(results_df['d'], results_df['is_stationary'].astype(int), 
                            c=colors, s=100, alpha=0.6)
            if min_d is not None:
                axes[1, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 1].set_xlabel('d', fontsize=12)
            axes[1, 1].set_ylabel('Is Stationary (1=Yes, 0=No)', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
            axes[1, 1].set_ylim(-0.1, 1.1)
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'No stationarity data', 
                            ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
        
        plt.tight_layout()
        plt.show()

    def _plot_min_d_results(self, result: dict, d: float):
        """
        繪製使用最小 d 值後的結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 的返回結果
        d : float
            使用的 d 值
        """
        import matplotlib.pyplot as plt
        
        if result is None:
            print("⚠️ 無效的結果")
            return
        
        # 這個方法可以繪製使用 d 值後的時間序列對比
        # 但需要實際的分數階差分序列，所以這裡只顯示統計資訊
        min_d_stats = result.get('min_d_stats', {})
        
        print("\n" + "=" * 60)
        print("📊 最小 d 值統計資訊")
        print("=" * 60)
        print(f"使用的 d 值: {d:.4f}")
        if min_d_stats:
            print(f"ADF Statistic: {min_d_stats.get('adf_stat', 'N/A'):.4f}")
            print(f"ADF p-value: {min_d_stats.get('p_value', 'N/A'):.4f}")
            print(f"與原始序列相關性: {min_d_stats.get('correlation', 'N/A'):.4f}")
            print(f"樣本數: {min_d_stats.get('sample_size', 'N/A'):,}")
            print(f"是否平穩: {min_d_stats.get('is_stationary', 'N/A')}")
        print("=" * 60)

    def generate_features(self, df: pd.DataFrame, event_indices: pd.DatetimeIndex,
                      price_col: str = 'Close', d: Optional[float] = None,
                      method: str = 'FFD', thres: float = 1e-5) -> pd.DataFrame:
        """
        為事件點生成分數階差分特徵
        
        參數:
        -----------
        df : pd.DataFrame
            包含價格資料的 DataFrame
        event_indices : pd.DatetimeIndex
            事件時間點
        price_col : str
            價格欄位名稱
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
            
        Returns:
        --------
        features : pd.DataFrame
            分數階差分特徵（index 為 event_indices）
        """
        if price_col not in df.columns:
            raise ValueError(f"找不到價格欄位: {price_col}")
        
        # 使用指定的 d 值或最優 d 值
        if d is None:
            if self.optimal_d is None:
                raise ValueError("請先執行 find_optimal_d() 或 find_min_d_adf()，或指定 d 參數")
            d = self.optimal_d
        
        print(f"📊 生成分數階差分特徵 (d={d:.4f}, method={method})")
        
        # 計算整個序列的分數階差分
        price_series = df[price_col]
        
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d, thres)
        
        # 提取事件點的特徵值
        features = pd.DataFrame(index=event_indices)
        features['fracdiff'] = fracdiff_series.loc[event_indices].values
        
        # 統計資訊
        valid_count = features['fracdiff'].notna().sum()
        print(f"✅ 特徵生成完成")
        print(f"   事件數: {len(event_indices):,}")
        print(f"   有效特徵數: {valid_count:,}")
        print(f"   NaN 數: {len(event_indices) - valid_count:,}")
        
        return features
    
    def plot_price_vs_fracdiff(self, price_series: pd.Series, 
                            d: Optional[float] = None,
                            method: str = 'FFD',
                            thres: float = 1e-5,
                            n_points: Optional[int] = None,
                            normalize: bool = False):
        """
        繪製原始價格與分數階差分特徵的對比圖
        
        參數:
        -----------
        price_series : pd.Series
            原始價格序列
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d 或預設值 0.5）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
        n_points : int, optional
            如果指定，只繪製最近 n_points 個點（用於大量數據時）
        normalize : bool
            是否標準化後繪製在同一張圖上（預設 False，分開兩個子圖）
            
        Returns:
        --------
        fracdiff_series : pd.Series
            計算出的分數階差分序列
        """
        import matplotlib.pyplot as plt
        
        # 確定使用的 d 值
        if d is None:
            if self.optimal_d is not None:
                d_value = self.optimal_d
            else:
                d_value = 0.5
                print(f"⚠️ 未指定 d 值，使用預設值 {d_value}")
        else:
            d_value = d
        
        # 計算整個序列的分數階差分
        print(f"📊 計算分數階差分 (d={d_value:.4f}, method={method})...")
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d_value, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d_value, thres)
        
        # 選擇要繪製的數據範圍
        if n_points is not None and len(price_series) > n_points:
            plot_idx = price_series.index[-n_points:]
            price_plot = price_series.loc[plot_idx]
            fracdiff_plot = fracdiff_series.loc[plot_idx]
            title_suffix = f" (Last {n_points:,} Points)"
        else:
            plot_idx = price_series.index
            price_plot = price_series
            fracdiff_plot = fracdiff_series
            title_suffix = ""
        
        # 繪製圖表
        if normalize:
            # 方法 1: 標準化後繪製在同一張圖上
            common_idx = plot_idx.intersection(fracdiff_plot.dropna().index)
            if len(common_idx) > 0:
                price_normalized = (price_plot.loc[common_idx] - price_plot.loc[common_idx].mean()) / price_plot.loc[common_idx].std()
                fracdiff_normalized = (fracdiff_plot.loc[common_idx] - fracdiff_plot.loc[common_idx].mean()) / fracdiff_plot.loc[common_idx].std()
                
                fig, ax = plt.subplots(figsize=(15, 6))
                
                ax.plot(common_idx, price_normalized, 
                    label='Original Price (Normalized)', linewidth=1.5, color='blue', alpha=0.7)
                ax.plot(common_idx, fracdiff_normalized, 
                    label='Fractional Differentiation (Normalized)', linewidth=1.5, color='red', alpha=0.7)
                ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.3)
                ax.set_xlabel('Time', fontsize=12)
                ax.set_ylabel('Normalized Value', fontsize=12)
                ax.set_title(f'Normalized Comparison: Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
                ax.legend(loc='upper left')
                ax.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        else:
            # 方法 2: 分開兩個子圖（預設）
            fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
            
            # 上圖：原始價格
            axes[0].plot(plot_idx, price_plot.values, 
                        label='Original Price', linewidth=1.5, color='blue', alpha=0.7)
            axes[0].set_ylabel('Price', fontsize=12)
            axes[0].set_title(f'Original Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
            axes[0].legend(loc='upper left')
            axes[0].grid(True, alpha=0.3)
            
            # 下圖：分數階差分特徵
            valid_fracdiff = fracdiff_plot.dropna()
            if len(valid_fracdiff) > 0:
                axes[1].plot(valid_fracdiff.index, valid_fracdiff.values, 
                            label='Fractional Differentiation', linewidth=1.5, color='red', alpha=0.7)
            axes[1].axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
            axes[1].set_xlabel('Time', fontsize=12)
            axes[1].set_ylabel('Fractional Differentiation', fontsize=12)
            axes[1].set_title('Fractional Differentiation Feature', fontsize=14)
            axes[1].legend(loc='upper left')
            axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # 統計資訊
        valid_count = fracdiff_series.notna().sum()
        print(f"✅ 繪圖完成")
        print(f"   原始數據點數: {len(price_series):,}")
        print(f"   分數階差分有效值: {valid_count:,}")
        print(f"   NaN 值: {len(price_series) - valid_count:,}")
        
        return fracdiff_series

其他商品

In [ ]:
fracdiff_features_all = pd.read_csv('fracdiff_features_all_products.csv')
if 'Date' in fracdiff_features_all.columns:
    fracdiff_features_all['Date'] = pd.to_datetime(fracdiff_features_all['Date'])
    fracdiff_features_all.set_index('Date', inplace=True)
fracdiff_features_all

## 步驟6 ： 其他特徵產生

技術指標

In [ ]:
import pandas as pd
import numpy as np

def calculate_technical_indicators(df):
    """15分鐘資料用的技術指標（四組尺度：SS=4H、S=9H、M=24H、L=72H）"""
    tech_df = pd.DataFrame(index=df.index)

    Open = df['Open']
    High = df['High']
    Low = df['Low']
    Close = df['Close']
    Volume = df['Volume']

    # 通用工具
    def wilders_rma(s, period):
        return s.ewm(alpha=1/period, adjust=False).mean()

    def true_range(high, low, close):
        prev_close = close.shift(1)
        return pd.concat([
            (high - low),
            (high - prev_close).abs(),
            (low - prev_close).abs()
        ], axis=1).max(axis=1)

    def mfi(high, low, close, volume, period=14):
        tp = (high + low + close) / 3.0
        rmf = tp * volume
        pos = (tp > tp.shift(1)).astype(int)
        neg = (tp < tp.shift(1)).astype(int)
        pos_mf = (rmf * pos).rolling(period).sum()
        neg_mf = (rmf * neg).rolling(period).sum()
        mr = pos_mf / neg_mf
        return 100 - (100 / (1 + mr))

    def dm_di_dx(high, low, close, period=14):
        up_move = high.diff()
        down_move = -low.diff()
        plus_dm = ((up_move > down_move) & (up_move > 0)) * up_move
        minus_dm = ((down_move > up_move) & (down_move > 0)) * down_move
        tr = true_range(high, low, close)
        atr = wilders_rma(tr, period)
        plus_di = 100 * wilders_rma(plus_dm, period) / atr
        minus_di = 100 * wilders_rma(minus_dm, period) / atr
        dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di)
        return plus_dm, minus_dm, plus_di, minus_di, dx

    # ========= TRANGE/ATR/NATR（加入 SS）=========
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    tech_df['TRANGE'] = true_range(High, Low, Close)
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        atr = wilders_rma(tech_df['TRANGE'], p)
        tech_df[f'ATR_{tag}'] = atr
        tech_df[f'NATR_{tag}'] = atr / Close * 100.0

    # ========= Momentum & Oscillators =========
    # MOM / ROC / ROCR（加入 SS）
    # 15分鐘資料：4H=16, 10H=40, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (40,'S'), (96,'M'), (288,'L')]:
        tech_df[f'MOM_{tag}'] = Close - Close.shift(p)
        tech_df[f'ROC_{tag}'] = Close.pct_change(p) * 100.0
        tech_df[f'ROCR_{tag}'] = Close / Close.shift(p)

    # MFI（加入 SS）
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        tech_df[f'MFI_{tag}'] = mfi(High, Low, Close, Volume, p)

    # ADX/DI/DM/DX（只保留 MINUS_DM）
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        _, minus_dm, _, _, _ = dm_di_dx(High, Low, Close, p)
        tech_df[f'MINUS_DM_{tag}'] = minus_dm

    # ========= Volume & Accumulation =========
    # Volume_SMA_20: 20個15分鐘K線（約5小時），保持不變
    tech_df['Volume_SMA_20'] = Volume.rolling(20).mean()
    tech_df['Volume_Ratio_20'] = Volume / tech_df['Volume_SMA_20']

    return tech_df

# 計算技術指標
tech_indicators = calculate_technical_indicators(nas100_raw)  # 使用15分鐘原始資料
print(f"技術指標計算完成，共 {len(tech_indicators.columns)} 個指標")

# 檢查資料品質
print(f"\n資料品質檢查:")
print(f"總筆數: {len(tech_indicators)}")
print(f"缺失值統計:")
missing_stats = tech_indicators.isnull().sum()
print(missing_stats[missing_stats > 0])

基礎統計

In [ ]:
volatility_indicator = pd.read_csv('NAS100_RS_Volatility.csv')
if 'Date' in volatility_indicator.columns:
    volatility_indicator['Date'] = pd.to_datetime(volatility_indicator['Date'])
    volatility_indicator.set_index('Date', inplace=True)
volatility_indicator

In [ ]:
stats_indicator = pd.read_csv('NAS100_Stats_Indicator.csv')
if 'Date' in stats_indicator.columns:
    stats_indicator['Date'] = pd.to_datetime(stats_indicator['Date'])
    stats_indicator.set_index('Date', inplace=True)
stats_indicator

量價行為

In [ ]:
market_indicator = pd.read_csv('NAS100_Market_Indicator.csv')
if 'Date' in market_indicator.columns:
    market_indicator['Date'] = pd.to_datetime(market_indicator['Date'])
    market_indicator.set_index('Date', inplace=True)
market_indicator

資訊熵

In [ ]:
# entropy_indicator = pd.read_csv('NAS100_Entropy_Indicator.csv')
# if 'Date' in entropy_indicator.columns:
#     entropy_indicator['Date'] = pd.to_datetime(entropy_indicator['Date'])
#     entropy_indicator.set_index('Date', inplace=True)
# entropy_indicator

市場微結構

In [ ]:
micro_indicator = pd.read_csv('NAS100_Micro_Indicator.csv')
if 'Date' in micro_indicator.columns:
    micro_indicator['Date'] = pd.to_datetime(micro_indicator['Date'])
    micro_indicator.set_index('Date', inplace=True)
micro_indicator

## 步驟6.5 ： 特徵雜訊篩除

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("📊 步驟6.5：特徵合併與篩選 Pipeline")
print("=" * 60)

# =====================================================
# 第1部分：特徵合併
# =====================================================
print(f"\n" + "=" * 60)
print("🔗 第1部分：特徵合併")
print("=" * 60)

# 第0步：指定數據
y = bins['bin']
t1 = events['t1']
sample_weight = final_weights

# 準備特徵矩陣 X
event_indices = events.index
features_list = []

print(f"\n事件索引數量: {len(event_indices):,}")

# =====================================================
# 列出所有特徵 DataFrame 及其欄位
# =====================================================
print(f"\n" + "=" * 60)
print("📋 檢查所有特徵 DataFrame 及其欄位...")
print("=" * 60)

feature_dataframes = {
    '1. fracdiff_features_all': 'fracdiff_features_all',
    '2. tech_indicators': 'tech_indicators',
    '3. volatility_indicator': 'volatility_indicator',
    '4. market_indicator': 'market_indicator',
    '5. entropy_indicator': 'entropy_indicator',
    '6. micro_indicator': 'micro_indicator',
    '7. stats_indicator': 'stats_indicator',
}

available_features = {}
for name, var_name in feature_dataframes.items():
    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    if var_name in globals():
        df_var = globals()[var_name]
        if isinstance(df_var, pd.DataFrame):
            print(f"✅ DataFrame 存在")
            print(f"   形狀: {df_var.shape}")
            print(f"   欄位數: {len(df_var.columns)}")
            print(f"   索引範圍: {df_var.index.min()} 至 {df_var.index.max()}")
            
            # 打印所有欄位名稱
            print(f"\n   欄位列表:")
            if len(df_var.columns) > 0:
                cols_list = list(df_var.columns)
                if len(cols_list) <= 20:
                    for i, col in enumerate(cols_list, 1):
                        print(f"     {i:3d}. {col}")
                else:
                    for i, col in enumerate(cols_list[:10], 1):
                        print(f"     {i:3d}. {col}")
                    print(f"     ... (省略 {len(cols_list) - 20} 個欄位) ...")
                    for i, col in enumerate(cols_list[-10:], len(cols_list) - 9):
                        print(f"     {i:3d}. {col}")
                    print(f"     總共 {len(cols_list)} 個欄位")
            else:
                print(f"     ⚠️ 沒有欄位")
            
            # 對齊到事件索引
            try:
                common_idx = df_var.index.intersection(event_indices)
                missing_idx = event_indices.difference(df_var.index)
                
                if len(missing_idx) > 0:
                    print(f"\n   ⚠️ 警告：有 {len(missing_idx)} 個事件索引在 DataFrame 中不存在")
                    print(f"      前5個缺失索引: {list(missing_idx[:5])}")
                
                if len(common_idx) > 0:
                    df_aligned = df_var.reindex(event_indices, method='ffill')
                    df_aligned = df_aligned.fillna(method='bfill')
                    df_aligned = df_aligned.fillna(0)
                    
                    if len(df_aligned.columns) > 0:
                        available_features[name] = df_aligned
                        print(f"\n   ✅ 對齊後: {len(df_aligned.columns)} 個特徵, {len(df_aligned)} 個樣本")
                        print(f"      有效值比例: {(df_aligned.notna().sum().sum() / (len(df_aligned) * len(df_aligned.columns)) * 100):.2f}%")
                    else:
                        print(f"\n   ⚠️ 對齊後沒有欄位")
                else:
                    print(f"\n   ❌ 沒有共同索引，無法對齊")
            except Exception as e:
                print(f"\n   ❌ 對齊失敗: {str(e)}")
        else:
            print(f"⚠️ 變數存在但不是 DataFrame (類型: {type(df_var)})")
    else:
        print(f"❌ 變數不存在")

print(f"\n" + "=" * 60)
print(f"📊 總結")
print("=" * 60)
print(f"找到 {len(available_features)} 個可用的特徵 DataFrame")

# =====================================================
# 添加 HL115% 和 突破signal價 特徵
# =====================================================
print(f"\n" + "=" * 60)
print("📊 添加 HL115% 和 突破signal價 特徵...")
print("=" * 60)

if 'strategy' in globals() and hasattr(strategy, 'signals') and strategy.signals is not None:
    signal_features = pd.DataFrame(index=strategy.signals.index)
    
    if 'HL115%' in strategy.signals.columns:
        signal_features['HL115%'] = strategy.signals['HL115%']
        print(f"✅ 加入 HL115% 特徵")
    else:
        print(f"⚠️ 找不到 HL115% 欄位")
    
    if '突破signal價%' in strategy.signals.columns:
        signal_features['突破signal價%'] = strategy.signals['突破signal價%']
        print(f"✅ 加入 突破signal價% 特徵")
    else:
        print(f"⚠️ 找不到 突破signal價% 欄位")
    
    if len(signal_features.columns) > 0:
        signal_features_aligned = signal_features.reindex(event_indices)
        features_list.append(signal_features_aligned)
        print(f"✅ 成功添加 {len(signal_features.columns)} 個信號特徵到特徵列表")
    else:
        print(f"⚠️ 沒有成功提取任何信號特徵")
else:
    print(f"⚠️ strategy 不存在或尚未計算指標，跳過信號特徵")

# =====================================================
# 合併所有特徵
# =====================================================
print(f"\n" + "=" * 60)
print("🔗 合併所有特徵...")
print("=" * 60)

for name, df_feat in available_features.items():
    numeric_cols = df_feat.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        features_list.append(df_feat[numeric_cols])
        print(f"✅ 加入 {name}: {len(numeric_cols)} 個數值特徵")
    else:
        print(f"⚠️ {name}: 沒有數值型欄位，跳過")

if len(features_list) == 0:
    raise ValueError("❌ 錯誤：沒有找到任何特徵！")

print(f"\n準備合併 {len(features_list)} 個特徵 DataFrame...")

try:
    X = pd.concat(features_list, axis=1, join='outer', sort=False)
    print(f"✅ 合併完成")
    print(f"   合併前特徵數: {len(X.columns):,}")
    print(f"   合併前樣本數: {len(X):,}")
    
    X = X.reindex(event_indices)
    print(f"   對齊到事件索引後樣本數: {len(X):,}")
    
except Exception as e:
    raise ValueError(f"❌ 合併失敗: {str(e)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 為 X 的每個特徵分別繪製一張圖，並標示 95% winsorize 的上下限
for col in X.columns:
    fig, ax = plt.subplots(figsize=(15, 6))
    
    # 提取特徵數據
    feature_data = X[col].dropna()
    
    if len(feature_data) == 0:
        ax.text(0.5, 0.5, f'{col}\n(No data)', 
               ha='center', va='center', transform=ax.transAxes, fontsize=14)
        ax.set_title(col, fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        continue
    
    # 計算 95% winsorize 的上下限（2.5% 和 97.5% 分位數）
    lower_bound = np.percentile(feature_data, 2.5)
    upper_bound = np.percentile(feature_data, 97.5)
    
    # 繪製特徵曲線
    ax.plot(feature_data.index, feature_data.values, 
           linewidth=1.5, alpha=0.7, color='blue', label='Feature Value')
    
    # 繪製 winsorize 上下限
    ax.axhline(y=upper_bound, color='red', linestyle='--', 
              linewidth=2, label=f'Upper Bound (97.5%: {upper_bound:.4f})')
    ax.axhline(y=lower_bound, color='green', linestyle='--', 
              linewidth=2, label=f'Lower Bound (2.5%: {lower_bound:.4f})')
    
    # 填充 winsorize 範圍
    ax.fill_between(feature_data.index, lower_bound, upper_bound, 
                   alpha=0.15, color='blue', label='95% Winsorize Range')
    
    # 設置標題和標籤
    ax.set_title(f'{col}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Value', fontsize=12)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # 旋轉 x 軸標籤（如果是時間索引）
    if isinstance(feature_data.index, pd.DatetimeIndex):
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    # 打印該特徵的統計資訊
    print(f"\n{col} Statistics:")
    print(f"  Lower Bound (2.5%): {lower_bound:.6f}")
    print(f"  Upper Bound (97.5%): {upper_bound:.6f}")
    print(f"  Mean: {feature_data.mean():.6f}")
    print(f"  Std: {feature_data.std():.6f}")
    print(f"  Min: {feature_data.min():.6f}")
    print(f"  Max: {feature_data.max():.6f}")
    print("-" * 60)

In [ ]:
# =====================================================
# Winsorize 處理：對 X 的所有特徵進行 winsorize
# =====================================================

# 設定 winsorize 百分比（例如：95 表示 95%，即上下各截斷 2.5%）
winsorize_percent = 95  # 可以改為 90, 95, 99 等

# 計算對應的分位數
lower_percentile = (100 - winsorize_percent) / 2
upper_percentile = 100 - lower_percentile

print("=" * 60)
print(f"Applying {winsorize_percent}% Winsorize to All Features")
print("=" * 60)
print(f"Lower Percentile: {lower_percentile}%")
print(f"Upper Percentile: {upper_percentile}%")
print()

# 創建 X_winsorized 副本
X_winsorized = X.copy()

# 對每個特徵進行 winsorize
for col in X.columns:
    feature_data = X[col].dropna()
    
    if len(feature_data) == 0:
        print(f"⚠️ {col}: No data, skipping")
        continue
    
    # 計算上下限
    lower_bound = np.percentile(feature_data, lower_percentile)
    upper_bound = np.percentile(feature_data, upper_percentile)
    
    # 執行 winsorize：將超出範圍的值截斷到上下限
    X_winsorized[col] = X[col].clip(lower=lower_bound, upper=upper_bound)
    
    # 統計被截斷的數據點
    n_clipped_lower = (X[col] < lower_bound).sum()
    n_clipped_upper = (X[col] > upper_bound).sum()
    n_clipped_total = n_clipped_lower + n_clipped_upper
    
    # print(f"✅ {col}:")
    # print(f"   Lower Bound ({lower_percentile:.1f}%): {lower_bound:.6f}")
    # print(f"   Upper Bound ({upper_percentile:.1f}%): {upper_bound:.6f}")
    # print(f"   Clipped: {n_clipped_total} points ({n_clipped_lower} lower, {n_clipped_upper} upper)")

print("=" * 60)
print(f"✅ Winsorize completed!")
print(f"   Original X shape: {X.shape}")
print(f"   Winsorized X shape: {X_winsorized.shape}")
print("=" * 60)

# 將 winsorized 的結果賦值回 X（如果需要）
X = X_winsorized.copy()

In [ ]:
# =====================================================
# 數據清理
# =====================================================
print(f"\n" + "=" * 60)
print("🧹 數據清理...")
print("=" * 60)

X = X.fillna(method='ffill').fillna(method='bfill').fillna(0).replace([np.inf, -np.inf], 0)

# 移除完全為常數的特徵
constant_cols = []
for col in X.columns:
    try:
        unique_count = int(X[col].nunique())
        if unique_count <= 1:
            constant_cols.append(col)
    except (ValueError, TypeError):
        continue

if len(constant_cols) > 0:
    print(f"移除 {len(constant_cols)} 個常數特徵")
    X = X.drop(columns=constant_cols)

# 移除完全為 NaN 的特徵
nan_cols = [col for col in X.columns if X[col].isna().all()]
if len(nan_cols) > 0:
    print(f"移除 {len(nan_cols)} 個全 NaN 特徵")
    X = X.drop(columns=nan_cols)

print(f"\n✅ 清理完成")
print(f"   最終特徵數: {len(X.columns):,}")
print(f"   最終樣本數: {len(X):,}")

if len(X.columns) == 0:
    raise ValueError("❌ 錯誤：合併後特徵數量為 0！")

# =====================================================
# 數據對齊
# =====================================================
print(f"\n" + "=" * 60)
print("🔗 數據對齊...")
print("=" * 60)

common_idx = X.index.intersection(y.index).intersection(t1.index)
print(f"對齊前: X={len(X)}, y={len(y)}, t1={len(t1)}")
print(f"對齊後: {len(common_idx)}")

X = X.loc[common_idx]
y = y.loc[common_idx]
t1 = t1.loc[common_idx]
sample_weight = sample_weight.loc[common_idx] if sample_weight is not None else None

# 標籤映射：將 -1, 0, 1 映射為 0, 1, 2
label_mapping = {-1: 0, 0: 1, 1: 2}
reverse_mapping = {0: -1, 1: 0, 2: 1}
y_mapped = y.map(label_mapping)

print(f"\n✅ 對齊完成")
print(f"   X: {len(X):,} 個樣本, {len(X.columns):,} 個特徵")
print(f"   y: {len(y_mapped):,} 個樣本")
print(f"   標籤分布: {y_mapped.value_counts().to_dict()}")

# =====================================================
# 第2部分：特徵篩選 Pipeline（只使用 test_start 之前的數據）
# =====================================================
print(f"\n" + "=" * 60)
print("🔍 第2部分：特徵篩選 Pipeline")
print("=" * 60)
print(f"⚠️ 只使用 test_start ({test_start}) 之前的數據進行篩選")

# 切分訓練集（用於篩選）
train_mask = (X.index >= train_start) & (X.index < test_start)
X_train_filter = X.loc[train_mask].copy()
y_train_filter = y_mapped.loc[train_mask].copy()

print(f"\n篩選用訓練集:")
print(f"   樣本數: {len(X_train_filter):,}")
print(f"   特徵數: {len(X_train_filter.columns):,}")
print(f"   時間範圍: {X_train_filter.index.min()} 至 {X_train_filter.index.max()}")

# =====================================================
# 特徵篩選 Pipeline Class
# =====================================================

class FeatureFilteringPipeline:
    """
    完整的特徵篩選 Pipeline
    
    包含：
    1. Correlation Filtering（相關性過濾）
    2. Variance Threshold（低變異剔除）
    3. IC (Information Coefficient) Ranking
    4. ANOVA / t-test / χ² 統計檢定
    5. MDA (Permutation Importance)
    6. Stability Selection（可選）
    """
    
    def __init__(self, 
                 corr_threshold=0.95,
                 variance_threshold=0.0,
                 ic_threshold=None,
                 anova_p_threshold=0.05,
                 mda_top_n=None,
                 stability_n_iter=50,
                 stability_threshold=0.6):
        """
        Parameters:
        -----------
        corr_threshold : float
            相關性閾值，超過此值視為高度相關（預設 0.95）
        variance_threshold : float
            變異數閾值，低於此值視為低變異（預設 0.0）
        ic_threshold : float or None
            IC 閾值，低於此值視為無預測力（None 表示保留 top N）
        anova_p_threshold : float
            ANOVA p-value 閾值（預設 0.05）
        mda_top_n : int or None
            MDA 保留前 N 個特徵（None 表示保留所有通過檢定的）
        stability_n_iter : int
            Stability Selection 迭代次數（預設 50）
        stability_threshold : float
            Stability Selection 選擇頻率閾值（預設 0.6）
        """
        self.corr_threshold = corr_threshold
        self.variance_threshold = variance_threshold
        self.ic_threshold = ic_threshold
        self.anova_p_threshold = anova_p_threshold
        self.mda_top_n = mda_top_n
        self.stability_n_iter = stability_n_iter
        self.stability_threshold = stability_threshold
        
        self.selected_features = None
        self.filter_results = {}
        
    def correlation_filter(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        1️⃣ Correlation Filtering（相關性過濾）
        
        移除高度相關的特徵（|ρ| > threshold）
        """
        print(f"\n{'='*60}")
        print("1️⃣ Correlation Filtering")
        print(f"{'='*60}")
        
        corr_matrix = X.corr().abs()
        
        # 找出高度相關的特徵對
        upper_triangle = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )
        
        # 找出需要移除的特徵
        to_remove = set()
        for col in upper_triangle.columns:
            high_corr = upper_triangle.index[upper_triangle[col] > self.corr_threshold]
            if len(high_corr) > 0:
                # 保留與其他特徵相關性較高的（保留更重要的）
                # 簡單策略：保留第一個，移除其他的
                to_remove.update(high_corr[1:])
        
        removed_count = len(to_remove)
        remaining_features = [col for col in X.columns if col not in to_remove]
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除高度相關特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(remaining_features):,}")
        print(f"   相關性閾值: {self.corr_threshold}")
        
        self.filter_results['correlation'] = {
            'removed': list(to_remove),
            'remaining': remaining_features,
            'removed_count': removed_count
        }
        
        return X[remaining_features]
    
    def variance_filter(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        2️⃣ Variance Threshold（低變異剔除）
        """
        print(f"\n{'='*60}")
        print("2️⃣ Variance Threshold")
        print(f"{'='*60}")
        
        selector = VarianceThreshold(threshold=self.variance_threshold)
        X_selected = selector.fit_transform(X)
        selected_features = X.columns[selector.get_support()]
        removed_count = len(X.columns) - len(selected_features)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低變異特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected_features):,}")
        print(f"   變異數閾值: {self.variance_threshold}")
        
        self.filter_results['variance'] = {
            'removed': [col for col in X.columns if col not in selected_features],
            'remaining': list(selected_features),
            'removed_count': removed_count
        }
        
        return pd.DataFrame(X_selected, index=X.index, columns=selected_features)
    
    def ic_ranking(self, X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
        """
        3️⃣ IC (Information Coefficient) Ranking
        
        計算每個特徵與標籤的 Rank Correlation
        使用 |IC| 來判斷預測能力，正負 IC 都有預測價值
        """
        print(f"\n{'='*60}")
        print("3️⃣ IC (Information Coefficient) Ranking")
        print(f"{'='*60}")
        
        ic_scores = {}
        
        for col in X.columns:
            try:
                # 計算 Spearman rank correlation
                ic = stats.spearmanr(X[col], y, nan_policy='omit')[0]
                if np.isnan(ic):
                    ic = 0.0
                ic_scores[col] = ic
            except:
                ic_scores[col] = 0.0
        
        ic_series = pd.Series(ic_scores)
        ic_series = ic_series.sort_values(ascending=False)
        
        # 根據 |IC| 閾值或保留 top N（使用絕對值）
        if self.ic_threshold is not None:
            # 使用絕對值判斷預測能力
            selected = ic_series[ic_series.abs() >= self.ic_threshold].index.tolist()
        else:
            # 保留 |IC| 前 50% 的特徵（按絕對值排序）
            n_keep = max(1, int(len(ic_series) * 0.5))
            selected = ic_series.abs().nlargest(n_keep).index.tolist()
        
        removed_count = len(X.columns) - len(selected)
        
        # 統計資訊
        positive_ic = ic_series[ic_series > 0]
        negative_ic = ic_series[ic_series < 0]
        abs_ic = ic_series.abs()
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低 |IC| 特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"   IC 閾值: |IC| >= {self.ic_threshold if self.ic_threshold is not None else 'Top 50%'}")
        print(f"\n   IC 統計:")
        print(f"     平均 IC: {ic_series.mean():.4f}")
        print(f"     平均 |IC|: {abs_ic.mean():.4f}")
        print(f"     中位數 |IC|: {abs_ic.median():.4f}")
        print(f"     最大 IC: {ic_series.max():.4f}")
        print(f"     最小 IC: {ic_series.min():.4f}")
        print(f"     最大 |IC|: {abs_ic.max():.4f}")
        print(f"\n   IC 分布:")
        print(f"     正 IC 特徵數: {len(positive_ic):,} (平均: {positive_ic.mean():.4f})")
        print(f"     負 IC 特徵數: {len(negative_ic):,} (平均: {negative_ic.mean():.4f})")
        print(f"     零 IC 特徵數: {len(ic_series[ic_series == 0]):,}")
        
        # 顯示篩選後的特徵（按 |IC| 排序）
        selected_ic = ic_series[selected].abs().sort_values(ascending=False)
        print(f"\n   Top 10 篩選後特徵 (按 |IC| 排序):")
        for i, (feat, ic_val) in enumerate(selected_ic.head(10).items(), 1):
            original_ic = ic_series[feat]
            direction = "正相關" if original_ic > 0 else "負相關"
            print(f"     {i:2d}. {feat}: |IC|={ic_val:.4f} (IC={original_ic:.4f}, {direction})")
        
        self.filter_results['ic'] = {
            'scores': ic_series.to_dict(),
            'abs_scores': abs_ic.to_dict(),
            'selected': selected,
            'removed_count': removed_count,
            'positive_count': len(positive_ic),
            'negative_count': len(negative_ic)
        }
        
        return X[selected]
    
    def anova_filter(self, X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
        """
        4️⃣ ANOVA / t-test / χ² 統計檢定
        """
        print(f"\n{'='*60}")
        print("4️⃣ ANOVA / t-test / χ² Statistical Tests")
        print(f"{'='*60}")
        
        selected_features = []
        p_values = {}
        
        for col in X.columns:
            try:
                # 根據標籤分組
                groups = [X[col][y == label].dropna() for label in y.unique()]
                groups = [g for g in groups if len(g) > 0]
                
                if len(groups) < 2:
                    p_values[col] = 1.0
                    continue
                
                # 使用 ANOVA F-test
                f_stat, p_val = stats.f_oneway(*groups)
                p_values[col] = p_val
                
                if p_val < self.anova_p_threshold:
                    selected_features.append(col)
            except:
                p_values[col] = 1.0
        
        removed_count = len(X.columns) - len(selected_features)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除高 p-value 特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected_features):,}")
        print(f"   p-value 閾值: {self.anova_p_threshold}")
        
        # 顯示統計顯著的特徵
        significant = pd.Series(p_values)
        significant = significant[significant < self.anova_p_threshold].sort_values()
        print(f"\n   Top 10 最顯著特徵:")
        for i, (feat, p_val) in enumerate(significant.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: p={p_val:.6f}")
        
        self.filter_results['anova'] = {
            'p_values': p_values,
            'selected': selected_features,
            'removed_count': removed_count
        }
        
        return X[selected_features]
    
    def mda_importance(self, X: pd.DataFrame, y: pd.Series, 
                       sample_weight=None, n_estimators=100) -> pd.DataFrame:
        """
        5️⃣ MDA (Permutation Importance)
        
        使用 Random Forest 計算 Permutation Importance
        """
        print(f"\n{'='*60}")
        print("5️⃣ MDA (Permutation Importance)")
        print(f"{'='*60}")
        
        print(f"   訓練 Random Forest 模型...")
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=10,
            min_samples_split=20,
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )
        
        rf.fit(X, y, sample_weight=sample_weight)
        
        # 計算 baseline score
        baseline_score = rf.score(X, y, sample_weight=sample_weight)
        
        print(f"   Baseline Score: {baseline_score:.4f}")
        print(f"   計算 Permutation Importance...")
        
        # 計算每個特徵的 permutation importance
        importances = {}
        for col in X.columns:
            X_permuted = X.copy()
            X_permuted[col] = np.random.permutation(X_permuted[col].values)
            permuted_score = rf.score(X_permuted, y, sample_weight=sample_weight)
            importance = baseline_score - permuted_score
            importances[col] = importance
        
        importance_series = pd.Series(importances).sort_values(ascending=False)
        
        # 選擇重要特徵
        if self.mda_top_n is not None:
            selected = importance_series.head(self.mda_top_n).index.tolist()
        else:
            # 保留重要性 > 0 的特徵
            selected = importance_series[importance_series > 0].index.tolist()
        
        removed_count = len(X.columns) - len(selected)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低重要性特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"\n   Top 10 最重要特徵:")
        for i, (feat, imp) in enumerate(importance_series.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: {imp:.6f}")
        
        self.filter_results['mda'] = {
            'importances': importance_series.to_dict(),
            'selected': selected,
            'removed_count': removed_count
        }
        
        return X[selected]
    
    def stability_selection(self, X: pd.DataFrame, y: pd.Series,
                           sample_weight=None, n_estimators=50):
        """
        6️⃣ Stability Selection
        
        通過多次子抽樣和模型訓練，統計特徵被選中的頻率
        """
        print(f"\n{'='*60}")
        print("6️⃣ Stability Selection")
        print(f"{'='*60}")
        print(f"   迭代次數: {self.stability_n_iter}")
        print(f"   選擇頻率閾值: {self.stability_threshold}")
        
        feature_selection_counts = {col: 0 for col in X.columns}
        
        for i in range(self.stability_n_iter):
            if (i + 1) % 10 == 0:
                print(f"   進度: {i+1}/{self.stability_n_iter}")
            
            # 子抽樣（80% 數據）
            n_samples = int(len(X) * 0.8)
            sample_idx = np.random.choice(X.index, size=n_samples, replace=False)
            X_sub = X.loc[sample_idx]
            y_sub = y.loc[sample_idx]
            w_sub = sample_weight.loc[sample_idx] if sample_weight is not None else None
            
            # 訓練模型並獲取特徵重要性
            rf = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=10,
                min_samples_split=20,
                random_state=i,
                n_jobs=-1,
                class_weight='balanced'
            )
            
            rf.fit(X_sub, y_sub, sample_weight=w_sub)
            
            # 獲取特徵重要性（使用 MDI）
            importances = pd.Series(rf.feature_importances_, index=X.columns)
            top_features = importances.nlargest(int(len(X.columns) * 0.3)).index  # 保留前30%
            
            for feat in top_features:
                feature_selection_counts[feat] += 1
        
        # 計算選擇頻率
        selection_freq = pd.Series(feature_selection_counts) / self.stability_n_iter
        selection_freq = selection_freq.sort_values(ascending=False)
        
        # 根據閾值選擇特徵
        selected = selection_freq[selection_freq >= self.stability_threshold].index.tolist()
        removed_count = len(X.columns) - len(selected)
        
        print(f"\n   原始特徵數: {len(X.columns):,}")
        print(f"   移除不穩定特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"\n   Top 10 最穩定特徵:")
        for i, (feat, freq) in enumerate(selection_freq.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: {freq:.4f}")
        
        self.filter_results['stability'] = {
            'frequencies': selection_freq.to_dict(),
            'selected': selected,
            'removed_count': removed_count
        }
        
        return X[selected]
    
    def fit_transform(self, X: pd.DataFrame, y: pd.Series, 
                     sample_weight=None, use_stability=False):
        """
        執行完整的篩選流程
        
        Parameters:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            標籤
        sample_weight : pd.Series, optional
            樣本權重
        use_stability : bool
            是否使用 Stability Selection（較慢）
        
        Returns:
        --------
        X_selected : pd.DataFrame
            篩選後的特徵矩陣
        """
        print(f"\n{'='*80}")
        print("🚀 開始執行特徵篩選 Pipeline")
        print(f"{'='*80}")
        print(f"原始特徵數: {len(X.columns):,}")
        print(f"原始樣本數: {len(X):,}")
        
        X_current = X.copy()
        
        # 1. Correlation Filter
        X_current = self.correlation_filter(X_current)
        
        # 2. Variance Filter
        X_current = self.variance_filter(X_current)
        
        # 3. IC Ranking
        X_current = self.ic_ranking(X_current, y)
        
        # 4. ANOVA Filter
        X_current = self.anova_filter(X_current, y)
        
        # 5. MDA Importance
        X_current = self.mda_importance(X_current, y, sample_weight)
        
        # 6. Stability Selection (可選，較慢)
        if use_stability:
            X_current = self.stability_selection(X_current, y, sample_weight)
        
        self.selected_features = list(X_current.columns)
        
        print(f"\n{'='*80}")
        print("✅ 特徵篩選完成")
        print(f"{'='*80}")
        print(f"最終特徵數: {len(self.selected_features):,}")
        print(f"特徵減少比例: {(1 - len(self.selected_features) / len(X.columns)) * 100:.2f}%")
        
        return X_current

# =====================================================
# 執行特徵篩選
# =====================================================

# 初始化 Pipeline
filter_pipeline = FeatureFilteringPipeline(
    corr_threshold=0.95,          # 相關性閾值
    variance_threshold=0.005,      # 變異數閾值
    ic_threshold=None,            # IC 閾值（None 表示保留 top 50%）
    anova_p_threshold=0.2,       # ANOVA p-value 閾值
    mda_top_n=None,               # MDA 保留所有重要性 > 0 的特徵
    stability_n_iter=5,          # Stability Selection 迭代次數
    stability_threshold=0.1      # Stability Selection 頻率閾值
)

# 執行篩選（不使用 Stability Selection，因為較慢）
# 如果需要最穩定的特徵，可以設置 use_stability=True
X_train_filtered = filter_pipeline.fit_transform(
    X_train_filter, 
    y_train_filter,
    sample_weight=sample_weight.loc[train_mask] if sample_weight is not None else None,
    use_stability=False  # 設為 True 可啟用 Stability Selection（較慢）
)



## 步驟7：ML Model構建

In [ ]:
from scipy.stats import rv_continuous
import numpy as np

class logUniform_gen(rv_continuous):
    """
    Log-uniform distribution random variable generator
    
    說明:
    -----
    隨機變數 x 在 [a, b] 區間內遵循 log-uniform 分布，
    當且僅當 log[x] ~ U[log[a], log[b]]
    
    這個分布對於探索非線性響應的參數空間特別有效，
    例如 SVC 的 C 參數和 RBF kernel 的 gamma 參數。
    """
    
    def _cdf(self, x):
        """累積分布函數 (CDF)"""
        return np.log(x / self.a) / np.log(self.b / self.a)


def logUniform(a=1, b=np.exp(1)):
    """
    創建 log-uniform 分布
    
    參數:
    -----------
    a : float
        下界（必須 > 0）
    b : float
        上界（必須 > a）
        
    Returns:
    --------
    logUniform_gen : logUniform_gen
        Log-uniform 分布生成器
    """
    return logUniform_gen(a=a, b=b, name='logUniform')

In [ ]:
# =====================================================
# 步驟7：ML Model 構建與訓練（擴展版 - 支援多種模型）
# =====================================================

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              ExtraTreesClassifier, AdaBoostClassifier,
                              HistGradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import (classification_report, confusion_matrix, 
                            roc_auc_score, precision_recall_curve, 
                            roc_curve, accuracy_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Dict, Any, Tuple, List
import warnings
warnings.filterwarnings('ignore')

class ModelTrainer:
    """
    擴展版模型訓練器：支援多種模型
    
    支援的模型類型:
    -----
    - 'xgboost': XGBoost
    - 'lightgbm': LightGBM
    - 'catboost': CatBoost
    - 'rf': Random Forest
    - 'et': Extra Trees
    - 'gb': Gradient Boosting
    - 'histgbm': HistGradientBoosting
    - 'dt': Decision Tree
    - 'lr': Logistic Regression
    - 'svm': Support Vector Machine
    - 'mlp': Multi-Layer Perceptron
    - 'lstm': LSTM (需要額外處理)
    - 'ada': AdaBoost
    - 'knn': K-Nearest Neighbors
    - 'nb': Naive Bayes
    - 'qda': Quadratic Discriminant Analysis
    """
    
    def __init__(self, model_type: str = 'xgboost'):
        """
        初始化模型訓練器
        
        參數:
        -----------
        model_type : str
            模型類型
        """
        self.model_type = model_type.lower()
        self.model = None
        self.best_params = None
        self.cv_scores = None
        
    def _create_model(self, **kwargs):
        """
        創建模型實例
        """
        if self.model_type == 'xgboost':
            default_params = {
                'random_state': 42,
                'eval_metric': 'logloss',
                'use_label_encoder': False,
                'verbosity': 0
            }
            default_params.update(kwargs)
            return XGBClassifier(**default_params)
        
        elif self.model_type == 'lightgbm':
            default_params = {
                'random_state': 42,
                'verbosity': -1,
                'force_col_wise': True
            }
            default_params.update(kwargs)
            return LGBMClassifier(**default_params)
        
        elif self.model_type == 'catboost':
            default_params = {
                'random_state': 42,
                'verbose': False,
                'thread_count': -1
            }
            default_params.update(kwargs)
            return CatBoostClassifier(**default_params)
        
        elif self.model_type == 'rf':
            default_params = {
                'random_state': 42,
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return RandomForestClassifier(**default_params)
        
        elif self.model_type == 'et':
            default_params = {
                'random_state': 42,
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return ExtraTreesClassifier(**default_params)
        
        elif self.model_type == 'gb':
            default_params = {
                'random_state': 42
            }
            default_params.update(kwargs)
            return GradientBoostingClassifier(**default_params)
        
        elif self.model_type == 'histgbm':
            default_params = {
                'random_state': 42,
                'max_iter': 100,
                'early_stopping': True
            }
            default_params.update(kwargs)
            return HistGradientBoostingClassifier(**default_params)
        
        elif self.model_type == 'dt':
            default_params = {
                'random_state': 42,
                'max_depth': 10
            }
            default_params.update(kwargs)
            return DecisionTreeClassifier(**default_params)
        
        elif self.model_type == 'lr':
            default_params = {
                'random_state': 42,
                'max_iter': 1000,
                'solver': 'lbfgs'
            }
            default_params.update(kwargs)
            return LogisticRegression(**default_params)
        
        elif self.model_type == 'svm':
            default_params = {
                'random_state': 42,
                'probability': True,
                'kernel': 'rbf'
            }
            default_params.update(kwargs)
            return SVC(**default_params)
        
        elif self.model_type == 'mlp':
            default_params = {
                'random_state': 42,
                'max_iter': 500,
                'early_stopping': True,
                'validation_fraction': 0.1
            }
            default_params.update(kwargs)
            return MLPClassifier(**default_params)
        
        elif self.model_type == 'ada':
            default_params = {
                'random_state': 42
            }
            default_params.update(kwargs)
            return AdaBoostClassifier(**default_params)
        
        elif self.model_type == 'knn':
            default_params = {
                'n_neighbors': 5,
                'weights': 'distance',
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return KNeighborsClassifier(**default_params)
        
        elif self.model_type == 'nb':
            default_params = {}
            default_params.update(kwargs)
            return GaussianNB(**default_params)
        
        elif self.model_type == 'qda':
            default_params = {}
            default_params.update(kwargs)
            return QuadraticDiscriminantAnalysis(**default_params)
        
        elif self.model_type == 'lstm':
            # LSTM 需要特殊處理，這裡先拋出提示
            raise NotImplementedError(
                "LSTM 需要序列數據和特殊處理，請使用專門的深度學習框架（如 TensorFlow/Keras）"
            )
        
        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        """
        print(f"🚀 取得 {self.model_type} 模型參數網格")
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
        
        elif self.model_type == 'lightgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10, -1],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__num_leaves': [15, 31, 50, 100],
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_samples': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__num_leaves': [31, 50],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
        
        elif self.model_type == 'catboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300, 500],
                    f'{self.model_type}__depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__l2_leaf_reg': [1, 3, 5, 7],
                    f'{self.model_type}__border_count': [32, 64, 128],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300],
                    f'{self.model_type}__depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__l2_leaf_reg': [3, 5],
                }
                return param_grid
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500, 1000],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': logUniform(1e-3, 1e2),
                    f'{self.model_type}__min_samples_leaf': logUniform(1e-3, 1e1),
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                    f'{self.model_type}__bootstrap': [True, False]
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None]
                }
                return param_grid
        
        elif self.model_type == 'et':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__max_features': ['sqrt', 'log2'],
                }
                return param_grid
        
        elif self.model_type == 'gb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200],
                    f'{self.model_type}__max_depth': [3, 5],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                }
                return param_grid
        
        elif self.model_type == 'histgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200, 300],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__min_samples_leaf': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                }
                return param_grid
        
        elif self.model_type == 'dt':
            param_grid = {
                f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                f'{self.model_type}__min_samples_split': [2, 5, 10],
                f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                f'{self.model_type}__max_features': ['sqrt', 'log2', None],
            }
            return param_grid
        
        elif self.model_type == 'lr':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-3, 1e3),
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.01, 0.1, 1.0, 10.0],
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
        
        elif self.model_type == 'svm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-2, 1e2),
                    f'{self.model_type}__gamma': logUniform(1e-4, 1e-1),
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.1, 1.0, 10.0],
                    f'{self.model_type}__gamma': ['scale', 'auto', 0.001, 0.01],
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
        
        elif self.model_type == 'mlp':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                    f'{self.model_type}__alpha': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50)],
                    f'{self.model_type}__alpha': [0.0001, 0.001, 0.01],
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
        
        elif self.model_type == 'ada':
            param_grid = {
                f'{self.model_type}__n_estimators': [50, 100, 200],
                f'{self.model_type}__learning_rate': [0.01, 0.1, 1.0],
            }
            return param_grid   
        
        elif self.model_type == 'knn':
            param_grid = {
                f'{self.model_type}__n_neighbors': [3, 5, 7, 10, 15],
                f'{self.model_type}__weights': ['uniform', 'distance'],
                f'{self.model_type}__metric': ['euclidean', 'manhattan', 'minkowski'],
            }
            return param_grid
        elif self.model_type == 'nb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__var_smoothing': logUniform(1e-9, 1e-3),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
                }
                return param_grid
        elif self.model_type == 'qda':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__reg_param': logUniform(1e-3, 1.0),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__reg_param': [0.0, 0.1, 0.5, 1.0],
                }   
                return param_grid
        else:
            # 如果模型類型沒有對應的參數網格，返回空字典
            param_grid = {}
            print(f"⚠️ 警告: 模型類型 '{self.model_type}' 沒有定義參數網格，使用空字典")
        
        return param_grid
    
    def train(self, 
              X: pd.DataFrame,
              y: pd.Series,
              t1: pd.Series,
              sample_weight: Optional[pd.Series] = None,
              cv: int = 5,
              pctEmbargo: float = 0.01,
              use_hyperopt: bool = True,
              use_randomized: bool = False,
              rndSearchIter: int = 50,
              n_jobs: int = -1,
              **model_kwargs) -> Any:
        """
        訓練模型
        """
        print("=" * 60)
        print(f"🚀 開始訓練 {self.model_type.upper()} 模型")
        print("=" * 60)
        
        if sample_weight is None:
            sample_weight = pd.Series(1.0, index=X.index)
        
        # 判斷是否需要標準化（線性模型和 SVM、MLP 需要）
        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb', 'qda']
        
        if use_hyperopt:
            print(f"\n📊 使用 {'RandomizedSearchCV' if use_randomized else 'GridSearchCV'} 進行超參數調優...")
            
            # 創建 pipeline
            steps = []
            if needs_scaling:
                steps.append(('scaler', StandardScaler()))
            steps.append((self.model_type, self._create_model(**model_kwargs)))
            
            pipe_clf = MyPipeline(steps)
            
            param_grid = self._get_param_grid(use_randomized=use_randomized)
            
            # 準備 fit_params（某些模型不支援 sample_weight）
            fit_params = {}
            if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                fit_params[f'{self.model_type}__sample_weight'] = sample_weight
            
            best_pipeline = clfHyperFit(
                feat=X,
                lbl=y,
                t1=t1,
                pipe_clf=pipe_clf,
                param_grid=param_grid,
                cv=cv,
                bagging=[0, None, 1.],
                rndSearchIter=rndSearchIter if use_randomized else 0,
                n_jobs=n_jobs,
                pctEmbargo=pctEmbargo,
                **fit_params
            )
            
            self.model = best_pipeline.named_steps[self.model_type]
            
            if hasattr(best_pipeline, 'best_params_'):
                self.best_params = best_pipeline.best_params_
            
            print(f"\n✅ 超參數調優完成")
            if self.best_params:
                print(f"最佳參數:")
                for key, value in list(self.best_params.items())[:5]:
                    print(f"  {key}: {value}")
                if len(self.best_params) > 5:
                    print(f"  ... (共 {len(self.best_params)} 個參數)")
        else:
            print(f"\n📊 直接訓練模型（不使用超參數調優）...")
            
            inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
            
            # 對於需要標準化的模型，使用 Pipeline
            if needs_scaling:
                from sklearn.pipeline import Pipeline
                clf = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', self._create_model(**model_kwargs))
                ])
                fit_params = {}
                if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                    fit_params['clf__sample_weight'] = sample_weight
            else:
                clf = self._create_model(**model_kwargs)
                fit_params = {}
                if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                    fit_params['sample_weight'] = sample_weight
            
            self.cv_scores = cvScore(
                clf=clf,
                X=X,
                y=y,
                sample_weight=sample_weight if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda'] else None,
                scoring='f1',
                cvGen=inner_cv
            )
            
            print(f"\n交叉驗證分數: {self.cv_scores}")
            print(f"平均分數: {self.cv_scores.mean():.4f} ± {self.cv_scores.std():.4f}")
            
            if needs_scaling:
                from sklearn.pipeline import Pipeline
                self.model = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', self._create_model(**model_kwargs))
                ])
                self.model.fit(X, y, **fit_params)
            else:
                self.model = clf.fit(X, y, **fit_params)
        
        print("=" * 60)
        print("✅ 模型訓練完成")
        print("=" * 60)
        
        return self.model
    
    def predict(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """進行預測"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        predictions = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        return predictions, probabilities
    
    def evaluate(self, 
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """評估模型性能"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        
        metrics = {
            'accuracy': accuracy_score(y, y_pred, sample_weight=sample_weight),
            'f1': f1_score(y, y_pred, sample_weight=sample_weight, average='weighted'),
        }
        
        if len(np.unique(y)) == 2:
            metrics['roc_auc'] = roc_auc_score(y, y_proba[:, 1], sample_weight=sample_weight)
            metrics['f1_binary'] = f1_score(y, y_pred, sample_weight=sample_weight)
        
        return metrics
    
    def plot_results(self, 
                    X: pd.DataFrame,
                    y: pd.Series,
                    sample_weight: Optional[pd.Series] = None):
        """繪製模型評估結果（與原版相同）"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        metrics = self.evaluate(X, y, sample_weight)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 混淆矩陣
        cm = confusion_matrix(y, y_pred, sample_weight=sample_weight)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
        axes[0, 0].set_title('Confusion Matrix', fontsize=14)
        axes[0, 0].set_ylabel('True Label', fontsize=12)
        axes[0, 0].set_xlabel('Predicted Label', fontsize=12)
        
        # 2. ROC 曲線
        if len(np.unique(y)) == 2:
            fpr, tpr, _ = roc_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[0, 1].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {metrics["roc_auc"]:.4f})')
            axes[0, 1].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[0, 1].set_xlabel('False Positive Rate', fontsize=12)
            axes[0, 1].set_ylabel('True Positive Rate', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'ROC Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
        
        # 3. Precision-Recall 曲線
        if len(np.unique(y)) == 2:
            precision, recall, _ = precision_recall_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[1, 0].plot(recall, precision, linewidth=2)
            axes[1, 0].set_xlabel('Recall', fontsize=12)
            axes[1, 0].set_ylabel('Precision', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
            axes[1, 0].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'Precision-Recall Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
        
        # 4. 評估指標摘要
        axes[1, 1].axis('off')
        metrics_text = f"""
        Model: {self.model_type.upper()}
        
        Evaluation Metrics:
        --------------------
        Accuracy: {metrics['accuracy']:.4f}
        F1 Score: {metrics['f1']:.4f}
        """
        if 'roc_auc' in metrics:
            metrics_text += f"ROC AUC: {metrics['roc_auc']:.4f}\n"
        if 'f1_binary' in metrics:
            metrics_text += f"F1 (Binary): {metrics['f1_binary']:.4f}\n"
        
        axes[1, 1].text(0.1, 0.5, metrics_text, fontsize=12, 
                        verticalalignment='center', family='monospace')
        
        plt.tight_layout()
        plt.show()
        
        print("\n" + "=" * 60)
        print("📊 詳細分類報告")
        print("=" * 60)
        print(classification_report(y, y_pred, sample_weight=sample_weight))
        print("=" * 60)

## 步驟8：Embargo Purged K-Fold Class交叉驗證

In [ ]:
# =====================================================
# Purged K-Fold Cross-Validation (Snippet 7.3 & 7.4)
# =====================================================

from sklearn.model_selection._split import _BaseKFold
import numpy as np
import pandas as pd
from typing import Optional, Generator, Tuple
from sklearn.metrics import log_loss, accuracy_score

class PurgedKFold(_BaseKFold):
    """
    Extend KFold class to work with labels that span intervals
    
    The train is purged of observations overlapping test-label intervals
    Test set is assumed contiguous (shuffle=False), w/o training samples in between
    
    說明:
    -----
    這個類別擴展了 scikit-learn 的 KFold，用於處理標籤重疊的情況。
    當標籤跨越時間區間時（例如 Triple Barrier Method），測試集的標籤
    可能會與訓練集的標籤重疊，導致數據洩漏。
    
    解決方法:
    ---------
    1. Purging（清除）: 從訓練集中移除與測試集標籤區間重疊的觀測值
    2. Embargo（禁制）: 在測試集結束後，額外移除一段時間的訓練樣本
    
    參數:
    -----------
    n_splits : int
        K 折交叉驗證的折數
    t1 : pd.Series
        標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
    pctEmbargo : float
        禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
    """
    
    def __init__(self, n_splits: int = 3, t1: Optional[pd.Series] = None, pctEmbargo: float = 0.0):
        """
        初始化 PurgedKFold
        
        參數:
        -----------
        n_splits : int
            K 折交叉驗證的折數
        t1 : pd.Series, optional
            標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
        pctEmbargo : float
            禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
        """
        if t1 is not None and not isinstance(t1, pd.Series):
            raise ValueError('Label Through Dates must be a pd.Series')
        
        super(PurgedKFold, self).__init__(n_splits, shuffle=False, random_state=None)
        self.t1 = t1
        self.pctEmbargo = pctEmbargo
    
    def split(self, X: pd.DataFrame, y: Optional[pd.Series] = None, 
              groups: Optional[np.ndarray] = None) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
        """
        生成訓練/測試集的索引
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據（index 必須與 t1 的 index 一致）
        y : pd.Series, optional
            標籤（未使用，但保留以符合 scikit-learn 接口）
        groups : np.ndarray, optional
            分組（未使用，但保留以符合 scikit-learn 接口）
            
        Yields:
        -------
        train_indices : np.ndarray
            訓練集索引
        test_indices : np.ndarray
            測試集索引
        """
        if self.t1 is None:
            raise ValueError('t1 (ThruDate) must be provided')
        
        # 驗證 X 和 t1 的 index 一致
        if (X.index == self.t1.index).sum() != len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')
        
        indices = np.arange(X.shape[0])
        mbrg = int(X.shape[0] * self.pctEmbargo)  # embargo 的樣本數
        
        # 將數據分成 n_splits 個連續的測試集
        test_starts = [(i[0], i[-1] + 1) for i in 
                      np.array_split(np.arange(X.shape[0]), self.n_splits)]
        
        for i, j in test_starts:
            # 測試集的起始時間
            t0 = self.t1.index[i]  # start of test set
            test_indices = indices[i:j]
            
            # 找到測試集中最晚的結束時間
            maxT1Idx = self.t1.index.searchsorted(self.t1[test_indices].max())
            
            # 左側訓練集：結束時間 <= t0 的所有樣本
            train_indices = self.t1.index.searchsorted(
                self.t1[self.t1 <= t0].index
            )
            
            # 右側訓練集：在測試集結束後，加上 embargo 期間
            if maxT1Idx < X.shape[0]:
                # right train (with embargo)
                train_indices = np.concatenate((
                    train_indices, 
                    indices[maxT1Idx + mbrg:]
                ))
            
            yield train_indices, test_indices


def macro_accuracy_score(y_true, y_pred, sample_weight=None, zero_division=0):
    """
    計算 Macro Accuracy（每個類別的準確率直接平均）
    
    參數:
    -----------
    y_true : array-like
        真實標籤
    y_pred : array-like
        預測標籤
    sample_weight : array-like, optional
        樣本權重
    zero_division : float
        當分母為 0 時返回的值
        
    Returns:
    --------
    macro_accuracy : float
        Macro Accuracy（每個類別的準確率直接平均）
    """
    from sklearn.metrics import accuracy_score
    from sklearn.utils.multiclass import unique_labels
    import numpy as np
    
    # 獲取所有類別
    labels = unique_labels(y_true, y_pred)
    
    # 計算每個類別的準確率
    per_class_accuracies = []
    
    for label in labels:
        # 找出該類別的所有樣本
        mask = (y_true == label)
        
        if mask.sum() == 0:
            # 如果該類別在真實標籤中不存在，跳過
            continue
            
        # 該類別的樣本
        y_true_class = y_true[mask]
        y_pred_class = y_pred[mask]
        
        # 如果有樣本權重
        if sample_weight is not None:
            sample_weight_class = sample_weight[mask]
        else:
            sample_weight_class = None
        
        # 計算該類別的準確率
        class_accuracy = accuracy_score(
            y_true_class, 
            y_pred_class, 
            sample_weight=sample_weight_class
        )
        per_class_accuracies.append(class_accuracy)
    
    # 如果沒有任何類別，返回 zero_division
    if len(per_class_accuracies) == 0:
        return zero_division
    
    # 返回平均準確率（Macro Accuracy）
    return np.mean(per_class_accuracies)

def cvScore(
    clf,
    X: pd.DataFrame,
    y: pd.Series,
    sample_weight: Optional[pd.Series] = None,
    scoring: str = 'f1',
    t1: Optional[pd.Series] = None,
    cv: Optional[int] = None,
    cvGen: Optional[PurgedKFold] = None,
    pctEmbargo: Optional[float] = None,
) -> np.ndarray:
    """
    使用 PurgedKFold 進行交叉驗證評分，確保每折標籤重新編碼為連續整數。
    """
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import (
        f1_score,
        precision_score,
        recall_score,
        accuracy_score,
        log_loss,
    )

    if cvGen is None:
        if cv is None:
            raise ValueError("請提供 cv 或 cvGen 其中之一")
        cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)

    if sample_weight is None:
        sample_weight = pd.Series(1.0, index=X.index)

    scores = []

    for fold_idx, (train_idx, test_idx) in enumerate(cvGen.split(X=X)):
        X_train_fold = X.iloc[train_idx, :]
        y_train_fold = y.iloc[train_idx]
        X_test_fold = X.iloc[test_idx, :]
        y_test_fold = y.iloc[test_idx]

        fold_label_encoder = LabelEncoder()
        y_train_fold_encoded = pd.Series(
            fold_label_encoder.fit_transform(y_train_fold),
            index=y_train_fold.index,
        )
        y_test_fold_encoded = pd.Series(
            fold_label_encoder.transform(y_test_fold),
            index=y_test_fold.index,
        )

        sw_train = sample_weight.iloc[train_idx].values
        sw_test = sample_weight.iloc[test_idx].values

        fit = clf.fit(
            X=X_train_fold,
            y=y_train_fold_encoded,
            sample_weight=sw_train,
        )

        if scoring == 'neg_log_loss':
            prob = fit.predict_proba(X_test_fold)
            score_ = -log_loss(
                y_test_fold_encoded,
                prob,
                sample_weight=sw_test,
            )
        elif scoring in ('f1', 'f1_macro'):
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'f1_weighted':
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='weighted',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'precision_macro':
            pred = fit.predict(X_test_fold)
            score_ = precision_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'recall_macro':
            pred = fit.predict(X_test_fold)
            score_ = recall_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'accuracy':
            pred = fit.predict(X_test_fold)
            score_ = accuracy_score(
                y_test_fold_encoded,
                pred,
                sample_weight=sw_test,
            )
        else:
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )

        scores.append(score_)

    return np.array(scores)


## 步驟9 ： 超參數調整

In [ ]:
# =====================================================
# Hyperparameter Tuning with Purged K-Fold CV
# Snippet 9.1, 9.2, 9.3, 9.4
# =====================================================

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import rv_continuous, kstest
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# =====================================================
# Snippet 9.2: Enhanced Pipeline Class
# =====================================================

class MyPipeline(Pipeline):
    """
    Enhanced Pipeline class that handles sample_weight argument
    
    說明:
    -----
    sklearn 的 Pipeline 的 fit 方法不接受 sample_weight 參數，
    而是期望 fit_params 關鍵字參數。這是一個已知的 bug。
    這個類別擴展了 Pipeline，重寫 fit 方法以處理 sample_weight。
    
    參考:
    -----
    - GitHub issue: sklearn Pipeline sample_weight bug
    - Stackoverflow: Understanding Python super with __init__ methods
    """
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """
        重寫 fit 方法以處理 sample_weight
        
        參數:
        -----------
        X : array-like
            特徵數據
        y : array-like
            標籤
        sample_weight : array-like, optional
            樣本權重
        **fit_params : dict
            其他 fit 參數
            
        Returns:
        --------
        self : MyPipeline
            擬合後的 pipeline
        """
        if sample_weight is not None:
            # 將 sample_weight 傳遞給最後一個步驟（通常是分類器）
            last_step_name = self.steps[-1][0]
            fit_params[f'{last_step_name}__sample_weight'] = sample_weight
        
        return super(MyPipeline, self).fit(X, y, **fit_params)


# =====================================================
# Snippet 9.4: Log-Uniform Distribution
# =====================================================

class logUniform_gen(rv_continuous):
    """
    Log-uniform distribution random variable generator
    
    說明:
    -----
    隨機變數 x 在 [a, b] 區間內遵循 log-uniform 分布，
    當且僅當 log[x] ~ U[log[a], log[b]]
    
    這個分布對於探索非線性響應的參數空間特別有效，
    例如 SVC 的 C 參數和 RBF kernel 的 gamma 參數。
    
    數學定義:
    ---------
    CDF: F[x] = log[x] - log[a] / log[b] - log[a]  for a ≤ x ≤ b
    PDF: f[x] = 1 / (x * log[b/a])                 for a ≤ x ≤ b
    """
    
    def _cdf(self, x):
        """累積分布函數 (CDF)"""
        return np.log(x / self.a) / np.log(self.b / self.a)


def logUniform(a=1, b=np.exp(1)):
    """
    創建 log-uniform 分布
    
    參數:
    -----------
    a : float
        下界（必須 > 0）
    b : float
        上界（必須 > a）
        
    Returns:
    --------
    logUniform_gen : logUniform_gen
        Log-uniform 分布生成器
    """
    return logUniform_gen(a=a, b=b, name='logUniform')


# =====================================================
# Snippet 9.1 & 9.3: Hyperparameter Fitting Function
# =====================================================

def clfHyperFit(feat: pd.DataFrame, 
                lbl: pd.Series,
                t1: pd.Series,
                pipe_clf: Pipeline,
                param_grid: Dict[str, List[Any]],
                cv: int = 3,
                bagging: List[Any] = [0, None, 1.],
                rndSearchIter: int = 0,
                n_jobs: int = -1,
                pctEmbargo: float = 0.0,
                scoring: str = 'f1_macro',  # ⭐ 新增參數
                **fit_params) -> Pipeline:
    """
    使用 Purged K-Fold 交叉驗證進行超參數調優
    
    參數:
    -----------
    scoring : str
        評分方法（用於 GridSearchCV/RandomizedSearchCV）
        - 'f1_macro': Macro-F1
        - 'f1_weighted': Weighted-F1
        - 'accuracy_macro': Macro Accuracy
        - 'precision_macro': Macro Precision
        - 'recall_macro': Macro Recall
        - 'accuracy': 準確率
        - 'neg_log_loss': 負對數損失
    **fit_params : dict
        其他 fit 參數（例如 sample_weight），但不包括 scoring
    """
    
    # ⭐ 重要：從 fit_params 中移除 scoring（如果存在的話）
    # 因為 scoring 是 GridSearchCV 的參數，不是 Pipeline.fit() 的參數
    fit_params_clean = {k: v for k, v in fit_params.items() if k != 'scoring'}
    
    # 1) 超參數搜尋，在訓練數據上
    inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    
    # ⚠️ 注意：sklearn 的 GridSearchCV/RandomizedSearchCV 不直接支援自定義的 scoring 字符串
    # 需要創建一個自定義的 scorer 或使用 make_scorer
    # 但由於我們使用的是 PurgedKFold，可能需要自定義評分邏輯
    
    # 暫時使用 scoring 參數（如果 sklearn 支援的話）
    # 如果不支援，需要創建自定義 scorer
    from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score
    
    # 創建自定義 scorer（如果需要）
    if scoring == 'f1_macro':
        scorer = make_scorer(f1_score, average='macro', zero_division=0)
    elif scoring == 'f1_weighted':
        scorer = make_scorer(f1_score, average='weighted', zero_division=0)
    elif scoring == 'precision_macro':
        scorer = make_scorer(precision_score, average='macro', zero_division=0)
    elif scoring == 'recall_macro':
        scorer = make_scorer(recall_score, average='macro', zero_division=0)
    elif scoring == 'accuracy_macro':
        # 使用自定義的 macro_accuracy_score
        scorer = make_scorer(macro_accuracy_score, zero_division=0)
    elif scoring == 'accuracy':
        scorer = make_scorer(accuracy_score)
    elif scoring == 'neg_log_loss':
        from sklearn.metrics import log_loss
        scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
    else:
        # 預設使用 f1_macro
        scorer = make_scorer(f1_score, average='macro', zero_division=0)
    
    if rndSearchIter == 0:
        gs = GridSearchCV(
            estimator=pipe_clf,
            param_grid=param_grid,
            scoring=scorer,  # ⭐ 使用自定義 scorer
            cv=inner_cv,
            n_jobs=n_jobs
        )
    else:
        gs = RandomizedSearchCV(
            estimator=pipe_clf,
            param_distributions=param_grid,
            scoring=scorer,  # ⭐ 使用自定義 scorer
            cv=inner_cv,
            n_jobs=n_jobs,
            n_iter=rndSearchIter
        )
    
    # ⭐ 使用清理後的 fit_params（不包含 scoring）
    gs = gs.fit(feat, lbl, **fit_params_clean).best_estimator_
    
    # 2) 在全部數據上擬合驗證後的模型
    if bagging[1] is not None and bagging[1] > 0:
        gs = BaggingClassifier(
            base_estimator=MyPipeline(gs.steps),
            n_estimators=int(bagging[0]),
            max_samples=float(bagging[1]),
            max_features=float(bagging[2]),
            n_jobs=n_jobs
        )
        gs = gs.fit(feat, lbl, **fit_params_clean)
    
    return gs


## 步驟10 ： 執行預測

In [ ]:
# =====================================================
# 步驟10：PredictionPipeline - 整合步驟7、8、9
# =====================================================

import pandas as pd
import numpy as np
from typing import Optional, Dict, Any, Tuple, List
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class PredictionPipeline:
    """
    預測 Pipeline：整合模型訓練、交叉驗證、超參數調整
    
    功能:
    -----
    1. 模型訓練（步驟7）
    2. PurgedKFold 交叉驗證（步驟8）
    3. 超參數調整（步驟9）
    4. 模型預測
    5. 結果評估和可視化
    
    使用方式:
    --------
    pipeline = PredictionPipeline(
        model_type='xgboost',  # 或 'rf'
        use_hyperopt=True,     # 是否使用超參數調整
        use_cv=True            # 是否使用交叉驗證
    )
    
    # 傳入準備好的數據
    pipeline.fit(
        X=X,                    # 特徵矩陣
        y=y,                    # 標籤
        t1=t1,                  # 標籤結束時間
        sample_weight=weights   # 樣本權重（可選）
    )
    
    # 預測
    predictions, probabilities = pipeline.predict(X_test)
    """
    
    def __init__(self,
                 model_type: str = 'xgboost',
                 cv_scoring: str = 'f1_macro',  # ⭐ 新增這個參數
                 use_hyperopt: bool = True,
                 use_cv: bool = True,
                 cv_splits: int = 5,
                 pct_embargo: float = 0.01,
                 hyperopt_method: str = 'randomized',  # 'randomized' 或 'grid'
                 n_iter: int = 30,
                 bagging: Optional[List[Any]] = None,
                 n_jobs: int = -1):
        """
        初始化預測 Pipeline
        
        參數:
        -----------
        model_type : str
            模型類型：'xgboost' 或 'rf'
        use_hyperopt : bool
            是否使用超參數調整（預設 True）
        use_cv : bool
            是否使用交叉驗證（預設 True）
        cv_splits : int
            交叉驗證折數（預設 5）
        pct_embargo : float
            禁制比例（預設 0.01）
        hyperopt_method : str
            超參數調整方法：'randomized' 或 'grid'（預設 'randomized'）
        n_iter : int
            隨機搜尋迭代次數（預設 30）
        bagging : list, optional
            Bagging 參數 [n_estimators, max_samples, max_features]
        n_jobs : int
            並行作業數（預設 -1）
        """
        self.model_type = model_type.lower()
        self.cv_scoring = cv_scoring  # ⭐ 新增這行
        self.use_hyperopt = use_hyperopt
        self.use_cv = use_cv
        self.cv_splits = cv_splits
        self.pct_embargo = pct_embargo
        self.hyperopt_method = hyperopt_method
        self.n_iter = n_iter
        self.bagging = bagging if bagging is not None else [0, None, 1.]
        self.n_jobs = n_jobs
        
        # 內部變量
        self.model = None
        self.scaler = None
        self.best_params = None
        self.cv_scores = None
        self.best_cv_scores = None
        self.X_train = None
        self.y_train = None
        self.t1_train = None
        self.sample_weight_train = None
        self.feature_columns = None
        
    # 在 PredictionPipeline 類中，修改 _create_pipeline 方法
    def _create_pipeline(self, n_classes: Optional[int] = None, **model_kwargs) -> MyPipeline:
        """
        建立模型 Pipeline，會依照實際類別數調整模型目標函式。
        """
        from sklearn.preprocessing import StandardScaler

        if self.model_type == 'xgboost':
            from xgboost import XGBClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'eval_metric': 'logloss',
                    'use_label_encoder': False,
                    'objective': 'binary:logistic',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'eval_metric': 'mlogloss',
                    'use_label_encoder': False,
                    'objective': 'multi:softprob',
                    'num_class': n_classes,
                }
            default_params.update(model_kwargs)
            clf = XGBClassifier(**default_params)

        elif self.model_type in ('lightgbm', 'lgb'):
            from lightgbm import LGBMClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'verbosity': -1,
                    'force_col_wise': True,
                    'objective': 'binary',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'verbosity': -1,
                    'force_col_wise': True,
                    'objective': 'multiclass',
                    'num_class': n_classes,
                }
            default_params.update(model_kwargs)
            clf = LGBMClassifier(**default_params)

        elif self.model_type == 'catboost':
            from catboost import CatBoostClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'verbose': False,
                    'thread_count': -1,
                    'loss_function': 'Logloss',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'verbose': False,
                    'thread_count': -1,
                    'loss_function': 'MultiClass',
                }
            default_params.update(model_kwargs)
            clf = CatBoostClassifier(**default_params)

        elif self.model_type == 'rf':
            from sklearn.ensemble import RandomForestClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = RandomForestClassifier(**default_params)

        elif self.model_type == 'et':
            from sklearn.ensemble import ExtraTreesClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = ExtraTreesClassifier(**default_params)

        elif self.model_type == 'gb':
            from sklearn.ensemble import GradientBoostingClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = GradientBoostingClassifier(**default_params)

        elif self.model_type == 'histgbm':
            from sklearn.ensemble import HistGradientBoostingClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = HistGradientBoostingClassifier(**default_params)

        elif self.model_type == 'dt':
            from sklearn.tree import DecisionTreeClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = DecisionTreeClassifier(**default_params)

        elif self.model_type == 'lr':
            from sklearn.linear_model import LogisticRegression
            default_params = {'random_state': 42, 'max_iter': 1000, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = LogisticRegression(**default_params)

        elif self.model_type == 'svm':
            from sklearn.svm import SVC
            default_params = {'random_state': 42, 'probability': True, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = SVC(**default_params)

        elif self.model_type == 'mlp':
            from sklearn.neural_network import MLPClassifier
            default_params = {'random_state': 42, 'max_iter': 1000}
            default_params.update(model_kwargs)
            clf = MLPClassifier(**default_params)

        elif self.model_type == 'ada':
            from sklearn.ensemble import AdaBoostClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = AdaBoostClassifier(**default_params)

        elif self.model_type == 'nb':
            from sklearn.naive_bayes import GaussianNB
            clf = GaussianNB()

        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")

        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb']
        if needs_scaling:
            pipe = MyPipeline([('scaler', StandardScaler()), (self.model_type, clf)])
        else:
            pipe = MyPipeline([(self.model_type, clf)])

        return pipe
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        """
        print(f"🚀 取得 {self.model_type} 模型參數網格")
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
        
        elif self.model_type == 'lightgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10, -1],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__num_leaves': [15, 31, 50, 100],
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_samples': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__num_leaves': [31, 50],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
        
        elif self.model_type == 'catboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300, 500],
                    f'{self.model_type}__depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__l2_leaf_reg': [1, 3, 5, 7],
                    f'{self.model_type}__border_count': [32, 64, 128],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300],
                    f'{self.model_type}__depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__l2_leaf_reg': [3, 5],
                }
                return param_grid
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500, 1000],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': logUniform(1e-3, 1e2),
                    f'{self.model_type}__min_samples_leaf': logUniform(1e-3, 1e1),
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                    f'{self.model_type}__bootstrap': [True, False]
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None]
                }
                return param_grid
        
        elif self.model_type == 'et':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__max_features': ['sqrt', 'log2'],
                }
                return param_grid
        
        elif self.model_type == 'gb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200],
                    f'{self.model_type}__max_depth': [3, 5],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                }
                return param_grid
        
        elif self.model_type == 'histgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200, 300],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__min_samples_leaf': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                }
                return param_grid
        
        elif self.model_type == 'dt':
            param_grid = {
                f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                f'{self.model_type}__min_samples_split': [2, 5, 10],
                f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                f'{self.model_type}__max_features': ['sqrt', 'log2', None],
            }
            return param_grid
        
        elif self.model_type == 'lr':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-3, 1e3),
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.01, 0.1, 1.0, 10.0],
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
        
        elif self.model_type == 'svm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-2, 1e2),
                    f'{self.model_type}__gamma': logUniform(1e-4, 1e-1),
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.1, 1.0, 10.0],
                    f'{self.model_type}__gamma': ['scale', 'auto', 0.001, 0.01],
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
        
        elif self.model_type == 'mlp':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                    f'{self.model_type}__alpha': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50)],
                    f'{self.model_type}__alpha': [0.0001, 0.001, 0.01],
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
        
        elif self.model_type == 'ada':
            param_grid = {
                f'{self.model_type}__n_estimators': [50, 100, 200],
                f'{self.model_type}__learning_rate': [0.01, 0.1, 1.0],
            }
            return param_grid   
        
        elif self.model_type == 'knn':
            param_grid = {
                f'{self.model_type}__n_neighbors': [3, 5, 7, 10, 15],
                f'{self.model_type}__weights': ['uniform', 'distance'],
                f'{self.model_type}__metric': ['euclidean', 'manhattan', 'minkowski'],
            }
            return param_grid
        elif self.model_type == 'nb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__var_smoothing': logUniform(1e-9, 1e-3),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
                }
                return param_grid
        elif self.model_type == 'qda':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__reg_param': logUniform(1e-3, 1.0),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__reg_param': [0.0, 0.1, 0.5, 1.0],
                }   
                return param_grid
        else:
            # 如果模型類型沒有對應的參數網格，返回空字典
            param_grid = {}
            print(f"⚠️ 警告: 模型類型 '{self.model_type}' 沒有定義參數網格，使用空字典")
        
        return param_grid
    
    def fit(
        self,
        X: pd.DataFrame,
        y: pd.Series,
        t1: Optional[pd.Series],
        sample_weight: Optional[pd.Series] = None,
        **model_kwargs,
    ) -> "PredictionPipeline":
        """
        訓練模型（整合 PurgedKFold、超參數搜尋）
        """
        from sklearn.preprocessing import LabelEncoder

        print("=" * 60)
        print(f"🚀 PredictionPipeline 開始訓練 ({self.model_type.upper()})")
        print("=" * 60)

        # 步驟0：標籤重新編碼
        print("\n" + "=" * 60)
        print("📊 步驟0：重新編碼標籤")
        print("=" * 60)
        original_unique = sorted(y.unique())
        print(f"原始標籤唯一值: {original_unique}")
        print(f"原始標籤分布: {y.value_counts().to_dict()}")

        label_encoder = LabelEncoder()
        y_encoded = pd.Series(label_encoder.fit_transform(y), index=y.index)
        self.label_encoder_ = label_encoder

        print(f"編碼後標籤唯一值: {sorted(y_encoded.unique())}")
        print(f"編碼後標籤分布: {y_encoded.value_counts().to_dict()}")
        print(f"編碼映射: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

        n_classes = len(label_encoder.classes_)
        print(f"實際類別數: {n_classes}")

        y = y_encoded

        # 步驟1：索引對齊
        print("\n" + "=" * 60)
        print("📊 步驟1：數據對齊與驗證")
        print("=" * 60)

        common_idx = X.index.intersection(y.index)
        if t1 is not None:
            common_idx = common_idx.intersection(t1.index)
        if sample_weight is not None:
            common_idx = common_idx.intersection(sample_weight.index)

        X = X.loc[common_idx]
        y = y.loc[common_idx]
        if t1 is not None:
            t1 = t1.loc[common_idx]
        if sample_weight is not None:
            sample_weight = sample_weight.loc[common_idx]

        print(f"對齊後樣本數: {len(X):,}")
        print(f"對齊後標籤分布: {y.value_counts().to_dict()}")

        self.X_train = X
        self.y_train = y
        self.t1_train = t1
        self.sample_weight_train = sample_weight
        self.feature_columns = X.columns.tolist()

        # 步驟2：建立基準模型
        print("\n" + "=" * 60)
        print("📊 步驟2：建立基準模型")
        print("=" * 60)
        baseline_pipe = self._create_pipeline(n_classes=n_classes, **model_kwargs)

        # 步驟3：PurgedKFold 交叉驗證
        if self.use_cv:
            print("\n" + "=" * 60)
            print("📊 步驟3：執行 PurgedKFold 交叉驗證")
            print("=" * 60)

            cv = PurgedKFold(
                n_splits=self.cv_splits,
                t1=self.t1_train,
                pctEmbargo=self.pct_embargo,
            )
            scoring = self.cv_scoring
            print("交叉驗證配置:")
            print(f"  K 折數: {self.cv_splits}")
            print(f"  禁制比例: {self.pct_embargo:.2%}")
            print(f"  評分方法: {scoring}")

            self.cv_scores = cvScore(
                clf=baseline_pipe,
                X=self.X_train,
                y=self.y_train,
                sample_weight=self.sample_weight_train,
                scoring=scoring,
                cvGen=cv,
            )
            print("\n✅ 基準模型交叉驗證完成")
            print(f"   分數: {self.cv_scores}")

        # 步驟4：超參數搜尋
        if self.use_hyperopt:
            print("\n" + "=" * 60)
            print("📊 步驟4：超參數搜尋")
            print("=" * 60)

            param_grid = self._get_param_grid(
                use_randomized=(self.hyperopt_method == 'randomized')
            )
            print(f"超參數搜尋方式: {self.hyperopt_method}")
            print(f"參數空間大小: {len(param_grid)}")

            pipe_for_search = self._create_pipeline(n_classes=n_classes, **model_kwargs)
            self.model = clfHyperFit(
                feat=self.X_train,
                lbl=self.y_train,
                t1=self.t1_train,
                pipe_clf=pipe_for_search,
                param_grid=param_grid,
                cv=self.cv_splits,
                bagging=self.bagging,
                rndSearchIter=self.n_iter if self.hyperopt_method == 'randomized' else 0,
                n_jobs=self.n_jobs,
                pctEmbargo=self.pct_embargo,
                scoring=self.cv_scoring,
                sample_weight=(
                    self.sample_weight_train.values
                    if self.sample_weight_train is not None
                    else None
                ),
            )
            print("\n✅ 超參數搜尋完成")
        else:
            print("\n" + "=" * 60)
            print("📊 步驟4：使用預設超參數訓練")
            print("=" * 60)
            self.model = baseline_pipe
            self.model.fit(
                self.X_train,
                self.y_train,
                **(
                    {}
                    if self.sample_weight_train is None
                    else {f"{self.model_type}__sample_weight": self.sample_weight_train.values}
                ),
            )

        # 步驟5：建立標準化器（如有需要）
        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb']
        if needs_scaling:
            from sklearn.preprocessing import StandardScaler
            self.scaler = StandardScaler()
            self.scaler.fit(self.X_train)
        else:
            self.scaler = None

        print("\n✅ 模型訓練完成")
        return self
    
    def predict(self, X: pd.DataFrame):
        """
        預測
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
            
        Returns:
        --------
        predictions : np.ndarray
            預測標籤（編碼後的）
        probabilities : np.ndarray
            預測概率
        """
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        # 預測
        predictions_encoded = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        # 如果需要，可以將編碼後的標籤轉回原始標籤
        # 但通常我們保持編碼後的標籤，讓調用者自己處理映射
        
        return predictions_encoded, probabilities
    
    def evaluate(self,
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """
        評估模型性能
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            真實標籤
        sample_weight : pd.Series, optional
            樣本權重
            
        Returns:
        --------
        metrics : dict
            評估指標
        """
        from sklearn.metrics import (accuracy_score, f1_score, 
                                   precision_score, recall_score,
                                   roc_auc_score)
        
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        print("=" * 60)
        print("📊 評估模型性能")
        print("=" * 60)
        
        # 預測
        predictions, probabilities = self.predict(X)
        
        # 計算指標
        metrics = {
            'accuracy': accuracy_score(y, predictions, sample_weight=sample_weight),
            'precision': precision_score(y, predictions, average='weighted', 
                                        sample_weight=sample_weight, zero_division=0),
            'recall': recall_score(y, predictions, average='weighted', 
                                  sample_weight=sample_weight, zero_division=0),
            'f1': f1_score(y, predictions, average='weighted', 
                          sample_weight=sample_weight, zero_division=0),
        }
        
        # 如果是二分類，計算更多指標
        if len(np.unique(y)) == 2:
            metrics['f1_binary'] = f1_score(y, predictions, sample_weight=sample_weight, zero_division=0)
            metrics['roc_auc'] = roc_auc_score(y, probabilities[:, 1], sample_weight=sample_weight)
        
        # 打印結果
        print(f"\n評估指標:")
        for key, value in metrics.items():
            print(f"  {key}: {value:.4f}")
        
        # 混淆矩陣
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
        print(f"\n混淆矩陣:")
        print(cm)
        
        print("=" * 60)
        
        return metrics
    
    def plot_results(self,
                    X: pd.DataFrame = None,
                    y: pd.Series = None,
                    sample_weight: Optional[pd.Series] = None):
        """
        繪製結果（交叉驗證分數、預測結果等）
        
        參數:
        -----------
        X : pd.DataFrame, optional
            特徵矩陣（用於預測和評估）
        y : pd.Series, optional
            真實標籤（用於評估）
        sample_weight : pd.Series, optional
            樣本權重
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 交叉驗證分數對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.6, 
                          label='Baseline Model', color='blue', width=0.4)
            axes[0, 0].bar([i + 0.4 for i in range(len(self.best_cv_scores))], 
                          self.best_cv_scores, alpha=0.6, 
                          label='Best Model (After Tuning)', color='red', width=0.4)
            axes[0, 0].axhline(self.cv_scores.mean(), color='blue', linestyle='--', 
                             label=f'Baseline Mean: {self.cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].axhline(self.best_cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Best Mean: {self.best_cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores: Baseline vs Best Model', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].set_xticks(range(len(self.cv_scores)))
            axes[0, 0].set_xticklabels([f'Fold {i+1}' for i in range(len(self.cv_scores))])
        elif self.cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.7, color='blue')
            axes[0, 0].axhline(self.cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Mean: {self.cv_scores.mean():.4f}', linewidth=2)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
        else:
            axes[0, 0].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
        
        # 2. 分數分布對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores, self.best_cv_scores], 
                             labels=['Baseline', 'Best Model'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].scatter([2] * len(self.best_cv_scores), self.best_cv_scores, 
                             alpha=0.3, s=30, color='red')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution Comparison', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        elif self.cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores], labels=['Baseline'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
        
        # 3. 預測結果（如果有 X 和 y）
        if X is not None and y is not None:
            predictions, probabilities = self.predict(X)
            
            # 混淆矩陣
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
            sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', ax=axes[1, 0])  # 使用浮點數格式
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 0].set_ylabel('True Label', fontsize=12)
            axes[1, 0].set_xlabel('Predicted Label', fontsize=12)
            
            # 預測分布
            pred_counts = pd.Series(np.ravel(predictions)).value_counts().sort_index()
            axes[1, 1].bar(pred_counts.index, pred_counts.values, alpha=0.7)
            axes[1, 1].set_xlabel('Predicted Label', fontsize=12)
            axes[1, 1].set_ylabel('Count', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 1].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
        
        plt.tight_layout()
        plt.show()

整理特徵目標與切割數據集

In [ ]:
final_weights = final_weights_B

In [ ]:

# =====================================================
# 步驟10：數據準備（修正對齊問題）
# =====================================================

# =====================================================
# 將篩選結果應用到完整數據集
# =====================================================
print(f"\n" + "=" * 60)
print("📊 應用篩選結果到完整數據集")
print("=" * 60)

# 只保留篩選後的特徵
X_filtered = X[filter_pipeline.selected_features].copy()

print(f"篩選前: {len(X.columns):,} 個特徵")
print(f"篩選後: {len(X_filtered.columns):,} 個特徵")
print(f"特徵減少: {len(X.columns) - len(X_filtered.columns):,} 個 ({(1 - len(X_filtered.columns) / len(X.columns)) * 100:.2f}%)")

# 更新 X 為篩選後的特徵
X = X_filtered

# =====================================================
# 數據分割：訓練集和測試集
# =====================================================

# 方法 1：如果 sample_weight 是 SampleWeight 實例
if hasattr(sample_weight, 'sample_weights'):
    # sample_weight 是 SampleWeight 實例
    sample_weight_series = sample_weight.sample_weights
else:
    # sample_weight 已經是 pd.Series
    sample_weight_series = sample_weight

# 分割數據
X_train = X.loc[(X.index >= train_start) & (X.index < test_start)].copy()
X_test = X.loc[X.index >= test_start].copy()
y_train = y_mapped.loc[(y_mapped.index >= train_start) & (y_mapped.index < test_start)].copy()
y_test = y_mapped.loc[y_mapped.index >= test_start].copy()
t1_train = t1.loc[(t1.index >= train_start) & (t1.index < test_start)].copy()
t1_test = t1.loc[t1.index >= test_start].copy()

# 分割樣本權重
if sample_weight_series is not None:
    sample_weight_train = sample_weight_series.loc[
        (sample_weight_series.index >= train_start) & 
        (sample_weight_series.index < test_start)
    ].copy()
    sample_weight_test = sample_weight_series.loc[
        sample_weight_series.index >= test_start
    ].copy()
else:
    sample_weight_train = None
    sample_weight_test = None

print(f"訓練集: {len(X_train):,} 個樣本, {len(X_train.columns):,} 個特徵")
print(f"測試集: {len(X_test):,} 個樣本, {len(X_test.columns):,} 個特徵")
print(f"訓練集時間範圍: {X_train.index.min()} 至 {X_train.index.max()}")
print(f"測試集時間範圍: {X_test.index.min()} 至 {X_test.index.max()}")

print(f"\n✅ 步驟6.5 完成！")
print(f"   特徵已合併並篩選")
print(f"   訓練/測試集已切分")
print(f"   可以進行下一步模型訓練")

# =====================================================
# 最終驗證
# =====================================================
print(f"\n" + "=" * 60)
print("✅ 數據準備完成")
print("=" * 60)
print(f"訓練集:")
print(f"  特徵數: {len(X_train.columns):,}")
print(f"  樣本數: {len(X_train):,}")
print(f"  標籤分布: {y_train.value_counts().to_dict()}")
print(f"\n測試集:")
print(f"  特徵數: {len(X_test.columns):,}")
print(f"  樣本數: {len(X_test):,}")
print(f"  標籤分布: {y_test.value_counts().to_dict()}")

# 顯示特徵來源統計
print(f"\n📊 特徵來源統計:")
feature_source_count = {}
for name, df_feat in available_features.items():
    numeric_cols = df_feat.select_dtypes(include=[np.number]).columns
    final_cols = [col for col in numeric_cols if col in X.columns]
    if len(final_cols) > 0:
        feature_source_count[name] = len(final_cols)
        print(f"  {name}: {len(final_cols)} 個特徵")

if len(X_train.columns) == 0:
    raise ValueError("❌ 錯誤：訓練集特徵數量為 0！")

if len(X_train) == 0:
    raise ValueError("❌ 錯誤：訓練集樣本數量為 0！")

print("=" * 60)

In [ ]:
X

In [ ]:
y

執行模型

In [ ]:
# =====================================================
# 步驟11：模型訓練與預測
# =====================================================

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, f1_score, precision_score, 
                             recall_score, confusion_matrix, roc_auc_score)

print("=" * 60)
print("🚀 步驟11：模型訓練與預測")
print("=" * 60)

# =====================================================
# 第1步：初始化 Pipeline
# =====================================================
print(f"\n1️⃣ 初始化 PredictionPipeline...")

    # 支援的模型類型:
    # -----
    # - 'xgboost': XGBoost
    # - 'lightgbm': LightGBM
    # - 'catboost': CatBoost
    # - 'rf': Random Forest
    # - 'et': Extra Trees
    # - 'gb': Gradient Boosting
    # - 'histgbm': HistGradientBoosting
    # - 'dt': Decision Tree
    # - 'lr': Logistic Regression
    # - 'svm': Support Vector Machine
    # - 'mlp': Multi-Layer Perceptron
    # - 'ada': AdaBoost
    # - 'nb': Naive Bayes


pipeline = PredictionPipeline(
    model_type='lightgbm',  
    cv_scoring='f1_weighted',  # ['neg_log_loss', 'accuracy', 'f1', 'f1_weighted', 'precision', 'recall', 'roc_auc'].
    use_hyperopt=True,
    use_cv=True,
    cv_splits=5,
    pct_embargo=0.01,
    hyperopt_method='grid',  # 'grid' 或 'random'
    n_iter=20,
    n_jobs=-1
)

print(f"✅ Pipeline 初始化完成")
print(f"   模型類型: {pipeline.model_type}")
print(f"   使用超參數優化: {pipeline.use_hyperopt}")
print(f"   使用交叉驗證: {pipeline.use_cv}")

# =====================================================
# 第2步：訓練模型
# =====================================================
print(f"\n2️⃣ 開始訓練模型...")
print(f"   訓練集特徵數: {len(X_train.columns):,}")
print(f"   訓練集樣本數: {len(X_train):,}")
print(f"   標籤分布: {y_train.value_counts().to_dict()}")

pipeline.fit(X_train, y_train, t1_train, sample_weight_train)

print(f"✅ 模型訓練完成")

# =====================================================
# 第3步：可視化訓練結果
# =====================================================
print(f"\n3️⃣ 可視化訓練結果...")

pipeline.plot_results(X=X_test, y=y_test, sample_weight=sample_weight_test)

# =====================================================
# 第4步：預測
# =====================================================
print(f"\n4️⃣ 開始預測...")
print(f"   測試集特徵數: {len(X_test.columns):,}")
print(f"   測試集樣本數: {len(X_test):,}")

predictions_test_mapped, probabilities_test = pipeline.predict(X_test)

# 將預測結果映射回原始標籤 (-1, 0, 1)
predictions_test = pd.Series(np.ravel(predictions_test_mapped)).map(reverse_mapping).values

print(f"✅ 預測完成")
print(f"   預測樣本數: {len(predictions_test):,}")

# 檢查實際的類別數量
n_classes = probabilities_test.shape[1]
print(f"   實際類別數: {n_classes}")
print(f"   概率矩陣形狀: {probabilities_test.shape}")

# =====================================================
# 第5步：可視化概率分布
# =====================================================
print(f"\n5️⃣ 可視化概率分布...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 根據實際類別數量動態設置顏色和標籤
if n_classes == 2:
    colors = ['red', 'blue']
    class_labels = ['Class 0 (跌)', 'Class 1 (不跌)']
    class_labels_short = ['Class 0\n(跌)', 'Class 1\n(不跌)']
elif n_classes == 3:
    colors = ['red', 'blue', 'green']
    class_labels = ['Class 0 (原-1)', 'Class 1 (原0)', 'Class 2 (原1)']
    class_labels_short = ['Class 0\n(原-1)', 'Class 1\n(原0)', 'Class 2\n(原1)']
else:
    colors = ['red', 'blue', 'green', 'orange', 'purple'][:n_classes]
    class_labels = [f'Class {i}' for i in range(n_classes)]
    class_labels_short = [f'Class {i}' for i in range(n_classes)]

# 概率分布直方圖
for i in range(n_classes):
    axes[0, 0].hist(probabilities_test[:, i], bins=50, alpha=0.7, 
                    label=class_labels[i], color=colors[i])
axes[0, 0].set_xlabel('Probability', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Probability Distribution by Class', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 平均概率
mean_probs = probabilities_test.mean(axis=0)
axes[0, 1].bar(range(n_classes), mean_probs, alpha=0.7, color=colors)
axes[0, 1].set_xticks(range(n_classes))
axes[0, 1].set_xticklabels(class_labels_short)
axes[0, 1].set_ylabel('Mean Probability', fontsize=12)
axes[0, 1].set_title('Mean Probability by Class', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# 預測類別分布
pred_counts = pd.Series(predictions_test_mapped).value_counts().sort_index()
axes[1, 0].bar(pred_counts.index, pred_counts.values, alpha=0.7, 
               color=[colors[i] if i < len(colors) else 'gray' for i in pred_counts.index])
axes[1, 0].set_xlabel('Predicted Class', fontsize=12)
axes[1, 0].set_ylabel('Count', fontsize=12)
axes[1, 0].set_title('Predicted Class Distribution', fontsize=14)
axes[1, 0].set_xticks(range(n_classes))
axes[1, 0].set_xticklabels(class_labels_short)
axes[1, 0].grid(True, alpha=0.3)

# 真實類別分布
true_counts = y_test.value_counts().sort_index()
axes[1, 1].bar(true_counts.index, true_counts.values, alpha=0.7, 
               color=[colors[i] if i < len(colors) else 'gray' for i in true_counts.index])
axes[1, 1].set_xlabel('True Class', fontsize=12)
axes[1, 1].set_ylabel('Count', fontsize=12)
axes[1, 1].set_title('True Class Distribution (Test Set)', fontsize=14)
axes[1, 1].set_xticks(range(n_classes))
axes[1, 1].set_xticklabels(class_labels_short)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印概率統計
print("\n" + "=" * 60)
print("📊 預測概率統計")
print("=" * 60)
for i in range(n_classes):
    print(f"{class_labels[i]} 概率: 平均={probabilities_test[:, i].mean():.4f}, 中位數={np.median(probabilities_test[:, i]):.4f}")
print("=" * 60)

# =====================================================
# 第6步：評估模型性能
# =====================================================
print(f"\n6️⃣ 評估模型性能...")

# 獲取原始標籤（未映射的）
y_test_original = y.loc[y_test.index]

# 計算評估指標
metrics_test = {
    'accuracy': accuracy_score(y_test_original, predictions_test, sample_weight=sample_weight_test),
    'precision': precision_score(y_test_original, predictions_test, average='weighted', 
                                 sample_weight=sample_weight_test, zero_division=0),
    'recall': recall_score(y_test_original, predictions_test, average='weighted', 
                          sample_weight=sample_weight_test, zero_division=0),
    'f1': f1_score(y_test_original, predictions_test, average='weighted', 
                  sample_weight=sample_weight_test, zero_division=0),
}

print("\n" + "=" * 60)
print("📊 測試集評估結果")
print("=" * 60)
print(f"評估樣本數: {len(y_test_original):,}")
for key, value in metrics_test.items():
    print(f"  {key}: {value:.4f}")

# 混淆矩陣
cm = confusion_matrix(y_test_original, predictions_test, sample_weight=sample_weight_test)
print(f"\n混淆矩陣:")
print(cm)

# 計算每個類別的詳細指標
print(f"\n各類別詳細指標:")
unique_labels = sorted(y_test_original.unique())
for i, label in enumerate(unique_labels):
    if i < len(cm):
        true_pos = cm[i, i] if i < cm.shape[0] and i < cm.shape[1] else 0
        false_pos = cm[:, i].sum() - true_pos if i < cm.shape[1] else 0
        false_neg = cm[i, :].sum() - true_pos if i < cm.shape[0] else 0
        
        precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
        recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"  類別 {label}: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

print("=" * 60)

# =====================================================
# 第7步：創建預測結果 DataFrame
# =====================================================
print(f"\n7️⃣ 創建預測結果 DataFrame...")

pred_df = pd.DataFrame(index=X_test.index)
pred_df['t1'] = t1_test
pred_df['side'] = events.loc[X_test.index, 'side']
pred_df['true_label'] = y_test_original
pred_df['predicted_label'] = predictions_test

# 根據實際類別數量動態添加概率列
for i in range(n_classes):
    if n_classes == 2:
        prob_label = f'prob_class_{i}' + ('_跌' if i == 0 else '_不跌')
    else:
        prob_label = f'prob_class_{i}'
    pred_df[prob_label] = probabilities_test[:, i]

pred_df['prediction_confidence'] = np.max(probabilities_test, axis=1)
pred_df['is_correct'] = (pred_df['predicted_label'] == pred_df['true_label'])

# 交易信號：根據預測標籤和 side 生成
pred_df['trade_signal'] = np.where(
    (pred_df['predicted_label'] == 1) & (pred_df['side'] == 1), 1,
    np.where((pred_df['predicted_label'] == 1) & (pred_df['side'] == -1), -1, 0)
)

print(f"✅ 預測結果 DataFrame 創建完成")
print(f"   形狀: {pred_df.shape}")
print(f"   正確預測率: {pred_df['is_correct'].mean():.4f}")

# 顯示前幾筆結果
print(f"\n前 10 筆預測結果:")
print(pred_df.head(10))

# 顯示統計資訊
print(f"\n預測結果統計:")
print(f"  總樣本數: {len(pred_df):,}")
print(f"  正確預測數: {pred_df['is_correct'].sum():,}")
print(f"  錯誤預測數: {(~pred_df['is_correct']).sum():,}")
print(f"  平均置信度: {pred_df['prediction_confidence'].mean():.4f}")
print(f"  預測標籤分布:")
unique_pred_labels = sorted(pred_df['predicted_label'].unique())
for label in unique_pred_labels:
    count = (pred_df['predicted_label'] == label).sum()
    print(f"    {label}: {count:,}")

print("\n" + "=" * 60)
print("✅ 預測流程完成")
print("=" * 60)

# 顯示最終的預測結果 DataFrame
pred_df

In [ ]:
# pred_df.to_csv('Pred_lgb_23.csv')

所有模型輪流執行

In [ ]:
# # =====================================================
# # 自動遍歷所有模型並顯示混淆矩陣
# # =====================================================

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import (accuracy_score, f1_score, precision_score, 
#                              recall_score, confusion_matrix)
# from sklearn.preprocessing import LabelEncoder
# import numpy as np
# import pandas as pd

# print("=" * 60)
# print("🚀 自動遍歷所有模型 - 完整評估")
# print("=" * 60)

# # 所有支持的模型類型
# model_types = [
#     'xgboost',
#     'lightgbm',
#     'catboost',
#     'rf',
#     'et',
#     'gb',
#     'histgbm',
#     'dt',
#     'lr',
#     'svm',
#     'mlp',
#     'ada',
#     'nb'
# ]

# # =====================================================
# # 步驟1：檢查並清理標籤
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟1：檢查並清理標籤")
# print(f"{'='*60}")

# # 獲取原始標籤（未映射的）
# y_test_original = y.loc[y_test.index]

# # 檢查標籤分布
# print(f"\n原始標籤分布:")
# print(f"   y_test_original 唯一值: {sorted(y_test_original.unique())}")
# print(f"   y_train 唯一值: {sorted(y_train.unique())}")

# # 檢查是否有未映射的值
# valid_labels = set(label_mapping.keys())
# y_test_unique = set(y_test_original.unique())
# y_train_unique = set(y.loc[y_train.index].unique())

# unmapped_test = y_test_unique - valid_labels
# unmapped_train = y_train_unique - valid_labels

# if unmapped_test or unmapped_train:
#     print(f"\n⚠️  發現未映射的標籤值:")
#     if unmapped_test:
#         print(f"   測試集中: {sorted(unmapped_test)}")
#     if unmapped_train:
#         print(f"   訓練集中: {sorted(unmapped_train)}")
#     print(f"   有效標籤: {sorted(valid_labels)}")
#     print(f"   將過濾掉未映射的標籤...")

# # 過濾掉未映射的標籤（只保留 -1, 0, 1）
# y_test_original_cleaned = y_test_original[y_test_original.isin(valid_labels)].copy()
# y_train_original_cleaned = y.loc[y_train.index][y.loc[y_train.index].isin(valid_labels)].copy()

# print(f"\n過濾後:")
# print(f"   測試集樣本數: {len(y_test_original_cleaned):,} (原始: {len(y_test_original):,})")
# print(f"   訓練集樣本數: {len(y_train_original_cleaned):,} (原始: {len(y_train):,})")

# # =====================================================
# # 步驟2：對齊數據索引
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟2：對齊數據索引")
# print(f"{'='*60}")

# # 確保訓練集和測試集的索引對齊（只使用清理後的標籤）
# X_train_cleaned = X_train.loc[y_train_original_cleaned.index]
# y_train_mapped_cleaned = y_train.loc[y_train_original_cleaned.index]
# t1_train_cleaned = t1_train.loc[y_train_original_cleaned.index] if t1_train is not None else None
# sample_weight_train_cleaned = sample_weight_train.loc[y_train_original_cleaned.index] if sample_weight_train is not None else None

# X_test_cleaned = X_test.loc[y_test_original_cleaned.index]
# y_test_mapped_cleaned = y_test.loc[y_test_original_cleaned.index]
# t1_test_cleaned = t1_test.loc[y_test_original_cleaned.index] if t1_test is not None else None
# sample_weight_test_cleaned = sample_weight_test.loc[y_test_original_cleaned.index] if sample_weight_test is not None else None

# # 檢查映射後的標籤分布
# print(f"\n映射後的標籤分布:")
# print(f"   y_train_mapped_cleaned 唯一值: {sorted(y_train_mapped_cleaned.unique())}")
# print(f"   y_test_mapped_cleaned 唯一值: {sorted(y_test_mapped_cleaned.unique())}")

# # =====================================================
# # 步驟3：重新編碼標籤（確保從0開始且連續）
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟3：重新編碼標籤")
# print(f"{'='*60}")

# # 使用 LabelEncoder 重新編碼，確保標籤從 0 開始且連續
# label_encoder = LabelEncoder()
# # 合併訓練集和測試集的所有唯一標籤來擬合 encoder
# all_unique_labels = sorted(set(list(y_train_mapped_cleaned.unique()) + list(y_test_mapped_cleaned.unique())))
# label_encoder.fit(all_unique_labels)

# print(f"   原始唯一標籤: {all_unique_labels}")
# print(f"   編碼後標籤: {label_encoder.classes_.tolist()}")
# print(f"   編碼映射: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

# # 編碼訓練集和測試集標籤
# y_train_encoded = pd.Series(
#     label_encoder.transform(y_train_mapped_cleaned),
#     index=y_train_mapped_cleaned.index
# )
# y_test_encoded = pd.Series(
#     label_encoder.transform(y_test_mapped_cleaned),
#     index=y_test_mapped_cleaned.index
# )

# print(f"   編碼後訓練集標籤分布: {y_train_encoded.value_counts().to_dict()}")
# print(f"   編碼後測試集標籤分布: {y_test_encoded.value_counts().to_dict()}")

# # 獲取實際的類別數
# n_classes_actual = len(label_encoder.classes_)
# print(f"\n實際類別數: {n_classes_actual}")

# # 獲取所有唯一的原始標籤值（用於動態處理）
# unique_labels_original = sorted(y_test_original_cleaned.unique())
# n_classes_original = len(unique_labels_original)

# print(f"原始標籤類別: {unique_labels_original}")
# print(f"原始類別數: {n_classes_original}")

# # =====================================================
# # 步驟4：確保所有數據索引完全對齊
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟4：最終數據對齊檢查")
# print(f"{'='*60}")

# # 訓練集索引對齊
# common_idx_train = X_train_cleaned.index.intersection(y_train_encoded.index)
# if t1_train_cleaned is not None:
#     common_idx_train = common_idx_train.intersection(t1_train_cleaned.index)
# if sample_weight_train_cleaned is not None:
#     common_idx_train = common_idx_train.intersection(sample_weight_train_cleaned.index)

# X_train_final = X_train_cleaned.loc[common_idx_train]
# y_train_final = y_train_encoded.loc[common_idx_train]
# t1_train_final = t1_train_cleaned.loc[common_idx_train] if t1_train_cleaned is not None else None
# sample_weight_train_final = sample_weight_train_cleaned.loc[common_idx_train] if sample_weight_train_cleaned is not None else None

# # 測試集索引對齊
# common_idx_test = X_test_cleaned.index.intersection(y_test_encoded.index)
# if t1_test_cleaned is not None:
#     common_idx_test = common_idx_test.intersection(t1_test_cleaned.index)
# if sample_weight_test_cleaned is not None:
#     common_idx_test = common_idx_test.intersection(sample_weight_test_cleaned.index)

# X_test_final = X_test_cleaned.loc[common_idx_test]
# y_test_final = y_test_encoded.loc[common_idx_test]
# t1_test_final = t1_test_cleaned.loc[common_idx_test] if t1_test_cleaned is not None else None
# sample_weight_test_final = sample_weight_test_cleaned.loc[common_idx_test] if sample_weight_test_cleaned is not None else None

# # 獲取對應的原始標籤
# y_test_original_final = y_test_original_cleaned.loc[common_idx_test]

# print(f"   對齊後訓練集樣本數: {len(X_train_final):,}")
# print(f"   對齊後測試集樣本數: {len(X_test_final):,}")
# print(f"   訓練集索引對齊: {X_train_final.index.equals(y_train_final.index)}")
# print(f"   測試集索引對齊: {X_test_final.index.equals(y_test_final.index)}")

# # =====================================================
# # 步驟5：遍歷所有模型
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟5：遍歷所有模型進行訓練和評估")
# print(f"{'='*60}")

# # 存儲所有模型的結果
# results_summary = []

# for i, model_type in enumerate(model_types, 1):
#     print(f"\n{'='*60}")
#     print(f"📊 [{i}/{len(model_types)}] 模型: {model_type.upper()}")
#     print(f"{'='*60}")
    
#     try:
#         # 初始化 Pipeline
#         pipeline = PredictionPipeline(
#             model_type=model_type,
#             cv_scoring='f1',  # 使用 f1 作為評分標準
#             use_hyperopt=True,
#             use_cv=True,
#             cv_splits=5,
#             pct_embargo=0.01,
#             hyperopt_method='grid',  # 'grid' 或 'random'
#             n_iter=20,
#             n_jobs=-1
#         )
        
#         # 訓練模型
#         print(f"\n訓練中...")
#         print(f"   訓練集樣本數: {len(X_train_final):,}")
#         print(f"   訓練集標籤分布: {y_train_final.value_counts().to_dict()}")
#         print(f"   類別數: {n_classes_actual}")
        
#         pipeline.fit(X_train_final, y_train_final, t1_train_final, sample_weight_train_final)
        
#         # 預測
#         print(f"\n預測中...")
#         print(f"   測試集樣本數: {len(X_test_final):,}")
        
#         predictions_test_encoded, probabilities_test = pipeline.predict(X_test_final)
        
#         # 檢查實際的類別數量
#         n_classes_predicted = probabilities_test.shape[1] if probabilities_test is not None else len(np.unique(predictions_test_encoded))
#         print(f"   模型預測類別數: {n_classes_predicted}")
        
#         # 將編碼後的預測結果轉回映射後的標籤
#         predictions_test_mapped = label_encoder.inverse_transform(predictions_test_encoded)
        
#         # 將映射後的標籤轉回原始標籤 (-1, 0, 1)
#         predictions_test = pd.Series(predictions_test_mapped).map(reverse_mapping).values
        
#         # 計算評估指標
#         accuracy = accuracy_score(y_test_original_final, predictions_test, 
#                                 sample_weight=sample_weight_test_final)
#         precision = precision_score(y_test_original_final, predictions_test, average='weighted', 
#                                    sample_weight=sample_weight_test_final, zero_division=0)
#         recall = recall_score(y_test_original_final, predictions_test, average='weighted', 
#                             sample_weight=sample_weight_test_final, zero_division=0)
#         f1 = f1_score(y_test_original_final, predictions_test, average='weighted', 
#                     sample_weight=sample_weight_test_final, zero_division=0)
        
#         # 計算混淆矩陣
#         cm = confusion_matrix(y_test_original_final, predictions_test, 
#                             labels=unique_labels_original,
#                             sample_weight=sample_weight_test_final)
        
#         # 獲取實際出現在預測和真實標籤中的所有唯一值
#         all_unique_labels = sorted(set(list(y_test_original_final.unique()) + list(predictions_test)))
#         actual_n_classes = len(all_unique_labels)
        
#         print(f"\n混淆矩陣形狀: {cm.shape}")
#         print(f"實際混淆矩陣類別數: {actual_n_classes}")
        
#         # 動態生成標籤名稱
#         if actual_n_classes == 2:
#             if 0 in all_unique_labels and 1 in all_unique_labels:
#                 pred_labels = ['Pred 0 (跌)', 'Pred 1 (不跌)']
#                 true_labels = ['True 0 (跌)', 'True 1 (不跌)']
#                 header_labels = ['預測0(跌)', '預測1(不跌)']
#             elif -1 in all_unique_labels and 1 in all_unique_labels:
#                 pred_labels = ['Pred -1 (跌)', 'Pred 1 (不跌)']
#                 true_labels = ['True -1 (跌)', 'True 1 (不跌)']
#                 header_labels = ['預測-1(跌)', '預測1(不跌)']
#             else:
#                 pred_labels = [f'Pred {label}' for label in all_unique_labels]
#                 true_labels = [f'True {label}' for label in all_unique_labels]
#                 header_labels = [f'預測{label}' for label in all_unique_labels]
#         elif actual_n_classes == 3:
#             pred_labels = ['Pred -1', 'Pred 0', 'Pred 1']
#             true_labels = ['True -1', 'True 0', 'True 1']
#             header_labels = ['預測-1', '預測0', '預測1']
#         else:
#             pred_labels = [f'Pred {label}' for label in all_unique_labels]
#             true_labels = [f'True {label}' for label in all_unique_labels]
#             header_labels = [f'預測{label}' for label in all_unique_labels]
        
#         # 顯示混淆矩陣（文本格式）
#         print(f"\n混淆矩陣 (Confusion Matrix):")
#         header_str = f"{'':>10}"
#         for label in header_labels:
#             header_str += f" {label:>12}"
#         print(header_str)
        
#         for i, true_label in enumerate(true_labels):
#             if i < cm.shape[0]:
#                 row_str = f"{true_label:>10}"
#                 for j in range(cm.shape[1]):
#                     row_str += f" {cm[i, j]:>12.0f}"
#                 print(row_str)
        
#         # 顯示評估指標
#         print(f"\n評估指標:")
#         print(f"  Accuracy: {accuracy:.4f}")
#         print(f"  Precision: {precision:.4f}")
#         print(f"  Recall: {recall:.4f}")
#         print(f"  F1-Score: {f1:.4f}")
        
#         # 可視化混淆矩陣
#         plt.figure(figsize=(max(8, actual_n_classes * 2), max(6, actual_n_classes * 1.5)))
#         sns.heatmap(cm, annot=True, fmt='.0f', cmap='Blues', 
#                     xticklabels=pred_labels,
#                     yticklabels=true_labels,
#                     cbar_kws={'label': 'Count'})
#         plt.title(f'Confusion Matrix - {model_type.upper()}', fontsize=14, fontweight='bold')
#         plt.ylabel('True Label', fontsize=12)
#         plt.xlabel('Predicted Label', fontsize=12)
#         plt.tight_layout()
#         plt.show()
        
#         # 存儲結果
#         results_summary.append({
#             'model': model_type.upper(),
#             'accuracy': accuracy,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1,
#             'n_classes': actual_n_classes
#         })
        
#         print(f"✅ {model_type.upper()} 完成")
        
#     except Exception as e:
#         print(f"❌ {model_type.upper()} 失敗: {str(e)}")
#         import traceback
#         traceback.print_exc()
        
#         # 存儲失敗結果
#         results_summary.append({
#             'model': model_type.upper(),
#             'accuracy': np.nan,
#             'precision': np.nan,
#             'recall': np.nan,
#             'f1': np.nan,
#             'n_classes': np.nan,
#             'error': str(e)
#         })
#         continue

# # =====================================================
# # 步驟6：顯示所有模型結果摘要
# # =====================================================
# print(f"\n{'='*60}")
# print("📊 步驟6：所有模型結果摘要")
# print(f"{'='*60}")

# results_df = pd.DataFrame(results_summary)
# results_df = results_df.sort_values('f1', ascending=False, na_last=True)

# print("\n模型性能排名 (按 F1-Score 排序):")
# print("=" * 80)
# print(results_df.to_string(index=False))
# print("=" * 80)

# # 可視化結果比較
# if len(results_df) > 0:
#     successful_models = results_df.dropna(subset=['f1'])
    
#     if len(successful_models) > 0:
#         fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
#         # F1-Score 比較
#         axes[0, 0].barh(successful_models['model'], successful_models['f1'], color='steelblue')
#         axes[0, 0].set_xlabel('F1-Score', fontsize=12)
#         axes[0, 0].set_title('F1-Score by Model', fontsize=14, fontweight='bold')
#         axes[0, 0].grid(True, alpha=0.3, axis='x')
        
#         # Accuracy 比較
#         axes[0, 1].barh(successful_models['model'], successful_models['accuracy'], color='green')
#         axes[0, 1].set_xlabel('Accuracy', fontsize=12)
#         axes[0, 1].set_title('Accuracy by Model', fontsize=14, fontweight='bold')
#         axes[0, 1].grid(True, alpha=0.3, axis='x')
        
#         # Precision 比較
#         axes[1, 0].barh(successful_models['model'], successful_models['precision'], color='orange')
#         axes[1, 0].set_xlabel('Precision', fontsize=12)
#         axes[1, 0].set_title('Precision by Model', fontsize=14, fontweight='bold')
#         axes[1, 0].grid(True, alpha=0.3, axis='x')
        
#         # Recall 比較
#         axes[1, 1].barh(successful_models['model'], successful_models['recall'], color='red')
#         axes[1, 1].set_xlabel('Recall', fontsize=12)
#         axes[1, 1].set_title('Recall by Model', fontsize=14, fontweight='bold')
#         axes[1, 1].grid(True, alpha=0.3, axis='x')
        
#         plt.tight_layout()
#         plt.show()
        
#         # 顯示最佳模型
#         best_model = successful_models.iloc[0]
#         print(f"\n🏆 最佳模型: {best_model['model']}")
#         print(f"   F1-Score: {best_model['f1']:.4f}")
#         print(f"   Accuracy: {best_model['accuracy']:.4f}")
#         print(f"   Precision: {best_model['precision']:.4f}")
#         print(f"   Recall: {best_model['recall']:.4f}")

# print(f"\n{'='*60}")
# print("✅ 所有模型遍歷完成")
# print(f"{'='*60}")

# # 顯示最終結果摘要
# results_df

## 步驟11 ： 特徵重要性

In [ ]:
# =====================================================
# 步驟11：特徵重要性分析（使用已訓練的模型）
# =====================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
from scipy.stats import weightedtau
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("📊 步驟11：特徵重要性分析（使用已訓練的模型）")
print("=" * 60)

# 檢查是否有已訓練的模型
if 'pipeline' not in globals() or pipeline.model is None:
    raise ValueError("❌ 請先執行 pipeline.fit() 訓練模型")

print(f"✅ 使用已訓練的模型: {pipeline.model_type}")

# =====================================================
# 1️⃣ MDI (Mean Decrease Impurity) - 僅適用於樹模型
# =====================================================

def get_mdi_from_model(model, X_train):
    """
    從已訓練的樹模型獲取 MDI 重要性
    
    適用：XGBoost, LightGBM, Random Forest, Extra Trees, Decision Tree, Gradient Boosting
    """
    print(f"\n1️⃣ 計算 MDI (Mean Decrease Impurity)...")
    
    # 檢查模型是否有 feature_importances_ 屬性
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        
        # 創建 Series
        mdi_importance = pd.Series(importances, index=X_train.columns)
        mdi_importance = mdi_importance.sort_values(ascending=False)
        
        print(f"   ✅ MDI 計算完成")
        print(f"   前10個重要特徵:")
        for i, (feat, imp) in enumerate(mdi_importance.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: {imp:.6f}")
        
        return mdi_importance
    else:
        print(f"   ⚠️ 模型 {type(model).__name__} 不支援 MDI（非樹模型）")
        return None

# =====================================================
# 2️⃣ MDA (Mean Decrease Accuracy) - 使用已訓練的模型
# =====================================================

def calculate_mda_from_model(model, X_test, y_test, sample_weight_test=None,
                            scoring='neg_log_loss', n_permutations=1, random_state=42):
    """
    使用已訓練的模型計算 MDA (Mean Decrease Accuracy)
    
    適用：任意模型
    """
    print(f"\n2️⃣ 計算 MDA (Mean Decrease Accuracy)...")
    
    np.random.seed(random_state)
    
    # 計算基準分數
    if scoring == 'neg_log_loss':
        baseline_pred = model.predict_proba(X_test)
        baseline_score = -log_loss(y_test, baseline_pred, 
                                   sample_weight=sample_weight_test.values if sample_weight_test is not None else None)
    elif scoring == 'accuracy':
        baseline_pred = model.predict(X_test)
        baseline_score = accuracy_score(y_test, baseline_pred,
                                       sample_weight=sample_weight_test.values if sample_weight_test is not None else None)
    else:
        raise ValueError(f"不支援的評分方法: {scoring}")
    
    print(f"   基準分數 ({scoring}): {baseline_score:.6f}")
    
    # 對每個特徵進行置換測試
    mda_scores = {}
    
    for i, feature in enumerate(X_test.columns, 1):
        perm_scores = []
        
        for perm in range(n_permutations):
            # 複製測試集
            X_perm = X_test.copy()
            
            # 打亂該特徵的值
            perm_values = X_perm[feature].values.copy()
            np.random.shuffle(perm_values)
            X_perm[feature] = perm_values
            
            # 計算置換後的分數
            if scoring == 'neg_log_loss':
                perm_pred = model.predict_proba(X_perm)
                perm_score = -log_loss(y_test, perm_pred,
                                     sample_weight=sample_weight_test.values if sample_weight_test is not None else None)
            else:  # accuracy
                perm_pred = model.predict(X_perm)
                perm_score = accuracy_score(y_test, perm_pred,
                                          sample_weight=sample_weight_test.values if sample_weight_test is not None else None)
            
            perm_scores.append(perm_score)
        
        # 計算重要性（基準分數 - 置換後分數）
        avg_perm_score = np.mean(perm_scores)
        importance = baseline_score - avg_perm_score
        mda_scores[feature] = importance
        
        if (i - 1) % 50 == 0 or i == len(X_test.columns):
            print(f"   進度: {i}/{len(X_test.columns)} ({i/len(X_test.columns)*100:.1f}%)")
    
    # 創建 Series
    mda_importance = pd.Series(mda_scores)
    mda_importance = mda_importance.sort_values(ascending=False)
    
    print(f"   ✅ MDA 計算完成")
    print(f"   前10個重要特徵:")
    for i, (feat, imp) in enumerate(mda_importance.head(10).items(), 1):
        print(f"     {i:2d}. {feat}: {imp:.6f}")
    
    return mda_importance

# =====================================================
# 3️⃣ SFI (Single Feature Importance) - 使用已訓練的模型類型
# =====================================================

def calculate_sfi_from_model_type(model_type, X_train, y_train, t1_train, 
                                  sample_weight_train=None, cv_splits=5, 
                                  pct_embargo=0.01, scoring='neg_log_loss', 
                                  random_state=42):
    """
    使用相同的模型類型計算 SFI (Single Feature Importance)
    
    適用：任意模型
    """
    print(f"\n3️⃣ 計算 SFI (Single Feature Importance)...")
    
    # 輔助函數：Purged Time Series Split
    def purged_time_series_split(X, y, t1, n_splits=5, pct_embargo=0.01):
        indices = np.arange(len(X))
        mbrg = int(len(X) * pct_embargo)
        sorted_indices = X.index.argsort()
        test_size = len(X) // (n_splits + 1)
        
        for i in range(n_splits):
            test_start = (i + 1) * test_size
            test_end = min((i + 2) * test_size, len(X))
            test_idx = sorted_indices[test_start:test_end]
            train_end = test_start - mbrg
            train_idx = sorted_indices[:train_end]
            yield train_idx, test_idx
    
    # 輔助函數：交叉驗證評分
    def cv_score_single_feature(clf, X, y, sample_weight, scoring, cv):
        scores = []
        for train_idx, test_idx in cv:
            X_train_fold = X.iloc[train_idx]
            X_test_fold = X.iloc[test_idx]
            y_train_fold = y.iloc[train_idx]
            y_test_fold = y.iloc[test_idx]
            sw_train = sample_weight.iloc[train_idx].values if sample_weight is not None else None
            sw_test = sample_weight.iloc[test_idx].values if sample_weight is not None else None
            
            clf.fit(X_train_fold, y_train_fold, sample_weight=sw_train)
            
            if scoring == 'neg_log_loss':
                y_pred_proba = clf.predict_proba(X_test_fold)
                score = -log_loss(y_test_fold, y_pred_proba, sample_weight=sw_test)
            elif scoring == 'accuracy':
                y_pred = clf.predict(X_test_fold)
                score = accuracy_score(y_test_fold, y_pred, sample_weight=sw_test)
            scores.append(score)
        return np.array(scores)
    
    # 根據模型類型創建分類器
    if model_type == 'xgb' or model_type == 'xgboost':
        from xgboost import XGBClassifier
        clf = XGBClassifier(n_estimators=100, random_state=random_state, 
                           eval_metric='mlogloss', use_label_encoder=False)
    elif model_type == 'lgb' or model_type == 'lightgbm':
        from lightgbm import LGBMClassifier
        clf = LGBMClassifier(n_estimators=100, random_state=random_state, verbose=-1)
    elif model_type == 'rf' or model_type == 'random_forest':
        from sklearn.ensemble import RandomForestClassifier
        clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    elif model_type == 'et' or model_type == 'extra_trees':
        from sklearn.ensemble import ExtraTreesClassifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    elif model_type == 'gb' or model_type == 'gradient_boosting':
        from sklearn.ensemble import GradientBoostingClassifier
        clf = GradientBoostingClassifier(n_estimators=100, random_state=random_state)
    elif model_type == 'lr' or model_type == 'logistic_regression':
        from sklearn.linear_model import LogisticRegression
        clf = LogisticRegression(random_state=random_state, max_iter=1000, solver='lbfgs')
    elif model_type == 'svm':
        from sklearn.svm import SVC
        clf = SVC(probability=True, random_state=random_state)
    elif model_type == 'mlp':
        from sklearn.neural_network import MLPClassifier
        clf = MLPClassifier(random_state=random_state, max_iter=500)
    else:
        print(f"   ⚠️ 模型類型 {model_type} 不支援 SFI，跳過")
        return None
    
    # 對每個特徵單獨訓練模型
    sfi_scores = {}
    
    for i, feature in enumerate(X_train.columns, 1):
        # 只使用單一特徵
        X_single = X_train[[feature]]
        
        # 交叉驗證
        try:
            cv = purged_time_series_split(X_single, y_train, t1_train, 
                                         n_splits=cv_splits, pct_embargo=pct_embargo)
            scores = cv_score_single_feature(clf, X_single, y_train, sample_weight_train, 
                                           scoring, cv)
            sfi_scores[feature] = scores.mean()
        except Exception as e:
            print(f"   ⚠️ 特徵 {feature} 計算失敗: {str(e)}")
            sfi_scores[feature] = -np.inf
        
        if (i - 1) % 50 == 0 or i == len(X_train.columns):
            print(f"   進度: {i}/{len(X_train.columns)} ({i/len(X_train.columns)*100:.1f}%)")
    
    # 創建 Series
    sfi_importance = pd.Series(sfi_scores)
    sfi_importance = sfi_importance.sort_values(ascending=False)
    
    print(f"   ✅ SFI 計算完成")
    print(f"   前10個重要特徵:")
    for i, (feat, imp) in enumerate(sfi_importance.head(10).items(), 1):
        print(f"     {i:2d}. {feat}: {imp:.6f}")
    
    return sfi_importance

# =====================================================
# 4️⃣ PCA 正交化特徵
# =====================================================

def calculate_pca_importance(X_train, X_test, explained_variance_threshold=0.95):
    """
    計算 PCA 正交化特徵的重要性
    """
    print(f"\n4️⃣ 計算 PCA 正交化特徵...")
    
    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # PCA 分解
    pca = PCA()
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    # 計算累積解釋變異
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    # 找出達到閾值的主成分數量
    n_components = np.argmax(cumulative_variance >= explained_variance_threshold) + 1
    
    print(f"   總特徵數: {len(X_train.columns)}")
    print(f"   達到 {explained_variance_threshold*100}% 解釋變異需要 {n_components} 個主成分")
    print(f"   前10個主成分的解釋變異比例:")
    for i in range(min(10, len(explained_variance_ratio))):
        print(f"     PC{i+1}: {explained_variance_ratio[i]:.4f} ({cumulative_variance[i]*100:.2f}%)")
    
    # 創建主成分 DataFrame
    pca_components = pd.DataFrame(
        pca.components_[:n_components].T,
        index=X_train.columns,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # 創建重要性 Series（使用解釋變異比例）
    pca_importance = pd.Series(
        explained_variance_ratio[:n_components],
        index=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # 轉換為 DataFrame
    X_train_pca = pd.DataFrame(
        X_train_pca[:, :n_components],
        index=X_train.index,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    X_test_pca = pd.DataFrame(
        X_test_pca[:, :n_components],
        index=X_test.index,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    print(f"   ✅ PCA 計算完成")
    
    return pca_components, pca_importance, X_train_pca, X_test_pca

# =====================================================
# 5️⃣ Weighted Kendall's Tau
# =====================================================

def calculate_weighted_kendall_tau(importance_rank, pca_rank):
    """
    計算 Weighted Kendall's Tau
    """
    # 確保索引一致
    common_idx = importance_rank.index.intersection(pca_rank.index)
    
    if len(common_idx) == 0:
        return np.nan, np.nan
    
    importance_values = importance_rank.loc[common_idx].values
    pca_values = pca_rank.loc[common_idx].values
    
    # 計算 Weighted Kendall's Tau
    tau, p_value = weightedtau(importance_values, pca_values)
    
    return tau, p_value

# =====================================================
# 主程式：執行所有特徵重要性分析
# =====================================================

print(f"\n" + "=" * 60)
print("開始特徵重要性分析（使用已訓練的模型）...")
print("=" * 60)

# 獲取已訓練的模型
trained_model = pipeline.model
model_type = pipeline.model_type

# 如果模型在 scaler 之後，需要先標準化
if pipeline.scaler is not None:
    X_train_scaled = pd.DataFrame(
        pipeline.scaler.transform(X_train),
        index=X_train.index,
        columns=X_train.columns
    )
    X_test_scaled = pd.DataFrame(
        pipeline.scaler.transform(X_test),
        index=X_test.index,
        columns=X_test.columns
    )
else:
    X_train_scaled = X_train
    X_test_scaled = X_test

# 1. MDI（從已訓練的模型獲取）
print(f"\n{'='*60}")
print("1️⃣ MDI 分析")
print(f"{'='*60}")

mdi_importance = get_mdi_from_model(trained_model, X_train)

# 2. MDA（使用已訓練的模型）
print(f"\n{'='*60}")
print("2️⃣ MDA 分析")
print(f"{'='*60}")

mda_importance = calculate_mda_from_model(
    model=trained_model,
    X_test=X_test_scaled,
    y_test=y_test,
    sample_weight_test=sample_weight_test,
    scoring='neg_log_loss',
    n_permutations=1,
    random_state=42
)

# 3. SFI（使用相同的模型類型）
print(f"\n{'='*60}")
print("3️⃣ SFI 分析")
print(f"{'='*60}")

sfi_importance = calculate_sfi_from_model_type(
    model_type=model_type,
    X_train=X_train,
    y_train=y_train,
    t1_train=t1_train,
    sample_weight_train=sample_weight_train,
    cv_splits=5,
    pct_embargo=0.01,
    scoring='neg_log_loss',
    random_state=42
)

# 4. PCA 正交化
print(f"\n{'='*60}")
print("4️⃣ PCA 正交化分析")
print(f"{'='*60}")

pca_components, pca_importance, X_train_pca, X_test_pca = calculate_pca_importance(
    X_train, X_test,
    explained_variance_threshold=0.95
)

# 5. Weighted Kendall's Tau（一致性檢查）
print(f"\n{'='*60}")
print("5️⃣ Weighted Kendall's Tau 一致性檢查")
print(f"{'='*60}")

if pca_importance is not None:
    consistency_results = {}
    
    # MDI vs PCA
    if mdi_importance is not None:
        try:
            mdi_pca_mapped = {}
            for feat in mdi_importance.index:
                if feat in pca_components.index:
                    loadings = pca_components.loc[feat].abs() * pca_importance.values
                    mdi_pca_mapped[feat] = loadings.sum()
            
            if len(mdi_pca_mapped) > 0:
                mdi_pca_series = pd.Series(mdi_pca_mapped)
                tau, p_value = calculate_weighted_kendall_tau(mdi_importance, mdi_pca_series)
                consistency_results['MDI vs PCA'] = {'tau': tau, 'p_value': p_value}
                print(f"   MDI vs PCA: τ = {tau:.4f}, p-value = {p_value:.4f}")
        except Exception as e:
            print(f"   ⚠️ MDI vs PCA 計算失敗: {str(e)}")
    
    # MDA vs PCA
    if mda_importance is not None:
        try:
            mda_pca_mapped = {}
            for feat in mda_importance.index:
                if feat in pca_components.index:
                    loadings = pca_components.loc[feat].abs() * pca_importance.values
                    mda_pca_mapped[feat] = loadings.sum()
            
            if len(mda_pca_mapped) > 0:
                mda_pca_series = pd.Series(mda_pca_mapped)
                tau, p_value = calculate_weighted_kendall_tau(mda_importance, mda_pca_series)
                consistency_results['MDA vs PCA'] = {'tau': tau, 'p_value': p_value}
                print(f"   MDA vs PCA: τ = {tau:.4f}, p-value = {p_value:.4f}")
        except Exception as e:
            print(f"   ⚠️ MDA vs PCA 計算失敗: {str(e)}")
    
    # SFI vs PCA
    if sfi_importance is not None:
        try:
            sfi_pca_mapped = {}
            for feat in sfi_importance.index:
                if feat in pca_components.index:
                    loadings = pca_components.loc[feat].abs() * pca_importance.values
                    sfi_pca_mapped[feat] = loadings.sum()
            
            if len(sfi_pca_mapped) > 0:
                sfi_pca_series = pd.Series(sfi_pca_mapped)
                tau, p_value = calculate_weighted_kendall_tau(sfi_importance, sfi_pca_series)
                consistency_results['SFI vs PCA'] = {'tau': tau, 'p_value': p_value}
                print(f"   SFI vs PCA: τ = {tau:.4f}, p-value = {p_value:.4f}")
        except Exception as e:
            print(f"   ⚠️ SFI vs PCA 計算失敗: {str(e)}")

# =====================================================
# 可視化結果
# =====================================================

print(f"\n{'='*60}")
print("📊 可視化特徵重要性結果")
print(f"{'='*60}")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. MDI Top 20
if mdi_importance is not None:
    top_mdi = mdi_importance.head(20)
    axes[0, 0].barh(range(len(top_mdi)), top_mdi.values)
    axes[0, 0].set_yticks(range(len(top_mdi)))
    axes[0, 0].set_yticklabels(top_mdi.index, fontsize=8)
    axes[0, 0].set_xlabel('MDI Importance', fontsize=12)
    axes[0, 0].set_title('MDI: Top 20 Features', fontsize=14)
    axes[0, 0].invert_yaxis()
    axes[0, 0].grid(True, alpha=0.3)
else:
    axes[0, 0].text(0.5, 0.5, 'MDI not available\n(非樹模型)', 
                   ha='center', va='center', fontsize=12)
    axes[0, 0].set_title('MDI: Top 20 Features', fontsize=14)

# 2. MDA Top 20
if mda_importance is not None:
    top_mda = mda_importance.head(20)
    axes[0, 1].barh(range(len(top_mda)), top_mda.values)
    axes[0, 1].set_yticks(range(len(top_mda)))
    axes[0, 1].set_yticklabels(top_mda.index, fontsize=8)
    axes[0, 1].set_xlabel('MDA Importance', fontsize=12)
    axes[0, 1].set_title('MDA: Top 20 Features', fontsize=14)
    axes[0, 1].invert_yaxis()
    axes[0, 1].grid(True, alpha=0.3)

# 3. SFI Top 20
if sfi_importance is not None:
    top_sfi = sfi_importance.head(20)
    axes[1, 0].barh(range(len(top_sfi)), top_sfi.values)
    axes[1, 0].set_yticks(range(len(top_sfi)))
    axes[1, 0].set_yticklabels(top_sfi.index, fontsize=8)
    axes[1, 0].set_xlabel('SFI Importance', fontsize=12)
    axes[1, 0].set_title('SFI: Top 20 Features', fontsize=14)
    axes[1, 0].invert_yaxis()
    axes[1, 0].grid(True, alpha=0.3)

# 4. PCA 解釋變異
if pca_importance is not None:
    top_pca = pca_importance.head(20)
    axes[1, 1].bar(range(len(top_pca)), top_pca.values)
    axes[1, 1].set_xticks(range(len(top_pca)))
    axes[1, 1].set_xticklabels(top_pca.index, rotation=45, ha='right', fontsize=8)
    axes[1, 1].set_ylabel('Explained Variance Ratio', fontsize=12)
    axes[1, 1].set_title('PCA: Top 20 Components', fontsize=14)
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ 特徵重要性分析完成")
print("=" * 60)

## 自動化

In [ ]:
train_start = '2014-01-01'
test_horizon_start_year = 2020
rolling_years = 6          # 2020~2024
model_types = ['lr', 'xgboost', 'catboost']  # 需要跑的模型，可自行調整
results_dir = Path('pred_results')
results_dir.mkdir(exist_ok=True)

In [ ]:
def build_pred_df(test_idx, t1_series, events_df, y_true_series,
                  pred_series, prob_array, class_mapping):
    pred_df = pd.DataFrame(index=test_idx)
    pred_df['t1'] = t1_series.reindex(test_idx)
    pred_df['side'] = events_df.loc[test_idx, 'side']
    pred_df['true_label'] = y_true_series.reindex(test_idx).map(class_mapping)
    pred_df['predicted_label'] = pred_series.loc[test_idx].values

    for i, class_id in enumerate(sorted(class_mapping.values())):
        prob_col = f'prob_class_{class_id}'
        pred_df[prob_col] = prob_array[:, i]

    pred_df['prediction_confidence'] = np.max(prob_array, axis=1)
    pred_df['is_correct'] = (
        pred_df['predicted_label'] == pred_df['true_label']
    )
    return pred_df

In [ ]:
def run_walk_forward(model_type, first_test_year=2020, n_years=5):
    yearly_pred_dfs = []
    tz_info = X.index.tz  # 若 index 有時區，確保一致

    for year in range(first_test_year, first_test_year + n_years):
        test_start_dt = pd.Timestamp(f'{year}-01-01', tz=tz_info)
        test_end_dt = test_start_dt + pd.DateOffset(years=1)

        train_mask = (X.index >= train_start) & (X.index < test_start_dt)
        test_mask = (X.index >= test_start_dt) & (X.index < test_end_dt)

        if train_mask.sum() == 0 or test_mask.sum() == 0:
            print(f'⚠️ {model_type}: {year} 無可用資料，跳過')
            continue

        X_train = X.loc[train_mask]
        y_train = y_mapped.loc[train_mask]
        t1_train = t1.loc[train_mask]
        sw_train = sample_weight_series.loc[train_mask] if sample_weight_series is not None else None

        X_test = X.loc[test_mask]
        y_test = y_mapped.loc[test_mask]
        t1_test = t1.loc[test_mask]
        sw_test = sample_weight_series.loc[test_mask] if sample_weight_series is not None else None

        print('=' * 70)
        print(f'🎯 Model: {model_type.upper()} | Test year: {year}')
        print(f'Train window: {X_train.index.min()} → {X_train.index.max()} ({len(X_train)} rows)')
        print(f'Test window : {X_test.index.min()} → {X_test.index.max()} ({len(X_test)} rows)')

        pipeline = PredictionPipeline(
            model_type=model_type,
            cv_scoring='f1_weighted',
            use_hyperopt=True,
            use_cv=True,
            cv_splits=5,
            pct_embargo=0.01,
            hyperopt_method='grid',
            n_iter=20,
            n_jobs=-1,
        )
        pipeline.fit(X_train, y_train, t1_train, sw_train)

        preds_mapped, probs = pipeline.predict(X_test)
        preds_original = (
            pd.Series(np.ravel(preds_mapped), index=X_test.index)
            .map(reverse_mapping)
        )

        pred_df = build_pred_df(
            test_idx=X_test.index,
            t1_series=t1,
            events_df=events,
            y_true_series=y,
            pred_series=preds_original,
            prob_array=probs,
            class_mapping=reverse_mapping,
        )
        pred_df['model_type'] = model_type
        pred_df['test_year'] = year
        pred_df['train_window_start'] = pd.Timestamp(train_start)
        pred_df['train_window_end'] = test_start_dt - pd.Timedelta(seconds=1)

        yearly_pred_dfs.append(pred_df)

        # 若需要即時檢視，可在此呼叫 pipeline.plot_results(X_test, y_test, sw_test)

    if not yearly_pred_dfs:
        return pd.DataFrame()

    out_df = pd.concat(yearly_pred_dfs).sort_index()
    out_path = results_dir / f'pred_{model_type}_{first_test_year}_{first_test_year + len(yearly_pred_dfs) - 1}.csv'
    out_df.to_csv(out_path)
    print(f'✅ {model_type} 走勢預測完成，輸出 {out_path}')
    return out_df

In [ ]:
%%time
rolling_predictions = {}

for m in model_types:
    rolling_predictions[m] = run_walk_forward(
        model_type=m,
        first_test_year=test_horizon_start_year,
        n_years=rolling_years,
    )

display({m: df.tail() for m, df in rolling_predictions.items() if not df.empty})

## 全自動執行商品X模型

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import rv_continuous, kstest
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List# =====================================================
# 3️⃣ Meta-Labeling Class
# =====================================================
class MetaLabeling:
    """
    Meta-Labeling：Triple Barrier 和標籤生成
    
    Triple Barrier Method 說明：
    ============================
    Triple Barrier Method 是一種標籤方法，根據三個屏障中第一個被觸及的
    屏障來標記觀測值。
    
    用途：
    -----
    1. 動態設定止盈止損：根據估計的波動率（已實現或隱含）動態調整
    2. 避免目標過高或過低：考慮當前波動率，設定合理的止盈止損水平
    3. 路徑依賴標籤：考慮從 [t_i,0, t_i,0 + h] 的完整價格路徑
    
    三個屏障：
    --------
    1. 上方水平屏障（Profit Taking）：止盈限制
    2. 下方水平屏障（Stop Loss）：止損限制
    3. 垂直屏障（Vertical Barrier）：時間到期限制（持有期）
    
    屏障配置：
    --------
    用三元組 [pt, sl, t1] 表示，其中：
    - 0 表示屏障未啟用
    - 1 表示屏障啟用
    
    常用配置：
    - [1,1,1]: 標準設定（止盈、止損、時間到期）
    - [0,1,1]: 只有止損和時間到期
    - [1,1,0]: 只有止盈和止損（無時間限制）
    
    數學原理：
    ---------
    對於每個事件時間點 t_i,0：
    
    1. 上方屏障（止盈）：
       PT = t_i,0 + pt × σ_t_i,0
       其中 pt 是止盈倍數，σ_t_i,0 是目標波動率
    
    2. 下方屏障（止損）：
       SL = t_i,0 - sl × σ_t_i,0
       其中 sl 是止損倍數
    
    3. 垂直屏障（時間到期）：
       t_i,1 = t_i,0 + h
       其中 h 是持有期（期數）
    
    4. 第一個觸及的屏障：
       t_i,1 = min{PT_touch, SL_touch, t_i,0 + h}
    
    5. 標籤生成：
       - bin = 1:  觸及上方屏障（成功）
       - bin = -1: 觸及下方屏障（失敗）
       - bin = 0:  觸及垂直屏障（時間到期）
    
    路徑依賴性：
    ----------
    為了標記觀測值，必須考慮從 t_i,0 到 t_i,0 + h 的完整價格路徑。
    這使得標籤是路徑依賴的，而非僅依賴終點價格。
    
    動態閾值：
    --------
    使用 getDailyVol() 計算日波動率，作為動態調整止盈止損的基礎：
    
    σ_t = EWM_std(returns, span=span0)
    
    其中 returns 是日收益率序列。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 3
    - Snippet 3.1: getDailyVol - 計算日波動率
    - Snippet 3.2: applyPtSlOnT1 - Triple Barrier 標籤方法
    - Snippet 3.3: getEvents - 取得第一個屏障觸及時間
    - Snippet 3.4: getVerticalBarriers - 設定垂直屏障
    - Snippet 3.5: getBins - 生成標籤
    """
    
    def __init__(self, ptSl: List[float] = [1, 2], numPeriods: int = 100, 
                 minRet: float = 0.01, min_label_pct: float = 0.05):
        """
        Parameters:
        -----------
        ptSl : list
            [止盈倍數, 止損倍數]
            - ptSl[0]: 上方屏障的倍數（Profit Taking）
            - ptSl[1]: 下方屏障的倍數（Stop Loss）
            - 如果為 0，則該屏障不啟用
        numPeriods : int
            垂直屏障期數（例如：100 根 K 線）
            對應論文中的 h（持有期）
        minRet : float
            最小波動率閾值
            只有當 trgt > minRet 時才會生成事件
        min_label_pct : float
            最小標籤比例（用於 dropLabels）
            低於此比例的標籤會被移除
        """
        self.ptSl = ptSl
        self.numPeriods = numPeriods
        self.minRet = minRet
        self.min_label_pct = min_label_pct
        self.events = None
        self.bins = None
        
    def get_vertical_barriers(self, close: pd.Series, tEvents: pd.DatetimeIndex) -> pd.Series:
        """
        設定垂直時間屏障（以期數為單位）
        
        實現原理（基於 Snippet 3.4）：
        ----------------------------
        對每個事件時間點 t_i,0，找到 numPeriods 期後的時間點。
        
        公式：
        t_i,1 = close.index[position(t_i,0) + numPeriods]
        
        如果超出數據範圍，則使用最後一個時間點。
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點
            
        Returns:
        --------
        t1 : pd.Series
            垂直屏障時間點（index=事件起點, value=屏障時間點）
        """
        t1 = []
        close_index = close.index
        
        for tEvent in tEvents:
            # 找到 tEvent 在 close 中的位置
            try:
                event_pos = close_index.get_loc(tEvent)
                # 計算 numPeriods 期後的位置
                barrier_pos = event_pos + self.numPeriods
                
                # 確保不超過數據範圍
                if barrier_pos < len(close_index):
                    t1.append(close_index[barrier_pos])
                else:
                    # 如果超出範圍，使用最後一個時間點
                    t1.append(close_index[-1])
            except KeyError:
                # 如果找不到該時間點，跳過
                continue
        
        if len(t1) == 0:
            return pd.Series(dtype='datetime64[ns]')
        
        # 確保 t1 的索引與 tEvents 對應
        t1_series = pd.Series(t1, index=tEvents[:len(t1)])
        return t1_series
    
    def apply_pt_sl_on_t1(self, close: pd.Series, events: pd.DataFrame, 
                          molecule: pd.DatetimeIndex) -> pd.DataFrame:
        """
        套用止盈止損（基於 Snippet 3.2）
        
        實現原理：
        ---------
        對每個事件，計算從事件開始到垂直屏障期間的價格路徑：
        
        path_prices = close[loc:t1]
        path_returns = (path_prices / close[loc] - 1) × side
        
        然後找出：
        - 最早觸及止盈的時間：path_returns > pt × trgt
        - 最早觸及止損的時間：path_returns < -sl × trgt
        
        公式：
        - PT_level = pt × trgt
        - SL_level = -sl × trgt
        - path_returns = (close[t] / close[loc] - 1) × side
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        events : pd.DataFrame
            事件 DataFrame（包含 t1, trgt, side）
        molecule : pd.DatetimeIndex
            要處理的事件子集（用於並行計算）
            
        Returns:
        --------
        out : pd.DataFrame
            包含每個屏障觸及時間的 DataFrame
        """
        events_ = events.loc[molecule]
        out = events_[['t1']].copy(deep=True)
        
        # 計算止盈水平
        if self.ptSl[0] > 0:
            pt = self.ptSl[0] * events_['trgt']  # PT = pt × σ
        else:
            pt = pd.Series(index=events.index)  # 未啟用
        
        # 計算止損水平
        if self.ptSl[1] > 0:
            sl = -self.ptSl[1] * events_['trgt']  # SL = -sl × σ
        else:
            sl = pd.Series(index=events.index)  # 未啟用
        
        # 對每個事件計算路徑
        for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
            # 取得價格路徑
            df0 = close[loc:t1]
            # 計算路徑報酬（考慮方向）
            df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']
            
            # 找出最早觸及止損的時間
            out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
            # 找出最早觸及止盈的時間
            out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
        
        return out
    
    def get_events(self, close: pd.Series, tEvents: pd.DatetimeIndex, 
                   trgt: pd.Series, side: pd.Series) -> pd.DataFrame:
        """
        生成事件並套用 Triple Barrier（基於 Snippet 3.3）
        
        實現流程：
        ---------
        1. 過濾目標：只保留 trgt > minRet 的事件
        2. 設定垂直屏障：計算每個事件的時間到期點
        3. 套用 Triple Barrier：計算止盈止損觸及時間
        4. 找出第一個觸及的屏障：t1 = min{pt_touch, sl_touch, vertical_barrier}
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點（來自 CUSUM 過濾或策略信號）
        trgt : pd.Series
            目標波動率（daily_vol，用於動態調整止盈止損）
        side : pd.Series
            方向（1 為做多，-1 為做空）
            
        Returns:
        --------
        events : pd.DataFrame
            包含 t1, trgt, side, pt, sl 的事件 DataFrame
            - t1: 第一個屏障觸及時間
            - trgt: 目標波動率
            - side: 方向
            - pt: 止盈倍數
            - sl: 止損倍數
        """
        # 1. 過濾目標（只保留波動率足夠大的事件）
        trgt = trgt.loc[tEvents]
        trgt = trgt[trgt > self.minRet]
        
        # 2. 設定垂直屏障
        t1 = self.get_vertical_barriers(close, tEvents)
        
        # 3. 取得方向
        side_ = side.loc[trgt.index]
        
        # 4. 組合事件物件
        events = pd.concat({
            't1': t1.loc[trgt.index], 
            'trgt': trgt, 
            'side': side_
        }, axis=1)
        events = events.dropna(subset=['trgt'])
        
        # 5. 計算止盈止損觸及時間
        df0 = self.apply_pt_sl_on_t1(close, events, events.index)
        
        # 6. 找出第一個觸及的屏障（最早的時間）
        events['t1'] = df0.dropna(how='all').min(axis=1)
        
        # 7. 儲存屏障配置
        events['pt'] = self.ptSl[0]
        events['sl'] = self.ptSl[1]
        
        self.events = events
        return events
    
    def get_bins(self, close: pd.Series) -> pd.DataFrame:
        """
        生成標籤（基於 Snippet 3.5，並擴展為支援 Meta-Labeling）
        
        實現原理：
        ---------
        1. 對齊價格：取得事件開始和結束時間的所有價格點
        2. 計算實際報酬：
           ret = (close[t1] / close[t0] - 1) × side
        3. 判斷觸及的屏障：
           - 如果 ret > 0 且 ret > pt × trgt → bin = 1（觸及止盈）
           - 如果 ret < 0 且 ret < -sl × trgt → bin = -1（觸及止損）
           - 否則 → bin = 0（觸及垂直屏障）
        
        標籤定義：
        --------
        - bin = 1:  成功（觸及止盈屏障）
        - bin = 0:  時間到期（觸及垂直屏障）
        - bin = -1: 虧損（觸及止損屏障）
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
            
        Returns:
        --------
        bins : pd.DataFrame
            包含 ret, trgt, bin, side 的標籤 DataFrame
        """
        if self.events is None:
            print("⚠️ 請先執行 get_events()")
            return None
        
        events_ = self.events.dropna(subset=['t1'])
        px = events_.index.union(events_['t1'].values).drop_duplicates()
        px = close.reindex(px, method='bfill')
        
        out = pd.DataFrame(index=events_.index)
        out['ret'] = px.loc[events_['t1'].values].values / px.loc[events_.index] - 1
        out['ret'] *= events_['side']
        out['trgt'] = events_['trgt']
        
        # 判斷觸及哪個屏障
        out['bin'] = 0  # 預設為時間到期
        for date_time, values in out.iterrows():
            ret = values['ret']
            target = values['trgt']
            pt_level = ret > target * self.events.loc[date_time, 'pt']
            sl_level = ret < -target * self.events.loc[date_time, 'sl']
            
            if ret > 0.0 and pt_level:
                # 觸及止盈屏障 → bin = 1 (成功)
                out.loc[date_time, 'bin'] = 1
            elif ret < 0.0 and sl_level:
                # 觸及止損屏障 → bin = -1 (虧損)
                out.loc[date_time, 'bin'] = -1
            else:
                # 時間到期（垂直屏障）→ bin = 0
                out.loc[date_time, 'bin'] = 0
        
        # 如果有 side（Meta-Labeling），保留 side 資訊
        if 'side' in events_:
            out['side'] = events_['side']
        
        self.bins = out
        return out
    
    def drop_rare_labels(self) -> pd.DataFrame:
        """
        移除稀有標籤（基於 Snippet 3.8）
        
        目的：
        -----
        移除樣本數過少的標籤類別，避免模型學習偏斜。
        
        流程：
        -----
        1. 計算每個標籤的比例
        2. 如果最小比例 < min_label_pct，移除該標籤
        3. 重複直到所有標籤比例都 >= min_label_pct
        """
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return None
        
        bins = self.bins.copy()
        
        while True:
            df0 = bins['bin'].value_counts(normalize=True)
            if df0.min() > self.min_label_pct or df0.shape[0] < 3:
                break
            print(f"Dropped label {df0.idxmin()}, percentage: {df0.min():.4f}")
            bins = bins[bins['bin'] != df0.idxmin()]
        
        self.bins = bins
        self.events = self.events.loc[bins.index]
        
        return bins
    
    def get_label_stats(self) -> dict:
        """取得標籤統計資訊"""
        if self.bins is None:
            return None
        
        label_counts = self.bins['bin'].value_counts()
        
        return {
            'total_labels': len(self.bins),
            'label_distribution': label_counts.to_dict(),
            'avg_return': self.bins['ret'].mean(),
            'win_rate': (self.bins['bin'] == 1).sum() / len(self.bins),
            'loss_rate': (self.bins['bin'] == -1).sum() / len(self.bins),
            'timeout_rate': (self.bins['bin'] == 0).sum() / len(self.bins)
        }
    
    def plot_labels(self):
        """繪製標籤分布圖"""
        import matplotlib.pyplot as plt
        
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # 標籤分布
        label_counts = self.bins['bin'].value_counts().sort_index()
        colors = {-1: 'red', 0: 'gray', 1: 'green'}
        label_colors = [colors.get(idx, 'steelblue') for idx in label_counts.index]
        
        label_counts.plot(kind='bar', ax=axes[0], color=label_colors)
        axes[0].set_title('Label Distribution', fontsize=12)
        axes[0].set_xlabel('Label (-1: Loss, 0: Timeout, 1: Win)')
        axes[0].set_ylabel('Count')
        axes[0].grid(True, alpha=0.3, axis='y')
        
        # 回報分布
        self.bins['ret'].hist(bins=50, ax=axes[1], alpha=0.7, color='steelblue')
        axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Return')
        axes[1].set_title('Return Distribution', fontsize=12)
        axes[1].set_xlabel('Return')
        axes[1].set_ylabel('Frequency')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [2]:
# =====================================================
# 4️⃣ 樣本權重 Class（並發度、唯一性、時間衰減、類別平衡）
# =====================================================
class SampleWeight:
    """
    樣本權重計算：並發度、唯一性、時間衰減、類別平衡
    
    Chapter 4: Sample Weights 說明
    ===============================
    
    問題背景：
    --------
    在金融應用中，觀測值不是由獨立同分布（IID）過程生成的。
    當兩個標籤 y_i 和 y_j 的時間區間有重疊時（t_i,1 > t_j,0），
    它們會依賴共同的報酬 r_{t_j,0, min{t_i,1, t_j,1}}。
    
    這導致標籤序列 {y_i}_{i=1,...,I} 不是 IID，違反了大多數 ML 算法的假設。
    
    解決方案：
    --------
    通過設計採樣和加權方案來糾正重疊結果的不當影響：
    1. 計算並發度（Concurrency）：每個時間點有多少標籤同時存活
    2. 計算唯一性（Uniqueness）：每個標籤的平均唯一性
    3. 應用時間衰減（Time Decay）：讓舊樣本權重降低
    4. 類別平衡（Class Balance）：平衡不同類別的樣本權重
    5. 組合權重：最終權重 = 唯一性 × 時間衰減 × 類別平衡（可選：× 報酬歸因）
    
    並發度（Concurrency）：
    ---------------------
    定義：兩個標籤 y_i 和 y_j 在時間 t 是並發的，當它們都依賴至少一個
    共同的報酬 r_{t-1,t} = p_t / p_{t-1} - 1。
    
    計算方式：
    對每個時間點 t = 1,...,T，形成二元陣列 {1_{t,i}}_{i=1,...,I}：
    - 1_{t,i} = 1：如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊
    - 1_{t,i} = 0：否則
    
    並發度：c_t = Σ_{i=1}^I 1_{t,i}
    
    唯一性（Uniqueness）：
    --------------------
    定義：標籤 i 在時間 t 的唯一性為 u_{t,i} = 1_{t,i} / c_t
    
    平均唯一性：ū_i = (Σ_{t=1}^T 1_{t,i})^{-1} × (Σ_{t=1}^T u_{t,i})
    
    也可以解釋為：標籤 i 存活期間的並發度倒數的調和平均。
    
    報酬歸因（Return Attribution）：
    -----------------------------
    當標籤是報酬符號的函數時（{-1,1} 或 {0,1}），樣本權重可以定義為：
    
    ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
    
    其中 r_{t-1,t} 是對數報酬，c_t 是時間 t 的並發度。
    
    然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
    
    時間衰減（Time Decay）：
    ----------------------
    市場是適應性系統，舊樣本不如新樣本相關。
    
    線性時間衰減：
    d[x] = max{0, a + bx}
    
    邊界條件：
    1. d[Σ_{i=1}^I ū_i] = 1（最新樣本權重為 1）
    2. d[0] = c（最舊樣本權重為 c，c ∈ [0,1]）
    
    參數 c 的意義：
    - c = 1：無時間衰減
    - 0 < c < 1：線性衰減，所有樣本權重 > 0
    - c = 0：最舊樣本權重為 0
    - c < 0：最舊的 |c|×T 部分樣本權重為 0
    
    注意：時間衰減是基於累積唯一性，而非時間順序，因為在存在冗餘
    觀測值的情況下，按時間順序衰減會使權重降低太快。
    
    類別平衡（Class Balance）：
    ------------------------
    當標籤分布不平衡時（例如：上漲標籤多於下跌標籤），需要對少數類別
    給予更高的權重，以平衡模型訓練。
    
    計算方式：
    w_class = n_samples / (n_classes × n_class_samples)
    
    這樣可以讓少數類別的樣本權重更高，平衡類別分布。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 4
    - Snippet 4.1: mpNumCoEvents - 計算並發度
    - Snippet 4.2: mpSampleTW - 計算平均唯一性
    - Snippet 4.10: mpSampleW - 報酬歸因權重
    - Snippet 4.11: getTimeDecay - 時間衰減因子
    """
    
    def __init__(self):
        self.num_co_events = None
        self.uniqueness = None
        self.time_decay = None
        self.class_balance_weights = None
        self.sample_weights = None
        
    def compute_num_co_events(self, close_idx: pd.DatetimeIndex, 
                              t1: pd.Series) -> pd.Series:
        """
        計算每個時間點的並發事件數（Concurrency）
        基於 Snippet 4.1: mpNumCoEvents
        
        實現原理：
        ---------
        對每個時間點 t，計算有多少個事件的存活期間 [t_i,0, t_i,1] 
        與時間點 t 重疊。
        
        公式：
        c_t = Σ_{i=1}^I 1_{t,i}
        
        其中 1_{t,i} = 1 如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊，否則為 0。
        
        參數:
        -----------
        close_idx : pd.DatetimeIndex
            價格數據的索引
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
            
        Returns:
        --------
        num_co_events : pd.Series
            每個時間點的並發事件數
        """
        t1 = t1.fillna(close_idx[-1])  # 未結束事件仍要計算
        
        # 初始化計數序列
        iloc = close_idx.searchsorted(np.array([t1.index[0], t1.max()]))
        count = pd.Series(0, index=close_idx[iloc[0]:iloc[1]+1])
        
        # 對每個事件，在其存活期間 +1
        for tIn, tOut in t1.items():
            count.loc[tIn:tOut] += 1.0
        
        self.num_co_events = count
        
        print("=" * 60)
        print("📊 並發度統計")
        print("=" * 60)
        print(f"總時間點數: {len(count):,}")
        print(f"平均並發事件數: {count.mean():.2f}")
        print(f"最大並發事件數: {count.max():.0f}")
        print(f"\n說明:")
        print(f"  - 並發度衡量每個時間點有多少個標籤同時存活")
        print(f"  - 高並發度表示標籤之間重疊多，資訊冗餘")
        print(f"  - 低並發度表示標籤相對獨立")
        print(f"並發度分布:")
        print(count.describe())
        print("=" * 60)
        
        return count

    def compute_uniqueness(self, t1: pd.Series, 
                          num_co_events: pd.Series) -> pd.Series:
        """
        計算每個事件的唯一性（Uniqueness）
        基於 Snippet 4.2: mpSampleTW
        
        實現原理：
        ---------
        對每個事件 i，計算其在存活期間的平均唯一性：
        
        u_{t,i} = 1_{t,i} / c_t  （時間 t 的唯一性）
        ū_i = (1 / T_i) × Σ_{t=t_i,0}^{t_i,1} u_{t,i}
        
        其中 T_i 是事件 i 的存活期數。
        
        也可以解釋為：事件存活期間的並發度倒數的調和平均。
        
        公式：
        ū_i = mean(1 / c_t) for t ∈ [t_i,0, t_i,1]
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        num_co_events : pd.Series
            每個時間點的並發事件數序列
            
        Returns:
        --------
        uniqueness : pd.Series
            每個事件的唯一性（1 / 平均並發度）
        """
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算事件存活期間的平均並發度倒數
            avg_co_events = num_co_events.loc[tIn:tOut].mean()
            wght.loc[tIn] = 1.0 / avg_co_events if avg_co_events > 0 else 0
        
        self.uniqueness = wght
        
        print("=" * 60)
        print("📊 唯一性統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均唯一性: {wght.mean():.4f}")
        print(f"\n說明:")
        print(f"  - 唯一性衡量每個標籤的非重疊程度")
        print(f"  - 唯一性 = 1 / 平均並發度")
        print(f"  - 唯一性越高，表示該標籤越獨立，權重應該越大")
        print(f"  - 唯一性越低，表示該標籤與其他標籤重疊多，權重應該越小")
        print(f"唯一性分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def compute_return_attribution(self, 
                                  close: pd.Series,
                                  t1: pd.Series,
                                  num_co_events: pd.Series) -> pd.Series:
        """
        計算報酬歸因權重（Return Attribution）
        基於 Snippet 4.10: mpSampleW
        
        實現原理：
        ---------
        對每個事件，計算其存活期間的歸因報酬：
        
        ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
        
        其中：
        - r_{t-1,t} = log(close_t / close_{t-1}) 是對數報酬
        - c_t 是時間 t 的並發度
        
        然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
        
        這樣設計的意義：
        - 將報酬按並發度分配，避免重疊事件重複計算報酬
        - 絕對報酬大的事件權重更高
        - 標準化後權重總和為 I（樣本數）
        
        參數:
        -----------
        close : pd.Series
            價格序列
        t1 : pd.Series
            事件結束時間
        num_co_events : pd.Series
            並發度序列
            
        Returns:
        --------
        return_weights : pd.Series
            報酬歸因權重
        """
        ret = np.log(close).diff()  # 對數報酬
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算歸因報酬：報酬 / 並發度
            attributed_ret = (ret.loc[tIn:tOut] / 
                            num_co_events.loc[tIn:tOut]).sum()
            wght.loc[tIn] = abs(attributed_ret)
        
        # 標準化：權重總和 = 樣本數
        wght = wght * len(wght) / wght.sum()
        
        print("=" * 60)
        print("📊 報酬歸因權重統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均權重: {wght.mean():.4f}")
        print(f"權重總和: {wght.sum():.0f} (應等於樣本數)")
        print(f"\n說明:")
        print(f"  - 根據歸因的絕對報酬來加權")
        print(f"  - 報酬按並發度分配，避免重疊事件重複計算")
        print(f"  - 絕對報酬大的事件權重更高")
        print(f"權重分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def compute_class_balance_weight(self, labels: pd.Series) -> pd.Series:
        """
        計算類別平衡權重
        
        實現原理：
        ---------
        對每個類別，計算其權重為：
        w_class = n_samples / (n_classes × n_class_samples)
        
        這樣可以讓少數類別的樣本權重更高，平衡類別分布。
        
        使用 sklearn 的 compute_sample_weight('balanced', labels) 實現。
        
        參數:
        -----------
        labels : pd.Series
            標籤序列（index 必須與樣本權重的 index 一致）
            
        Returns:
        --------
        class_weights : pd.Series
            每個樣本的類別平衡權重
        """
        from sklearn.utils.class_weight import compute_sample_weight
        
        # 計算類別平衡權重
        class_weights = compute_sample_weight('balanced', labels)
        class_weights_series = pd.Series(class_weights, index=labels.index)
        
        self.class_balance_weights = class_weights_series
        
        print("=" * 60)
        print("📊 類別平衡權重統計")
        print("=" * 60)
        print(f"標籤分布:")
        label_counts = labels.value_counts().sort_index()
        print(label_counts)
        print(f"\n類別平衡權重:")
        for label in sorted(labels.unique()):
            mask = labels == label
            avg_weight = class_weights_series[mask].mean()
            count = mask.sum()
            pct = count / len(labels) * 100
            print(f"  標籤 {label}: 樣本數 = {count:,} ({pct:.2f}%), 平均權重 = {avg_weight:.4f}")
        print(f"\n說明:")
        print(f"  - 類別平衡權重用於平衡不同類別的樣本分布")
        print(f"  - 少數類別的樣本會獲得更高的權重")
        print(f"  - 多數類別的樣本會獲得較低的權重")
        print(f"  - 權重計算公式: w = n_samples / (n_classes × n_class_samples)")
        print("=" * 60)
        
        return class_weights_series

    def get_time_decay_linear(self, tW: pd.Series, 
                              clf_last_w: float = 1.0) -> pd.Series:
        """
        線性時間衰減（基於 Snippet 4.11: getTimeDecay）
        
        實現原理：
        ---------
        應用分段線性衰減到觀測的唯一性（tW）：
        
        d[x] = max{0, a + bx}
        
        邊界條件：
        1. d[Σ_{i=1}^I ū_i] = 1  （最新樣本權重為 1）
        2. d[0] = clf_last_w    （最舊樣本權重為 clf_last_w）
        
        求解：
        a = 1 - b × Σ_{i=1}^I ū_i
        b = (1 - clf_last_w) / Σ_{i=1}^I ū_i  （當 clf_last_w >= 0）
        
        注意：時間衰減是基於累積唯一性 x ∈ [0, Σ_{i=1}^I ū_i]，
        而非時間順序，因為在存在冗餘觀測值的情況下，按時間順序
        衰減會使權重降低太快。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        clf_last_w : float
            最舊樣本的權重（預設 1.0，表示不衰減）
            - c = 1: 無時間衰減
            - 0 < c < 1: 線性衰減，所有樣本權重 > 0
            - c = 0: 最舊樣本權重為 0
            - c < 0: 最舊的 |c|×T 部分樣本權重為 0
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clfW = tW.sort_index().cumsum()
        
        if clfW.iloc[-1] > 0:
            if clf_last_w >= 0:
                # 線性衰減：d[0] = clf_last_w, d[Σū] = 1
                slope = (1.0 - clf_last_w) / clfW.iloc[-1]
            else:
                # 當 clf_last_w < 0 時的特殊處理
                slope = 1 / ((clf_last_w + 1) * clfW.iloc[-1])
        else:
            slope = 0
        
        const = 1.0 - slope * clfW.iloc[-1]
        clfW = const + slope * clfW
        clfW[clfW < 0] = 0
        
        self.time_decay = clfW
        
        print("=" * 60)
        print("📊 線性時間衰減")
        print("=" * 60)
        print(f"時間衰減參數: const={const:.4f}, slope={slope:.6f}")
        print(f"最舊樣本權重 (c): {clf_last_w:.4f}")
        print(f"最新樣本權重: 1.0")
        print(f"平均時間衰減權重: {clfW.mean():.4f}")
        print(f"\n說明:")
        if clf_last_w == 1.0:
            print(f"  - c = 1: 無時間衰減")
        elif 0 < clf_last_w < 1:
            print(f"  - 0 < c < 1: 線性衰減，所有樣本權重 > 0")
        elif clf_last_w == 0:
            print(f"  - c = 0: 最舊樣本權重為 0")
        elif clf_last_w < 0:
            print(f"  - c < 0: 最舊的 |c|×T 部分樣本權重為 0")
        print(f"  - 衰減基於累積唯一性，而非時間順序")
        print("=" * 60)
        
        return clfW
    
    def get_time_decay_exp(self, tW: pd.Series, 
                          decay_rate: float = 1.0,
                          percent_of_zero_wts: float = 0.0) -> pd.Series:
        """
        指數時間衰減
        
        實現原理：
        ---------
        使用指數函數進行時間衰減：
        
        d[x] = exp((decay_rate - 1) × (Σū - x))
        
        其中 x 是累積唯一性。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        decay_rate : float
            衰減率（>1 表示衰減更快，<1 表示衰減更慢）
        percent_of_zero_wts : float
            最舊樣本中設為 0 的比例
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clf_w = tW.sort_index().cumsum()
        last_value = clf_w.iloc[-1]
        
        # 計算指數衰減權重
        out_wts = []
        zero_threshold = int(round(len(clf_w) * percent_of_zero_wts))
        
        for i in range(len(clf_w)):
            if i < zero_threshold:
                out_wts.append(0.0)
            else:
                # 指數衰減：exp((decay_rate - 1) * (last_value - current_value))
                decay_factor = np.exp((decay_rate - 1.0) * (last_value - clf_w.iloc[i]))
                out_wts.append(decay_factor)
        
        time_decay = pd.Series(out_wts, index=clf_w.index)
        self.time_decay = time_decay
        
        print("=" * 60)
        print("📊 指數時間衰減統計")
        print("=" * 60)
        print(f"衰減率 (decay_rate): {decay_rate}")
        print(f"零權重樣本比例: {percent_of_zero_wts:.2%}")
        print(f"平均時間衰減權重: {time_decay.mean():.4f}")
        print(f"時間衰減分布:")
        print(time_decay.describe())
        print("=" * 60)
        
        return time_decay
    
    def compute_sample_weights(self, 
                              t1: pd.Series,
                              close_idx: pd.DatetimeIndex,
                              labels: Optional[pd.Series] = None,
                              close: Optional[pd.Series] = None,
                              use_uniqueness: bool = True,
                              use_time_decay: bool = True,
                              use_return_attribution: bool = False,
                              use_class_balance: bool = False,
                              time_decay_type: str = 'linear',
                              time_decay_params: dict = None) -> pd.Series:
        """
        計算最終樣本權重（可選擇使用哪些機制）
        
        權重組合方式：
        ------------
        1. 基礎權重 = 唯一性（如果啟用）
        2. 可選：× 報酬歸因權重（如果啟用）
        3. 可選：× 時間衰減（如果啟用）
        4. 可選：× 類別平衡權重（如果啟用）
        
        最終權重 = uniqueness × (return_attribution) × (time_decay) × (class_balance)
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        close_idx : pd.DatetimeIndex
            價格數據的索引
        labels : pd.Series, optional
            標籤序列（用於計算類別平衡權重，index 必須與 t1 一致）
        close : pd.Series, optional
            價格序列（用於計算報酬歸因）
        use_uniqueness : bool
            是否使用唯一性權重（預設 True）
        use_time_decay : bool
            是否使用時間衰減（預設 True）
        use_return_attribution : bool
            是否使用報酬歸因權重（預設 False）
        use_class_balance : bool
            是否使用類別平衡權重（預設 False）
        time_decay_type : str
            時間衰減類型：'linear' 或 'exp'（預設 'linear'）
        time_decay_params : dict, optional
            時間衰減參數
            - linear: {'clf_last_w': 0.5}
            - exp: {'decay_rate': 1.2, 'percent_of_zero_wts': 0.0}
            
        Returns:
        --------
        sample_weights : pd.Series
            最終樣本權重（如果都不使用，返回全為 1 的權重）
        """
        print("=" * 60)
        print("📊 樣本權重計算配置")
        print("=" * 60)
        print(f"使用唯一性: {use_uniqueness}")
        print(f"使用報酬歸因: {use_return_attribution}")
        print(f"使用時間衰減: {use_time_decay}")
        print(f"使用類別平衡: {use_class_balance}")
        if use_time_decay:
            print(f"時間衰減類型: {time_decay_type}")
        print("=" * 60)
        
        # 如果都不使用，返回均勻權重
        if not use_uniqueness and not use_time_decay and not use_return_attribution and not use_class_balance:
            sample_weights = pd.Series(1.0, index=t1.index)
            self.sample_weights = sample_weights
            
            print("=" * 60)
            print("📊 樣本權重統計（均勻權重）")
            print("=" * 60)
            print(f"總樣本數: {len(sample_weights):,}")
            print(f"所有樣本權重: 1.0")
            print("=" * 60)
            return sample_weights
        
        # 計算並發度（唯一性和報酬歸因都需要）
        if use_uniqueness or use_return_attribution:
            num_co_events = self.compute_num_co_events(close_idx, t1)
        else:
            num_co_events = None
        
        # 計算唯一性（如果需要）
        if use_uniqueness:
            uniqueness = self.compute_uniqueness(t1, num_co_events)
        else:
            uniqueness = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用唯一性，設為 1.0")
        
        # 計算報酬歸因（如果需要）
        if use_return_attribution:
            if close is None:
                raise ValueError("使用報酬歸因時需要提供 close 參數")
            return_weights = self.compute_return_attribution(close, t1, num_co_events)
        else:
            return_weights = pd.Series(1.0, index=t1.index)
        
        # 計算時間衰減（如果需要）
        if use_time_decay:
            if time_decay_params is None:
                time_decay_params = {}
            
            if time_decay_type == 'linear':
                clf_last_w = time_decay_params.get('clf_last_w', 1.0)
                time_decay = self.get_time_decay_linear(uniqueness, clf_last_w)
            elif time_decay_type == 'exp':
                decay_rate = time_decay_params.get('decay_rate', 1.0)
                percent_of_zero_wts = time_decay_params.get('percent_of_zero_wts', 0.0)
                time_decay = self.get_time_decay_exp(uniqueness, decay_rate, percent_of_zero_wts)
            else:
                raise ValueError(f"不支援的時間衰減類型: {time_decay_type}")
        else:
            time_decay = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用時間衰減，設為 1.0")
        
        # 計算類別平衡權重（如果需要）
        if use_class_balance:
            if labels is None:
                raise ValueError("使用類別平衡時需要提供 labels 參數")
            # 確保 labels 的 index 與 t1 一致
            if not labels.index.equals(t1.index):
                # 嘗試對齊
                common_idx = labels.index.intersection(t1.index)
                if len(common_idx) == 0:
                    raise ValueError("labels 和 t1 的 index 沒有交集")
                labels = labels.loc[common_idx]
                t1 = t1.loc[common_idx]
                uniqueness = uniqueness.loc[common_idx]
                return_weights = return_weights.loc[common_idx]
                time_decay = time_decay.loc[common_idx]
            class_balance_weights = self.compute_class_balance_weight(labels)
        else:
            class_balance_weights = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用類別平衡，設為 1.0")
        
        # 計算最終權重（所有權重相乘）
        common_idx = (uniqueness.index
                     .intersection(time_decay.index)
                     .intersection(return_weights.index)
                     .intersection(class_balance_weights.index))
        
        sample_weights = (uniqueness.loc[common_idx] * 
                          return_weights.loc[common_idx] * 
                          time_decay.loc[common_idx] *
                          class_balance_weights.loc[common_idx])
        
        self.sample_weights = sample_weights
        
        print("=" * 60)
        print("📊 樣本權重統計（最終）")
        print("=" * 60)
        print(f"總樣本數: {len(sample_weights):,}")
        print(f"平均權重: {sample_weights.mean():.4f}")
        print(f"權重分布:")
        print(sample_weights.describe())
        print("=" * 60)
        
        return sample_weights
    
    def plot_concurrency_vs_volatility(self, daily_vol: pd.Series):
        """
        繪製並發度 vs 波動率的關係圖
        
        參數:
        -----------
        daily_vol : pd.Series
            日波動率序列
        """
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        if self.num_co_events is None:
            print("⚠️ 請先執行 compute_num_co_events()")
            return
        
        # 對齊數據（使用原始時間點）
        to_plot = pd.DataFrame({
            'conc_events': self.num_co_events,
            'daily_vol': daily_vol
        }).dropna()
        
        if len(to_plot) == 0:
            print("⚠️ 對齊後無有效數據")
            return
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 10))
        
        # 1. 時間序列圖
        ax1 = axes[0]
        ax1_twin = ax1.twinx()
        
        line1 = ax1.plot(to_plot.index, to_plot['conc_events'], 
                        label='Concurrent Events', color='blue', linewidth=1.5)
        line2 = ax1_twin.plot(to_plot.index, to_plot['daily_vol'], 
                            label='Daily Volatility', color='red', linewidth=1.5, alpha=0.7)
        
        ax1.set_xlabel('Time', fontsize=12)
        ax1.set_ylabel('Concurrent Events', color='blue', fontsize=12)
        ax1_twin.set_ylabel('Daily Volatility', color='red', fontsize=12)
        ax1.set_title('Concurrent Events vs Daily Volatility', fontsize=14)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1_twin.tick_params(axis='y', labelcolor='red')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='upper left')
        ax1_twin.legend(loc='upper right')
        
        # 2. 散點圖
        sns.regplot(x=to_plot['conc_events'], y=to_plot['daily_vol'], 
                ax=axes[1], scatter_kws={'alpha': 0.6, 's': 50})
        axes[1].set_xlabel('Concurrent Events', fontsize=12)
        axes[1].set_ylabel('Daily Volatility', fontsize=12)
        axes[1].set_title('Concurrent Events vs Daily Volatility (Scatter Plot)', fontsize=14)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 打印相關性
        correlation = to_plot['conc_events'].corr(to_plot['daily_vol'])
        print(f"\n📊 並發度與波動率的相關係數: {correlation:.4f}")
    
    def plot_weights_distribution(self):
        """繪製權重分布圖"""
        import matplotlib.pyplot as plt
        
        if self.sample_weights is None:
            print("⚠️ 請先執行 compute_sample_weights()")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. 唯一性分布
        if self.uniqueness is not None:
            self.uniqueness.hist(bins=50, ax=axes[0, 0], alpha=0.7, color='blue')
            axes[0, 0].set_title('Uniqueness Distribution')
            axes[0, 0].set_xlabel('Uniqueness')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].grid(True, alpha=0.3)
        
        # 2. 時間衰減分布
        if self.time_decay is not None:
            self.time_decay.hist(bins=50, ax=axes[0, 1], alpha=0.7, color='green')
            axes[0, 1].set_title('Time Decay Distribution')
            axes[0, 1].set_xlabel('Time Decay Weight')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].grid(True, alpha=0.3)
        
        # 3. 樣本權重分布
        self.sample_weights.hist(bins=50, ax=axes[1, 0], alpha=0.7, color='orange')
        axes[1, 0].set_title('Sample Weights Distribution')
        axes[1, 0].set_xlabel('Sample Weight')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. 權重隨時間變化
        if len(self.sample_weights) > 0:
            sorted_weights = self.sample_weights.sort_index()
            axes[1, 1].plot(sorted_weights.index, sorted_weights.values, 
                           linewidth=1, alpha=0.7, color='purple')
            axes[1, 1].set_title('Sample Weights Over Time')
            axes[1, 1].set_xlabel('Time')
            axes[1, 1].set_ylabel('Sample Weight')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def get_uniqueness_autocorr(self, lag: int = 1) -> float:
        """
        計算唯一性的序列相關性（AR(1)）
        
        參數:
        -----------
        lag : int
            滯後期數（預設 1）
            
        Returns:
        --------
        autocorr : float
            序列相關性
        """
        if self.uniqueness is None:
            print("⚠️ 請先執行 compute_uniqueness()")
            return None
        
        autocorr = self.uniqueness.autocorr(lag=lag)
        
        print("=" * 60)
        print("📊 唯一性序列相關性")
        print("=" * 60)
        print(f"AR({lag}) 自相關係數: {autocorr:.4f}")
        
        # 統計顯著性檢驗（簡化版）
        n = len(self.uniqueness)
        critical_value = 1.96 / np.sqrt(n)  # 95% 信心水準
        
        if abs(autocorr) > critical_value:
            print(f"✅ 統計顯著 (|{autocorr:.4f}| > {critical_value:.4f})")
            print(f"\n解釋:")
            print(f"  - 唯一性存在序列相關性，表示市場狀態具有持續性")
            print(f"  - 這在金融數據中是常見且預期的現象")
            print(f"  - 時間衰減機制有助於降低舊數據的影響")
        else:
            print(f"❌ 統計不顯著 (|{autocorr:.4f}| <= {critical_value:.4f})")
        print("=" * 60)
        
        return autocorr

In [3]:
# =====================================================
# 完整代碼：為所有商品產生分數階差分特徵
# =====================================================

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 2. 修正後的 FractionalDiff class（確保保留所有 index）
# =====================================================

class FractionalDiff:
    """分數階差分特徵生成"""
    
    def __init__(self):
        self.optimal_d = None
        self.fracdiff_series = None
        self.features = None
        
    def get_weights(self, d: float, size: int) -> np.ndarray:
        """計算權重（擴展窗口）"""
        w = [1.0]
        for k in range(1, size):
            w_ = -w[-1] / k * (d - k + 1)
            w.append(float(w_))
        return np.array(w[::-1]).reshape(-1, 1)
    
    def get_weights_FFD(self, d: float, thres: float = 1e-5) -> np.ndarray:
        """計算 FFD 權重（固定窗口）"""
        w = [1.0]
        k = 1
        while True:
            w_ = -w[-1] / k * (d - k + 1)
            if abs(w_) < thres:
                break
            w.append(float(w_))
            k += 1
            if k > 10000:
                break
        return np.array(w[::-1]).reshape(-1, 1)
    
    def frac_diff_FFD(self, series: pd.Series, d: float, thres: float = 1e-5) -> pd.Series:
        """
        分數階差分（固定窗口方法，FFD）
        
        重要：保留所有原始 index，只在前 width 個點設為 NaN
        """
        # 1. 計算權重（固定窗口）
        w = self.get_weights_FFD(d, thres)
        width = len(w) - 1
        
        if width < 1:
            return pd.Series(index=series.index, dtype=float)
        
        # 2. 應用權重到數值
        # 重要：先填充 NaN，但保留原始 index
        series_filled = series.fillna(method='ffill')
        
        # 建立結果 Series，使用原始 series 的完整 index
        df_ = pd.Series(index=series.index, dtype=float)
        
        # 3. 從 width 開始計算（前 width 個點會是 NaN）
        valid_count = 0
        for iloc1 in range(width, len(series_filled)):
            if iloc1 >= len(series.index):
                break
                
            loc0 = series_filled.index[iloc1 - width]
            loc1 = series_filled.index[iloc1]
            
            # 檢查原始 series 在 loc1 是否有有效值
            if loc1 not in series.index or not np.isfinite(series.loc[loc1]):
                continue
            
            # 使用固定窗口：[X_{t-width}, X_{t-width+1}, ..., X_t]
            window_data = series_filled.loc[loc0:loc1].values
            
            if len(window_data) == len(w):
                result = np.dot(w.flatten(), window_data)
                df_[loc1] = result
                valid_count += 1
        
        return df_
    
    def frac_diff(self, series: pd.Series, d: float, thres: float = 0.01) -> pd.Series:
        """分數階差分（擴展窗口）"""
        w = self.get_weights(d, series.shape[0])
        w_ = np.cumsum(np.abs(w))
        w_ /= w_[-1]
        skip = w_[w_ > thres].shape[0]
        
        series_filled = series.fillna(method='ffill')
        df_ = pd.Series(index=series.index, dtype=float)
        
        for iloc in range(skip, series_filled.shape[0]):
            loc = series_filled.index[iloc]
            if not np.isfinite(series.loc[loc]):
                continue
            
            window_data = series_filled.loc[:loc].values
            w_subset = w[-(iloc + 1):, :].flatten()
            
            if len(window_data) == len(w_subset):
                df_[loc] = np.dot(w_subset, window_data)
        
        return df_
    
    def find_optimal_d(self, series: pd.Series, 
                      d_range: Tuple[float, float] = (0, 1),
                      d_step: float = 0.05,
                      method: str = 'FFD',
                      thres: float = 1e-5,
                      target_pvalue: float = 0.05,
                      min_corr: float = 0.5) -> dict:
        """找出最適合的 d 值"""
        from statsmodels.tsa.stattools import adfuller
        from scipy.stats import jarque_bera
        
        results = []
        d_values = np.arange(d_range[0], d_range[1] + d_step, d_step)
        
        for d in d_values:
            try:
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                jb_stat, jb_pvalue = jarque_bera(fracdiff_series)[:2]
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'jb_stat': jb_stat,
                    'jb_pvalue': jb_pvalue,
                    'is_stationary': p_value < target_pvalue,
                    'meets_corr': corr >= min_corr,
                    'sample_size': len(fracdiff_series)
                })
                
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            return None
        
        valid_results = results_df[
            (results_df['is_stationary']) & 
            (results_df['meets_corr'])
        ]
        
        if len(valid_results) > 0:
            optimal = valid_results.loc[valid_results['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        else:
            optimal = results_df.loc[results_df['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        
        return {
            'optimal_d': self.optimal_d,
            'results': results_df,
            'optimal_stats': optimal.to_dict()
        }

    def find_min_d_adf(self, series: pd.Series,
                    d_range: Tuple[float, float] = (0, 1),
                    n_points: int = 11,
                    method: str = 'FFD',
                    thres: float = 0.01,
                    target_pvalue: float = 0.1) -> dict:
        """
        尋找最小 d 值（基於 Snippet 5.4）
        
        這個方法與 find_optimal_d 類似，但專門用於尋找滿足平穩性條件的最小 d 值。
        這是 AFML 書中 Snippet 5.4 的實現。
        
        參數:
        -----------
        series : pd.Series
            價格序列
        d_range : tuple
            d 值搜尋範圍 (min, max)
        n_points : int
            測試的 d 值數量（對應 np.linspace(d_range[0], d_range[1], n_points)）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值（FFD 方法）或累積權重閾值（expanding 方法）
        target_pvalue : float
            目標 p-value（ADF 檢驗）
            
        Returns:
        --------
        result : dict
            包含以下鍵值：
            - 'min_d': 最小 d 值（如果找到）
            - 'results': DataFrame，包含所有測試結果
            - 'min_d_stats': 最小 d 值的統計資訊
        """
        from statsmodels.tsa.stattools import adfuller
        
        results = []
        d_values = np.linspace(d_range[0], d_range[1], n_points)
        
        print(f"🔍 搜尋最小 d 值 (範圍: {d_range}, 測試點數: {n_points})")
        print(f"   方法: {method}, 閾值: {thres}, 目標 p-value: {target_pvalue}")
        
        for i, d in enumerate(d_values):
            try:
                # 計算分數階差分
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                # ADF 檢驗
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                # 計算與原始序列的相關性
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                is_stationary = p_value < target_pvalue
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'is_stationary': is_stationary,
                    'sample_size': len(fracdiff_series)
                })
                
                if (i + 1) % 5 == 0 or i == len(d_values) - 1:
                    print(f"   進度: {i+1}/{len(d_values)} (d={d:.3f}, p={p_value:.4f}, stationary={is_stationary})")
                    
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            print("❌ 未找到任何有效結果")
            return None
        
        # 尋找滿足平穩性條件的最小 d 值
        stationary_results = results_df[results_df['is_stationary'] == True]
        
        if len(stationary_results) > 0:
            min_d_row = stationary_results.loc[stationary_results['d'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n✅ 找到最小 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f}")
            print(f"   相關性: {min_d_row['correlation']:.4f}")
        else:
            # 如果沒有滿足條件的，選擇 p-value 最小的
            min_d_row = results_df.loc[results_df['p_value'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n⚠️ 未找到滿足平穩性條件的 d 值")
            print(f"   使用 p-value 最小的 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f} (目標: < {target_pvalue})")
        
        return {
            'min_d': min_d,
            'results': results_df,
            'min_d_stats': min_d_row.to_dict()
        }

    def _plot_d_search_results(self, result: dict):
        """
        繪製 d 值搜尋結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 或 find_optimal_d 的返回結果
        """
        import matplotlib.pyplot as plt
        
        if result is None or 'results' not in result:
            print("⚠️ 無效的結果")
            return
        
        results_df = result['results']
        min_d = result.get('min_d') or result.get('optimal_d')
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. ADF p-value vs d
        axes[0, 0].plot(results_df['d'], results_df['p_value'], 'o-', linewidth=2, markersize=6)
        if min_d is not None:
            min_d_pvalue = results_df[results_df['d'] == min_d]['p_value'].values[0]
            axes[0, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 0].scatter([min_d], [min_d_pvalue], color='red', s=100, zorder=5)
        axes[0, 0].axhline(0.05, color='gray', linestyle=':', alpha=0.5, label='p=0.05')
        axes[0, 0].axhline(0.1, color='gray', linestyle=':', alpha=0.5, label='p=0.1')
        axes[0, 0].set_xlabel('d', fontsize=12)
        axes[0, 0].set_ylabel('ADF p-value', fontsize=12)
        axes[0, 0].set_title('ADF Test p-value vs d', fontsize=14)
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. ADF Statistic vs d
        axes[0, 1].plot(results_df['d'], results_df['adf_stat'], 'o-', linewidth=2, markersize=6, color='green')
        if min_d is not None:
            min_d_stat = results_df[results_df['d'] == min_d]['adf_stat'].values[0]
            axes[0, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 1].scatter([min_d], [min_d_stat], color='red', s=100, zorder=5)
        axes[0, 1].axhline(0, color='gray', linestyle=':', alpha=0.5)
        axes[0, 1].set_xlabel('d', fontsize=12)
        axes[0, 1].set_ylabel('ADF Statistic', fontsize=12)
        axes[0, 1].set_title('ADF Test Statistic vs d', fontsize=14)
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Correlation vs d
        axes[1, 0].plot(results_df['d'], results_df['correlation'], 'o-', linewidth=2, markersize=6, color='orange')
        if min_d is not None:
            min_d_corr = results_df[results_df['d'] == min_d]['correlation'].values[0]
            axes[1, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 0].scatter([min_d], [min_d_corr], color='red', s=100, zorder=5)
        axes[1, 0].set_xlabel('d', fontsize=12)
        axes[1, 0].set_ylabel('Correlation with Original', fontsize=12)
        axes[1, 0].set_title('Correlation vs d', fontsize=14)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Stationarity Status
        if 'is_stationary' in results_df.columns:
            colors = ['red' if not stat else 'green' for stat in results_df['is_stationary']]
            axes[1, 1].scatter(results_df['d'], results_df['is_stationary'].astype(int), 
                            c=colors, s=100, alpha=0.6)
            if min_d is not None:
                axes[1, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 1].set_xlabel('d', fontsize=12)
            axes[1, 1].set_ylabel('Is Stationary (1=Yes, 0=No)', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
            axes[1, 1].set_ylim(-0.1, 1.1)
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'No stationarity data', 
                            ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
        
        plt.tight_layout()
        plt.show()

    def _plot_min_d_results(self, result: dict, d: float):
        """
        繪製使用最小 d 值後的結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 的返回結果
        d : float
            使用的 d 值
        """
        import matplotlib.pyplot as plt
        
        if result is None:
            print("⚠️ 無效的結果")
            return
        
        # 這個方法可以繪製使用 d 值後的時間序列對比
        # 但需要實際的分數階差分序列，所以這裡只顯示統計資訊
        min_d_stats = result.get('min_d_stats', {})
        
        print("\n" + "=" * 60)
        print("📊 最小 d 值統計資訊")
        print("=" * 60)
        print(f"使用的 d 值: {d:.4f}")
        if min_d_stats:
            print(f"ADF Statistic: {min_d_stats.get('adf_stat', 'N/A'):.4f}")
            print(f"ADF p-value: {min_d_stats.get('p_value', 'N/A'):.4f}")
            print(f"與原始序列相關性: {min_d_stats.get('correlation', 'N/A'):.4f}")
            print(f"樣本數: {min_d_stats.get('sample_size', 'N/A'):,}")
            print(f"是否平穩: {min_d_stats.get('is_stationary', 'N/A')}")
        print("=" * 60)

    def generate_features(self, df: pd.DataFrame, event_indices: pd.DatetimeIndex,
                      price_col: str = 'Close', d: Optional[float] = None,
                      method: str = 'FFD', thres: float = 1e-5) -> pd.DataFrame:
        """
        為事件點生成分數階差分特徵
        
        參數:
        -----------
        df : pd.DataFrame
            包含價格資料的 DataFrame
        event_indices : pd.DatetimeIndex
            事件時間點
        price_col : str
            價格欄位名稱
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
            
        Returns:
        --------
        features : pd.DataFrame
            分數階差分特徵（index 為 event_indices）
        """
        if price_col not in df.columns:
            raise ValueError(f"找不到價格欄位: {price_col}")
        
        # 使用指定的 d 值或最優 d 值
        if d is None:
            if self.optimal_d is None:
                raise ValueError("請先執行 find_optimal_d() 或 find_min_d_adf()，或指定 d 參數")
            d = self.optimal_d
        
        print(f"📊 生成分數階差分特徵 (d={d:.4f}, method={method})")
        
        # 計算整個序列的分數階差分
        price_series = df[price_col]
        
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d, thres)
        
        # 提取事件點的特徵值
        features = pd.DataFrame(index=event_indices)
        features['fracdiff'] = fracdiff_series.loc[event_indices].values
        
        # 統計資訊
        valid_count = features['fracdiff'].notna().sum()
        print(f"✅ 特徵生成完成")
        print(f"   事件數: {len(event_indices):,}")
        print(f"   有效特徵數: {valid_count:,}")
        print(f"   NaN 數: {len(event_indices) - valid_count:,}")
        
        return features
    
    def plot_price_vs_fracdiff(self, price_series: pd.Series, 
                            d: Optional[float] = None,
                            method: str = 'FFD',
                            thres: float = 1e-5,
                            n_points: Optional[int] = None,
                            normalize: bool = False):
        """
        繪製原始價格與分數階差分特徵的對比圖
        
        參數:
        -----------
        price_series : pd.Series
            原始價格序列
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d 或預設值 0.5）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
        n_points : int, optional
            如果指定，只繪製最近 n_points 個點（用於大量數據時）
        normalize : bool
            是否標準化後繪製在同一張圖上（預設 False，分開兩個子圖）
            
        Returns:
        --------
        fracdiff_series : pd.Series
            計算出的分數階差分序列
        """
        import matplotlib.pyplot as plt
        
        # 確定使用的 d 值
        if d is None:
            if self.optimal_d is not None:
                d_value = self.optimal_d
            else:
                d_value = 0.5
                print(f"⚠️ 未指定 d 值，使用預設值 {d_value}")
        else:
            d_value = d
        
        # 計算整個序列的分數階差分
        print(f"📊 計算分數階差分 (d={d_value:.4f}, method={method})...")
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d_value, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d_value, thres)
        
        # 選擇要繪製的數據範圍
        if n_points is not None and len(price_series) > n_points:
            plot_idx = price_series.index[-n_points:]
            price_plot = price_series.loc[plot_idx]
            fracdiff_plot = fracdiff_series.loc[plot_idx]
            title_suffix = f" (Last {n_points:,} Points)"
        else:
            plot_idx = price_series.index
            price_plot = price_series
            fracdiff_plot = fracdiff_series
            title_suffix = ""
        
        # 繪製圖表
        if normalize:
            # 方法 1: 標準化後繪製在同一張圖上
            common_idx = plot_idx.intersection(fracdiff_plot.dropna().index)
            if len(common_idx) > 0:
                price_normalized = (price_plot.loc[common_idx] - price_plot.loc[common_idx].mean()) / price_plot.loc[common_idx].std()
                fracdiff_normalized = (fracdiff_plot.loc[common_idx] - fracdiff_plot.loc[common_idx].mean()) / fracdiff_plot.loc[common_idx].std()
                
                fig, ax = plt.subplots(figsize=(15, 6))
                
                ax.plot(common_idx, price_normalized, 
                    label='Original Price (Normalized)', linewidth=1.5, color='blue', alpha=0.7)
                ax.plot(common_idx, fracdiff_normalized, 
                    label='Fractional Differentiation (Normalized)', linewidth=1.5, color='red', alpha=0.7)
                ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.3)
                ax.set_xlabel('Time', fontsize=12)
                ax.set_ylabel('Normalized Value', fontsize=12)
                ax.set_title(f'Normalized Comparison: Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
                ax.legend(loc='upper left')
                ax.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        else:
            # 方法 2: 分開兩個子圖（預設）
            fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
            
            # 上圖：原始價格
            axes[0].plot(plot_idx, price_plot.values, 
                        label='Original Price', linewidth=1.5, color='blue', alpha=0.7)
            axes[0].set_ylabel('Price', fontsize=12)
            axes[0].set_title(f'Original Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
            axes[0].legend(loc='upper left')
            axes[0].grid(True, alpha=0.3)
            
            # 下圖：分數階差分特徵
            valid_fracdiff = fracdiff_plot.dropna()
            if len(valid_fracdiff) > 0:
                axes[1].plot(valid_fracdiff.index, valid_fracdiff.values, 
                            label='Fractional Differentiation', linewidth=1.5, color='red', alpha=0.7)
            axes[1].axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
            axes[1].set_xlabel('Time', fontsize=12)
            axes[1].set_ylabel('Fractional Differentiation', fontsize=12)
            axes[1].set_title('Fractional Differentiation Feature', fontsize=14)
            axes[1].legend(loc='upper left')
            axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # 統計資訊
        valid_count = fracdiff_series.notna().sum()
        print(f"✅ 繪圖完成")
        print(f"   原始數據點數: {len(price_series):,}")
        print(f"   分數階差分有效值: {valid_count:,}")
        print(f"   NaN 值: {len(price_series) - valid_count:,}")
        
        return fracdiff_series

In [4]:
# =====================================================
# 特徵篩選 Pipeline Class
# =====================================================

class FeatureFilteringPipeline:
    """
    完整的特徵篩選 Pipeline
    
    包含：
    1. Correlation Filtering（相關性過濾）
    2. Variance Threshold（低變異剔除）
    3. IC (Information Coefficient) Ranking
    4. ANOVA / t-test / χ² 統計檢定
    5. MDA (Permutation Importance)
    6. Stability Selection（可選）
    """
    
    def __init__(self, 
                 corr_threshold=0.95,
                 variance_threshold=0.0,
                 ic_threshold=None,
                 anova_p_threshold=0.05,
                 mda_top_n=None,
                 stability_n_iter=50,
                 stability_threshold=0.6):
        """
        Parameters:
        -----------
        corr_threshold : float
            相關性閾值，超過此值視為高度相關（預設 0.95）
        variance_threshold : float
            變異數閾值，低於此值視為低變異（預設 0.0）
        ic_threshold : float or None
            IC 閾值，低於此值視為無預測力（None 表示保留 top N）
        anova_p_threshold : float
            ANOVA p-value 閾值（預設 0.05）
        mda_top_n : int or None
            MDA 保留前 N 個特徵（None 表示保留所有通過檢定的）
        stability_n_iter : int
            Stability Selection 迭代次數（預設 50）
        stability_threshold : float
            Stability Selection 選擇頻率閾值（預設 0.6）
        """
        self.corr_threshold = corr_threshold
        self.variance_threshold = variance_threshold
        self.ic_threshold = ic_threshold
        self.anova_p_threshold = anova_p_threshold
        self.mda_top_n = mda_top_n
        self.stability_n_iter = stability_n_iter
        self.stability_threshold = stability_threshold
        
        self.selected_features = None
        self.filter_results = {}
        
    def correlation_filter(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        1️⃣ Correlation Filtering（相關性過濾）
        
        移除高度相關的特徵（|ρ| > threshold）
        """
        print(f"\n{'='*60}")
        print("1️⃣ Correlation Filtering")
        print(f"{'='*60}")
        
        corr_matrix = X.corr().abs()
        
        # 找出高度相關的特徵對
        upper_triangle = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )
        
        # 找出需要移除的特徵
        to_remove = set()
        for col in upper_triangle.columns:
            high_corr = upper_triangle.index[upper_triangle[col] > self.corr_threshold]
            if len(high_corr) > 0:
                # 保留與其他特徵相關性較高的（保留更重要的）
                # 簡單策略：保留第一個，移除其他的
                to_remove.update(high_corr[1:])
        
        removed_count = len(to_remove)
        remaining_features = [col for col in X.columns if col not in to_remove]
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除高度相關特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(remaining_features):,}")
        print(f"   相關性閾值: {self.corr_threshold}")
        
        self.filter_results['correlation'] = {
            'removed': list(to_remove),
            'remaining': remaining_features,
            'removed_count': removed_count
        }
        
        return X[remaining_features]
    
    def variance_filter(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        2️⃣ Variance Threshold（低變異剔除）
        """
        print(f"\n{'='*60}")
        print("2️⃣ Variance Threshold")
        print(f"{'='*60}")
        
        selector = VarianceThreshold(threshold=self.variance_threshold)
        X_selected = selector.fit_transform(X)
        selected_features = X.columns[selector.get_support()]
        removed_count = len(X.columns) - len(selected_features)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低變異特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected_features):,}")
        print(f"   變異數閾值: {self.variance_threshold}")
        
        self.filter_results['variance'] = {
            'removed': [col for col in X.columns if col not in selected_features],
            'remaining': list(selected_features),
            'removed_count': removed_count
        }
        
        return pd.DataFrame(X_selected, index=X.index, columns=selected_features)
    
    def ic_ranking(self, X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
        """
        3️⃣ IC (Information Coefficient) Ranking
        
        計算每個特徵與標籤的 Rank Correlation
        使用 |IC| 來判斷預測能力，正負 IC 都有預測價值
        """
        print(f"\n{'='*60}")
        print("3️⃣ IC (Information Coefficient) Ranking")
        print(f"{'='*60}")
        
        ic_scores = {}
        
        for col in X.columns:
            try:
                # 計算 Spearman rank correlation
                ic = stats.spearmanr(X[col], y, nan_policy='omit')[0]
                if np.isnan(ic):
                    ic = 0.0
                ic_scores[col] = ic
            except:
                ic_scores[col] = 0.0
        
        ic_series = pd.Series(ic_scores)
        ic_series = ic_series.sort_values(ascending=False)
        
        # 根據 |IC| 閾值或保留 top N（使用絕對值）
        if self.ic_threshold is not None:
            # 使用絕對值判斷預測能力
            selected = ic_series[ic_series.abs() >= self.ic_threshold].index.tolist()
        else:
            # 保留 |IC| 前 50% 的特徵（按絕對值排序）
            n_keep = max(1, int(len(ic_series) * 0.5))
            selected = ic_series.abs().nlargest(n_keep).index.tolist()
        
        removed_count = len(X.columns) - len(selected)
        
        # 統計資訊
        positive_ic = ic_series[ic_series > 0]
        negative_ic = ic_series[ic_series < 0]
        abs_ic = ic_series.abs()
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低 |IC| 特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"   IC 閾值: |IC| >= {self.ic_threshold if self.ic_threshold is not None else 'Top 50%'}")
        print(f"\n   IC 統計:")
        print(f"     平均 IC: {ic_series.mean():.4f}")
        print(f"     平均 |IC|: {abs_ic.mean():.4f}")
        print(f"     中位數 |IC|: {abs_ic.median():.4f}")
        print(f"     最大 IC: {ic_series.max():.4f}")
        print(f"     最小 IC: {ic_series.min():.4f}")
        print(f"     最大 |IC|: {abs_ic.max():.4f}")
        print(f"\n   IC 分布:")
        print(f"     正 IC 特徵數: {len(positive_ic):,} (平均: {positive_ic.mean():.4f})")
        print(f"     負 IC 特徵數: {len(negative_ic):,} (平均: {negative_ic.mean():.4f})")
        print(f"     零 IC 特徵數: {len(ic_series[ic_series == 0]):,}")
        
        # 顯示篩選後的特徵（按 |IC| 排序）
        selected_ic = ic_series[selected].abs().sort_values(ascending=False)
        print(f"\n   Top 10 篩選後特徵 (按 |IC| 排序):")
        for i, (feat, ic_val) in enumerate(selected_ic.head(10).items(), 1):
            original_ic = ic_series[feat]
            direction = "正相關" if original_ic > 0 else "負相關"
            print(f"     {i:2d}. {feat}: |IC|={ic_val:.4f} (IC={original_ic:.4f}, {direction})")
        
        self.filter_results['ic'] = {
            'scores': ic_series.to_dict(),
            'abs_scores': abs_ic.to_dict(),
            'selected': selected,
            'removed_count': removed_count,
            'positive_count': len(positive_ic),
            'negative_count': len(negative_ic)
        }
        
        return X[selected]
    
    def anova_filter(self, X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
        """
        4️⃣ ANOVA / t-test / χ² 統計檢定
        """
        print(f"\n{'='*60}")
        print("4️⃣ ANOVA / t-test / χ² Statistical Tests")
        print(f"{'='*60}")
        
        selected_features = []
        p_values = {}
        
        for col in X.columns:
            try:
                # 根據標籤分組
                groups = [X[col][y == label].dropna() for label in y.unique()]
                groups = [g for g in groups if len(g) > 0]
                
                if len(groups) < 2:
                    p_values[col] = 1.0
                    continue
                
                # 使用 ANOVA F-test
                f_stat, p_val = stats.f_oneway(*groups)
                p_values[col] = p_val
                
                if p_val < self.anova_p_threshold:
                    selected_features.append(col)
            except:
                p_values[col] = 1.0
        
        removed_count = len(X.columns) - len(selected_features)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除高 p-value 特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected_features):,}")
        print(f"   p-value 閾值: {self.anova_p_threshold}")
        
        # 顯示統計顯著的特徵
        significant = pd.Series(p_values)
        significant = significant[significant < self.anova_p_threshold].sort_values()
        print(f"\n   Top 10 最顯著特徵:")
        for i, (feat, p_val) in enumerate(significant.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: p={p_val:.6f}")
        
        self.filter_results['anova'] = {
            'p_values': p_values,
            'selected': selected_features,
            'removed_count': removed_count
        }
        
        return X[selected_features]
    
    def mda_importance(self, X: pd.DataFrame, y: pd.Series, 
                       sample_weight=None, n_estimators=100) -> pd.DataFrame:
        """
        5️⃣ MDA (Permutation Importance)
        
        使用 Random Forest 計算 Permutation Importance
        """
        print(f"\n{'='*60}")
        print("5️⃣ MDA (Permutation Importance)")
        print(f"{'='*60}")
        
        print(f"   訓練 Random Forest 模型...")
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=10,
            min_samples_split=20,
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )
        
        rf.fit(X, y, sample_weight=sample_weight)
        
        # 計算 baseline score
        baseline_score = rf.score(X, y, sample_weight=sample_weight)
        
        print(f"   Baseline Score: {baseline_score:.4f}")
        print(f"   計算 Permutation Importance...")
        
        # 計算每個特徵的 permutation importance
        importances = {}
        for col in X.columns:
            X_permuted = X.copy()
            X_permuted[col] = np.random.permutation(X_permuted[col].values)
            permuted_score = rf.score(X_permuted, y, sample_weight=sample_weight)
            importance = baseline_score - permuted_score
            importances[col] = importance
        
        importance_series = pd.Series(importances).sort_values(ascending=False)
        
        # 選擇重要特徵
        if self.mda_top_n is not None:
            selected = importance_series.head(self.mda_top_n).index.tolist()
        else:
            # 保留重要性 > 0 的特徵
            selected = importance_series[importance_series > 0].index.tolist()
        
        removed_count = len(X.columns) - len(selected)
        
        print(f"   原始特徵數: {len(X.columns):,}")
        print(f"   移除低重要性特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"\n   Top 10 最重要特徵:")
        for i, (feat, imp) in enumerate(importance_series.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: {imp:.6f}")
        
        self.filter_results['mda'] = {
            'importances': importance_series.to_dict(),
            'selected': selected,
            'removed_count': removed_count
        }
        
        return X[selected]
    
    def stability_selection(self, X: pd.DataFrame, y: pd.Series,
                           sample_weight=None, n_estimators=50):
        """
        6️⃣ Stability Selection
        
        通過多次子抽樣和模型訓練，統計特徵被選中的頻率
        """
        print(f"\n{'='*60}")
        print("6️⃣ Stability Selection")
        print(f"{'='*60}")
        print(f"   迭代次數: {self.stability_n_iter}")
        print(f"   選擇頻率閾值: {self.stability_threshold}")
        
        feature_selection_counts = {col: 0 for col in X.columns}
        
        for i in range(self.stability_n_iter):
            if (i + 1) % 10 == 0:
                print(f"   進度: {i+1}/{self.stability_n_iter}")
            
            # 子抽樣（80% 數據）
            n_samples = int(len(X) * 0.8)
            sample_idx = np.random.choice(X.index, size=n_samples, replace=False)
            X_sub = X.loc[sample_idx]
            y_sub = y.loc[sample_idx]
            w_sub = sample_weight.loc[sample_idx] if sample_weight is not None else None
            
            # 訓練模型並獲取特徵重要性
            rf = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=10,
                min_samples_split=20,
                random_state=i,
                n_jobs=-1,
                class_weight='balanced'
            )
            
            rf.fit(X_sub, y_sub, sample_weight=w_sub)
            
            # 獲取特徵重要性（使用 MDI）
            importances = pd.Series(rf.feature_importances_, index=X.columns)
            top_features = importances.nlargest(int(len(X.columns) * 0.3)).index  # 保留前30%
            
            for feat in top_features:
                feature_selection_counts[feat] += 1
        
        # 計算選擇頻率
        selection_freq = pd.Series(feature_selection_counts) / self.stability_n_iter
        selection_freq = selection_freq.sort_values(ascending=False)
        
        # 根據閾值選擇特徵
        selected = selection_freq[selection_freq >= self.stability_threshold].index.tolist()
        removed_count = len(X.columns) - len(selected)
        
        print(f"\n   原始特徵數: {len(X.columns):,}")
        print(f"   移除不穩定特徵: {removed_count:,}")
        print(f"   剩餘特徵數: {len(selected):,}")
        print(f"\n   Top 10 最穩定特徵:")
        for i, (feat, freq) in enumerate(selection_freq.head(10).items(), 1):
            print(f"     {i:2d}. {feat}: {freq:.4f}")
        
        self.filter_results['stability'] = {
            'frequencies': selection_freq.to_dict(),
            'selected': selected,
            'removed_count': removed_count
        }
        
        return X[selected]
    
    def fit_transform(self, X: pd.DataFrame, y: pd.Series, 
                     sample_weight=None, use_stability=False):
        """
        執行完整的篩選流程
        
        Parameters:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            標籤
        sample_weight : pd.Series, optional
            樣本權重
        use_stability : bool
            是否使用 Stability Selection（較慢）
        
        Returns:
        --------
        X_selected : pd.DataFrame
            篩選後的特徵矩陣
        """
        print(f"\n{'='*80}")
        print("🚀 開始執行特徵篩選 Pipeline")
        print(f"{'='*80}")
        print(f"原始特徵數: {len(X.columns):,}")
        print(f"原始樣本數: {len(X):,}")
        
        X_current = X.copy()
        
        # 1. Correlation Filter
        X_current = self.correlation_filter(X_current)
        
        # 2. Variance Filter
        X_current = self.variance_filter(X_current)
        
        # 3. IC Ranking
        X_current = self.ic_ranking(X_current, y)
        
        # 4. ANOVA Filter
        X_current = self.anova_filter(X_current, y)
        
        # 5. MDA Importance
        X_current = self.mda_importance(X_current, y, sample_weight)
        
        # 6. Stability Selection (可選，較慢)
        if use_stability:
            X_current = self.stability_selection(X_current, y, sample_weight)
        
        self.selected_features = list(X_current.columns)
        
        print(f"\n{'='*80}")
        print("✅ 特徵篩選完成")
        print(f"{'='*80}")
        print(f"最終特徵數: {len(self.selected_features):,}")
        print(f"特徵減少比例: {(1 - len(self.selected_features) / len(X.columns)) * 100:.2f}%")
        
        return X_current

# =====================================================
# 執行特徵篩選
# =====================================================


In [5]:
from scipy.stats import rv_continuous
import numpy as np

class logUniform_gen(rv_continuous):
    """
    Log-uniform distribution random variable generator
    
    說明:
    -----
    隨機變數 x 在 [a, b] 區間內遵循 log-uniform 分布，
    當且僅當 log[x] ~ U[log[a], log[b]]
    
    這個分布對於探索非線性響應的參數空間特別有效，
    例如 SVC 的 C 參數和 RBF kernel 的 gamma 參數。
    """
    
    def _cdf(self, x):
        """累積分布函數 (CDF)"""
        return np.log(x / self.a) / np.log(self.b / self.a)


def logUniform(a=1, b=np.exp(1)):
    """
    創建 log-uniform 分布
    
    參數:
    -----------
    a : float
        下界（必須 > 0）
    b : float
        上界（必須 > a）
        
    Returns:
    --------
    logUniform_gen : logUniform_gen
        Log-uniform 分布生成器
    """
    return logUniform_gen(a=a, b=b, name='logUniform')

In [6]:
# =====================================================
# Purged K-Fold Cross-Validation (Snippet 7.3 & 7.4)
# =====================================================

from sklearn.model_selection._split import _BaseKFold
import numpy as np
import pandas as pd
from typing import Optional, Generator, Tuple
from sklearn.metrics import log_loss, accuracy_score

class PurgedKFold(_BaseKFold):
    """
    Extend KFold class to work with labels that span intervals
    
    The train is purged of observations overlapping test-label intervals
    Test set is assumed contiguous (shuffle=False), w/o training samples in between
    
    說明:
    -----
    這個類別擴展了 scikit-learn 的 KFold，用於處理標籤重疊的情況。
    當標籤跨越時間區間時（例如 Triple Barrier Method），測試集的標籤
    可能會與訓練集的標籤重疊，導致數據洩漏。
    
    解決方法:
    ---------
    1. Purging（清除）: 從訓練集中移除與測試集標籤區間重疊的觀測值
    2. Embargo（禁制）: 在測試集結束後，額外移除一段時間的訓練樣本
    
    參數:
    -----------
    n_splits : int
        K 折交叉驗證的折數
    t1 : pd.Series
        標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
    pctEmbargo : float
        禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
    """
    
    def __init__(self, n_splits: int = 3, t1: Optional[pd.Series] = None, pctEmbargo: float = 0.0):
        """
        初始化 PurgedKFold
        
        參數:
        -----------
        n_splits : int
            K 折交叉驗證的折數
        t1 : pd.Series, optional
            標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
        pctEmbargo : float
            禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
        """
        if t1 is not None and not isinstance(t1, pd.Series):
            raise ValueError('Label Through Dates must be a pd.Series')
        
        super(PurgedKFold, self).__init__(n_splits, shuffle=False, random_state=None)
        self.t1 = t1
        self.pctEmbargo = pctEmbargo
    
    def split(self, X: pd.DataFrame, y: Optional[pd.Series] = None, 
              groups: Optional[np.ndarray] = None) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
        """
        生成訓練/測試集的索引
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據（index 必須與 t1 的 index 一致）
        y : pd.Series, optional
            標籤（未使用，但保留以符合 scikit-learn 接口）
        groups : np.ndarray, optional
            分組（未使用，但保留以符合 scikit-learn 接口）
            
        Yields:
        -------
        train_indices : np.ndarray
            訓練集索引
        test_indices : np.ndarray
            測試集索引
        """
        if self.t1 is None:
            raise ValueError('t1 (ThruDate) must be provided')
        
        # 驗證 X 和 t1 的 index 一致
        if (X.index == self.t1.index).sum() != len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')
        
        indices = np.arange(X.shape[0])
        mbrg = int(X.shape[0] * self.pctEmbargo)  # embargo 的樣本數
        
        # 將數據分成 n_splits 個連續的測試集
        test_starts = [(i[0], i[-1] + 1) for i in 
                      np.array_split(np.arange(X.shape[0]), self.n_splits)]
        
        for i, j in test_starts:
            # 測試集的起始時間
            t0 = self.t1.index[i]  # start of test set
            test_indices = indices[i:j]
            
            # 找到測試集中最晚的結束時間
            maxT1Idx = self.t1.index.searchsorted(self.t1[test_indices].max())
            
            # 左側訓練集：結束時間 <= t0 的所有樣本
            train_indices = self.t1.index.searchsorted(
                self.t1[self.t1 <= t0].index
            )
            
            # 右側訓練集：在測試集結束後，加上 embargo 期間
            if maxT1Idx < X.shape[0]:
                # right train (with embargo)
                train_indices = np.concatenate((
                    train_indices, 
                    indices[maxT1Idx + mbrg:]
                ))
            
            yield train_indices, test_indices


def macro_accuracy_score(y_true, y_pred, sample_weight=None, zero_division=0):
    """
    計算 Macro Accuracy（每個類別的準確率直接平均）
    
    參數:
    -----------
    y_true : array-like
        真實標籤
    y_pred : array-like
        預測標籤
    sample_weight : array-like, optional
        樣本權重
    zero_division : float
        當分母為 0 時返回的值
        
    Returns:
    --------
    macro_accuracy : float
        Macro Accuracy（每個類別的準確率直接平均）
    """
    from sklearn.metrics import accuracy_score
    from sklearn.utils.multiclass import unique_labels
    import numpy as np
    
    # 獲取所有類別
    labels = unique_labels(y_true, y_pred)
    
    # 計算每個類別的準確率
    per_class_accuracies = []
    
    for label in labels:
        # 找出該類別的所有樣本
        mask = (y_true == label)
        
        if mask.sum() == 0:
            # 如果該類別在真實標籤中不存在，跳過
            continue
            
        # 該類別的樣本
        y_true_class = y_true[mask]
        y_pred_class = y_pred[mask]
        
        # 如果有樣本權重
        if sample_weight is not None:
            sample_weight_class = sample_weight[mask]
        else:
            sample_weight_class = None
        
        # 計算該類別的準確率
        class_accuracy = accuracy_score(
            y_true_class, 
            y_pred_class, 
            sample_weight=sample_weight_class
        )
        per_class_accuracies.append(class_accuracy)
    
    # 如果沒有任何類別，返回 zero_division
    if len(per_class_accuracies) == 0:
        return zero_division
    
    # 返回平均準確率（Macro Accuracy）
    return np.mean(per_class_accuracies)

def cvScore(
    clf,
    X: pd.DataFrame,
    y: pd.Series,
    sample_weight: Optional[pd.Series] = None,
    scoring: str = 'f1',
    t1: Optional[pd.Series] = None,
    cv: Optional[int] = None,
    cvGen: Optional[PurgedKFold] = None,
    pctEmbargo: Optional[float] = None,) -> np.ndarray:
    """
    使用 PurgedKFold 進行交叉驗證評分，確保每折標籤重新編碼為連續整數。
    """
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import (
        f1_score,
        precision_score,
        recall_score,
        accuracy_score,
        log_loss,
    )

    if cvGen is None:
        if cv is None:
            raise ValueError("請提供 cv 或 cvGen 其中之一")
        cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)

    if sample_weight is None:
        sample_weight = pd.Series(1.0, index=X.index)

    scores = []

    for fold_idx, (train_idx, test_idx) in enumerate(cvGen.split(X=X)):
        X_train_fold = X.iloc[train_idx, :]
        y_train_fold = y.iloc[train_idx]
        X_test_fold = X.iloc[test_idx, :]
        y_test_fold = y.iloc[test_idx]

        fold_label_encoder = LabelEncoder()
        y_train_fold_encoded = pd.Series(
            fold_label_encoder.fit_transform(y_train_fold),
            index=y_train_fold.index,
        )
        y_test_fold_encoded = pd.Series(
            fold_label_encoder.transform(y_test_fold),
            index=y_test_fold.index,
        )

        sw_train = sample_weight.iloc[train_idx].values
        sw_test = sample_weight.iloc[test_idx].values

        fit = clf.fit(
            X=X_train_fold,
            y=y_train_fold_encoded,
            sample_weight=sw_train,
        )

        if scoring == 'neg_log_loss':
            prob = fit.predict_proba(X_test_fold)
            score_ = -log_loss(
                y_test_fold_encoded,
                prob,
                sample_weight=sw_test,
            )
        elif scoring in ('f1', 'f1_macro'):
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'f1_weighted':
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='weighted',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'precision_macro':
            pred = fit.predict(X_test_fold)
            score_ = precision_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'recall_macro':
            pred = fit.predict(X_test_fold)
            score_ = recall_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )
        elif scoring == 'accuracy':
            pred = fit.predict(X_test_fold)
            score_ = accuracy_score(
                y_test_fold_encoded,
                pred,
                sample_weight=sw_test,
            )
        else:
            pred = fit.predict(X_test_fold)
            score_ = f1_score(
                y_test_fold_encoded,
                pred,
                average='macro',
                sample_weight=sw_test,
                zero_division=0,
            )

        scores.append(score_)

    return np.array(scores)


In [7]:
# =====================================================
# 步驟7：ML Model 構建與訓練（擴展版 - 支援多種模型）
# =====================================================

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              ExtraTreesClassifier, AdaBoostClassifier,
                              HistGradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import (classification_report, confusion_matrix, 
                            roc_auc_score, precision_recall_curve, 
                            roc_curve, accuracy_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Dict, Any, Tuple, List
import warnings
warnings.filterwarnings('ignore')

class ModelTrainer:
    """
    擴展版模型訓練器：支援多種模型
    
    支援的模型類型:
    -----
    - 'xgboost': XGBoost
    - 'lightgbm': LightGBM
    - 'catboost': CatBoost
    - 'rf': Random Forest
    - 'et': Extra Trees
    - 'gb': Gradient Boosting
    - 'histgbm': HistGradientBoosting
    - 'dt': Decision Tree
    - 'lr': Logistic Regression
    - 'svm': Support Vector Machine
    - 'mlp': Multi-Layer Perceptron
    - 'lstm': LSTM (需要額外處理)
    - 'ada': AdaBoost
    - 'knn': K-Nearest Neighbors
    - 'nb': Naive Bayes
    - 'qda': Quadratic Discriminant Analysis
    """
    
    def __init__(self, model_type: str = 'xgboost'):
        """
        初始化模型訓練器
        
        參數:
        -----------
        model_type : str
            模型類型
        """
        self.model_type = model_type.lower()
        self.model = None
        self.best_params = None
        self.cv_scores = None
        
    def _create_model(self, **kwargs):
        """
        創建模型實例
        """
        if self.model_type == 'xgboost':
            default_params = {
                'random_state': 42,
                'eval_metric': 'logloss',
                'use_label_encoder': False,
                'verbosity': 0
            }
            default_params.update(kwargs)
            return XGBClassifier(**default_params)
        
        elif self.model_type == 'lightgbm':
            default_params = {
                'random_state': 42,
                'verbosity': -1,
                'force_col_wise': True
            }
            default_params.update(kwargs)
            return LGBMClassifier(**default_params)
        
        elif self.model_type == 'catboost':
            default_params = {
                'random_state': 42,
                'verbose': False,
                'thread_count': -1
            }
            default_params.update(kwargs)
            return CatBoostClassifier(**default_params)
        
        elif self.model_type == 'rf':
            default_params = {
                'random_state': 42,
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return RandomForestClassifier(**default_params)
        
        elif self.model_type == 'et':
            default_params = {
                'random_state': 42,
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return ExtraTreesClassifier(**default_params)
        
        elif self.model_type == 'gb':
            default_params = {
                'random_state': 42
            }
            default_params.update(kwargs)
            return GradientBoostingClassifier(**default_params)
        
        elif self.model_type == 'histgbm':
            default_params = {
                'random_state': 42,
                'max_iter': 100,
                'early_stopping': True
            }
            default_params.update(kwargs)
            return HistGradientBoostingClassifier(**default_params)
        
        elif self.model_type == 'dt':
            default_params = {
                'random_state': 42,
                'max_depth': 10
            }
            default_params.update(kwargs)
            return DecisionTreeClassifier(**default_params)
        
        elif self.model_type == 'lr':
            default_params = {
                'random_state': 42,
                'max_iter': 1000,
                'solver': 'lbfgs'
            }
            default_params.update(kwargs)
            return LogisticRegression(**default_params)
        
        elif self.model_type == 'svm':
            default_params = {
                'random_state': 42,
                'probability': True,
                'kernel': 'rbf'
            }
            default_params.update(kwargs)
            return SVC(**default_params)
        
        elif self.model_type == 'mlp':
            default_params = {
                'random_state': 42,
                'max_iter': 500,
                'early_stopping': True,
                'validation_fraction': 0.1
            }
            default_params.update(kwargs)
            return MLPClassifier(**default_params)
        
        elif self.model_type == 'ada':
            default_params = {
                'random_state': 42
            }
            default_params.update(kwargs)
            return AdaBoostClassifier(**default_params)
        
        elif self.model_type == 'knn':
            default_params = {
                'n_neighbors': 5,
                'weights': 'distance',
                'n_jobs': -1
            }
            default_params.update(kwargs)
            return KNeighborsClassifier(**default_params)
        
        elif self.model_type == 'nb':
            default_params = {}
            default_params.update(kwargs)
            return GaussianNB(**default_params)
        
        elif self.model_type == 'qda':
            default_params = {}
            default_params.update(kwargs)
            return QuadraticDiscriminantAnalysis(**default_params)
        
        elif self.model_type == 'lstm':
            # LSTM 需要特殊處理，這裡先拋出提示
            raise NotImplementedError(
                "LSTM 需要序列數據和特殊處理，請使用專門的深度學習框架（如 TensorFlow/Keras）"
            )
        
        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        """
        print(f"🚀 取得 {self.model_type} 模型參數網格")
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
        
        elif self.model_type == 'lightgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10, -1],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__num_leaves': [15, 31, 50, 100],
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_samples': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__num_leaves': [31, 50],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
        
        elif self.model_type == 'catboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300, 500],
                    f'{self.model_type}__depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__l2_leaf_reg': [1, 3, 5, 7],
                    f'{self.model_type}__border_count': [32, 64, 128],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300],
                    f'{self.model_type}__depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__l2_leaf_reg': [3, 5],
                }
                return param_grid
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500, 1000],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': logUniform(1e-3, 1e2),
                    f'{self.model_type}__min_samples_leaf': logUniform(1e-3, 1e1),
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                    f'{self.model_type}__bootstrap': [True, False]
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None]
                }
                return param_grid
        
        elif self.model_type == 'et':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__max_features': ['sqrt', 'log2'],
                }
                return param_grid
        
        elif self.model_type == 'gb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200],
                    f'{self.model_type}__max_depth': [3, 5],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                }
                return param_grid
        
        elif self.model_type == 'histgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200, 300],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__min_samples_leaf': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                }
                return param_grid
        
        elif self.model_type == 'dt':
            param_grid = {
                f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                f'{self.model_type}__min_samples_split': [2, 5, 10],
                f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                f'{self.model_type}__max_features': ['sqrt', 'log2', None],
            }
            return param_grid
        
        elif self.model_type == 'lr':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-3, 1e3),
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.01, 0.1, 1.0, 10.0],
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
        
        elif self.model_type == 'svm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-2, 1e2),
                    f'{self.model_type}__gamma': logUniform(1e-4, 1e-1),
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.1, 1.0, 10.0],
                    f'{self.model_type}__gamma': ['scale', 'auto', 0.001, 0.01],
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
        
        elif self.model_type == 'mlp':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                    f'{self.model_type}__alpha': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50)],
                    f'{self.model_type}__alpha': [0.0001, 0.001, 0.01],
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
        
        elif self.model_type == 'ada':
            param_grid = {
                f'{self.model_type}__n_estimators': [50, 100, 200],
                f'{self.model_type}__learning_rate': [0.01, 0.1, 1.0],
            }
            return param_grid   
        
        elif self.model_type == 'knn':
            param_grid = {
                f'{self.model_type}__n_neighbors': [3, 5, 7, 10, 15],
                f'{self.model_type}__weights': ['uniform', 'distance'],
                f'{self.model_type}__metric': ['euclidean', 'manhattan', 'minkowski'],
            }
            return param_grid
        elif self.model_type == 'nb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__var_smoothing': logUniform(1e-9, 1e-3),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
                }
                return param_grid
        elif self.model_type == 'qda':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__reg_param': logUniform(1e-3, 1.0),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__reg_param': [0.0, 0.1, 0.5, 1.0],
                }   
                return param_grid
        else:
            # 如果模型類型沒有對應的參數網格，返回空字典
            param_grid = {}
            print(f"⚠️ 警告: 模型類型 '{self.model_type}' 沒有定義參數網格，使用空字典")
        
        return param_grid
    
    def train(self, 
              X: pd.DataFrame,
              y: pd.Series,
              t1: pd.Series,
              sample_weight: Optional[pd.Series] = None,
              cv: int = 5,
              pctEmbargo: float = 0.01,
              use_hyperopt: bool = True,
              use_randomized: bool = False,
              rndSearchIter: int = 50,
              n_jobs: int = -1,
              **model_kwargs) -> Any:
        """
        訓練模型
        """
        print("=" * 60)
        print(f"🚀 開始訓練 {self.model_type.upper()} 模型")
        print("=" * 60)
        
        if sample_weight is None:
            sample_weight = pd.Series(1.0, index=X.index)
        
        # 判斷是否需要標準化（線性模型和 SVM、MLP 需要）
        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb', 'qda']
        
        if use_hyperopt:
            print(f"\n📊 使用 {'RandomizedSearchCV' if use_randomized else 'GridSearchCV'} 進行超參數調優...")
            
            # 創建 pipeline
            steps = []
            if needs_scaling:
                steps.append(('scaler', StandardScaler()))
            steps.append((self.model_type, self._create_model(**model_kwargs)))
            
            pipe_clf = MyPipeline(steps)
            
            param_grid = self._get_param_grid(use_randomized=use_randomized)
            
            # 準備 fit_params（某些模型不支援 sample_weight）
            fit_params = {}
            if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                fit_params[f'{self.model_type}__sample_weight'] = sample_weight
            
            best_pipeline = clfHyperFit(
                feat=X,
                lbl=y,
                t1=t1,
                pipe_clf=pipe_clf,
                param_grid=param_grid,
                cv=cv,
                bagging=[0, None, 1.],
                rndSearchIter=rndSearchIter if use_randomized else 0,
                n_jobs=n_jobs,
                pctEmbargo=pctEmbargo,
                **fit_params
            )
            
            self.model = best_pipeline.named_steps[self.model_type]
            
            if hasattr(best_pipeline, 'best_params_'):
                self.best_params = best_pipeline.best_params_
            
            print(f"\n✅ 超參數調優完成")
            if self.best_params:
                print(f"最佳參數:")
                for key, value in list(self.best_params.items())[:5]:
                    print(f"  {key}: {value}")
                if len(self.best_params) > 5:
                    print(f"  ... (共 {len(self.best_params)} 個參數)")
        else:
            print(f"\n📊 直接訓練模型（不使用超參數調優）...")
            
            inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
            
            # 對於需要標準化的模型，使用 Pipeline
            if needs_scaling:
                from sklearn.pipeline import Pipeline
                clf = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', self._create_model(**model_kwargs))
                ])
                fit_params = {}
                if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                    fit_params['clf__sample_weight'] = sample_weight
            else:
                clf = self._create_model(**model_kwargs)
                fit_params = {}
                if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda']:
                    fit_params['sample_weight'] = sample_weight
            
            self.cv_scores = cvScore(
                clf=clf,
                X=X,
                y=y,
                sample_weight=sample_weight if self.model_type not in ['lr', 'svm', 'knn', 'nb', 'qda'] else None,
                scoring='f1',
                cvGen=inner_cv
            )
            
            print(f"\n交叉驗證分數: {self.cv_scores}")
            print(f"平均分數: {self.cv_scores.mean():.4f} ± {self.cv_scores.std():.4f}")
            
            if needs_scaling:
                from sklearn.pipeline import Pipeline
                self.model = Pipeline([
                    ('scaler', StandardScaler()),
                    ('clf', self._create_model(**model_kwargs))
                ])
                self.model.fit(X, y, **fit_params)
            else:
                self.model = clf.fit(X, y, **fit_params)
        
        print("=" * 60)
        print("✅ 模型訓練完成")
        print("=" * 60)
        
        return self.model
    
    def predict(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """進行預測"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        predictions = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        return predictions, probabilities
    
    def evaluate(self, 
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """評估模型性能"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        
        metrics = {
            'accuracy': accuracy_score(y, y_pred, sample_weight=sample_weight),
            'f1': f1_score(y, y_pred, sample_weight=sample_weight, average='weighted'),
        }
        
        if len(np.unique(y)) == 2:
            metrics['roc_auc'] = roc_auc_score(y, y_proba[:, 1], sample_weight=sample_weight)
            metrics['f1_binary'] = f1_score(y, y_pred, sample_weight=sample_weight)
        
        return metrics
    
    def plot_results(self, 
                    X: pd.DataFrame,
                    y: pd.Series,
                    sample_weight: Optional[pd.Series] = None):
        """繪製模型評估結果（與原版相同）"""
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        metrics = self.evaluate(X, y, sample_weight)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 混淆矩陣
        cm = confusion_matrix(y, y_pred, sample_weight=sample_weight)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
        axes[0, 0].set_title('Confusion Matrix', fontsize=14)
        axes[0, 0].set_ylabel('True Label', fontsize=12)
        axes[0, 0].set_xlabel('Predicted Label', fontsize=12)
        
        # 2. ROC 曲線
        if len(np.unique(y)) == 2:
            fpr, tpr, _ = roc_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[0, 1].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {metrics["roc_auc"]:.4f})')
            axes[0, 1].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[0, 1].set_xlabel('False Positive Rate', fontsize=12)
            axes[0, 1].set_ylabel('True Positive Rate', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'ROC Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
        
        # 3. Precision-Recall 曲線
        if len(np.unique(y)) == 2:
            precision, recall, _ = precision_recall_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[1, 0].plot(recall, precision, linewidth=2)
            axes[1, 0].set_xlabel('Recall', fontsize=12)
            axes[1, 0].set_ylabel('Precision', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
            axes[1, 0].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'Precision-Recall Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
        
        # 4. 評估指標摘要
        axes[1, 1].axis('off')
        metrics_text = f"""
        Model: {self.model_type.upper()}
        
        Evaluation Metrics:
        --------------------
        Accuracy: {metrics['accuracy']:.4f}
        F1 Score: {metrics['f1']:.4f}
        """
        if 'roc_auc' in metrics:
            metrics_text += f"ROC AUC: {metrics['roc_auc']:.4f}\n"
        if 'f1_binary' in metrics:
            metrics_text += f"F1 (Binary): {metrics['f1_binary']:.4f}\n"
        
        axes[1, 1].text(0.1, 0.5, metrics_text, fontsize=12, 
                        verticalalignment='center', family='monospace')
        
        plt.tight_layout()
        plt.show()
        
        print("\n" + "=" * 60)
        print("📊 詳細分類報告")
        print("=" * 60)
        print(classification_report(y, y_pred, sample_weight=sample_weight))
        print("=" * 60)

In [8]:
# =====================================================
# Hyperparameter Tuning with Purged K-Fold CV
# Snippet 9.1, 9.2, 9.3, 9.4
# =====================================================

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import rv_continuous, kstest
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# =====================================================
# Snippet 9.2: Enhanced Pipeline Class
# =====================================================

class MyPipeline(Pipeline):
    """
    Enhanced Pipeline class that handles sample_weight argument
    
    說明:
    -----
    sklearn 的 Pipeline 的 fit 方法不接受 sample_weight 參數，
    而是期望 fit_params 關鍵字參數。這是一個已知的 bug。
    這個類別擴展了 Pipeline，重寫 fit 方法以處理 sample_weight。
    
    參考:
    -----
    - GitHub issue: sklearn Pipeline sample_weight bug
    - Stackoverflow: Understanding Python super with __init__ methods
    """
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """
        重寫 fit 方法以處理 sample_weight
        
        參數:
        -----------
        X : array-like
            特徵數據
        y : array-like
            標籤
        sample_weight : array-like, optional
            樣本權重
        **fit_params : dict
            其他 fit 參數
            
        Returns:
        --------
        self : MyPipeline
            擬合後的 pipeline
        """
        if sample_weight is not None:
            # 將 sample_weight 傳遞給最後一個步驟（通常是分類器）
            last_step_name = self.steps[-1][0]
            fit_params[f'{last_step_name}__sample_weight'] = sample_weight
        
        return super(MyPipeline, self).fit(X, y, **fit_params)


# =====================================================
# Snippet 9.4: Log-Uniform Distribution
# =====================================================

class logUniform_gen(rv_continuous):
    """
    Log-uniform distribution random variable generator
    
    說明:
    -----
    隨機變數 x 在 [a, b] 區間內遵循 log-uniform 分布，
    當且僅當 log[x] ~ U[log[a], log[b]]
    
    這個分布對於探索非線性響應的參數空間特別有效，
    例如 SVC 的 C 參數和 RBF kernel 的 gamma 參數。
    
    數學定義:
    ---------
    CDF: F[x] = log[x] - log[a] / log[b] - log[a]  for a ≤ x ≤ b
    PDF: f[x] = 1 / (x * log[b/a])                 for a ≤ x ≤ b
    """
    
    def _cdf(self, x):
        """累積分布函數 (CDF)"""
        return np.log(x / self.a) / np.log(self.b / self.a)


def logUniform(a=1, b=np.exp(1)):
    """
    創建 log-uniform 分布
    
    參數:
    -----------
    a : float
        下界（必須 > 0）
    b : float
        上界（必須 > a）
        
    Returns:
    --------
    logUniform_gen : logUniform_gen
        Log-uniform 分布生成器
    """
    return logUniform_gen(a=a, b=b, name='logUniform')


# =====================================================
# Snippet 9.1 & 9.3: Hyperparameter Fitting Function
# =====================================================

def clfHyperFit(feat: pd.DataFrame, 
                lbl: pd.Series,
                t1: pd.Series,
                pipe_clf: Pipeline,
                param_grid: Dict[str, List[Any]],
                cv: int = 3,
                bagging: List[Any] = [0, None, 1.],
                rndSearchIter: int = 0,
                n_jobs: int = -1,
                pctEmbargo: float = 0.0,
                scoring: str = 'f1_macro',  # ⭐ 新增參數
                **fit_params) -> Pipeline:
    """
    使用 Purged K-Fold 交叉驗證進行超參數調優
    
    參數:
    -----------
    scoring : str
        評分方法（用於 GridSearchCV/RandomizedSearchCV）
        - 'f1_macro': Macro-F1
        - 'f1_weighted': Weighted-F1
        - 'accuracy_macro': Macro Accuracy
        - 'precision_macro': Macro Precision
        - 'recall_macro': Macro Recall
        - 'accuracy': 準確率
        - 'neg_log_loss': 負對數損失
    **fit_params : dict
        其他 fit 參數（例如 sample_weight），但不包括 scoring
    """
    
    # ⭐ 重要：從 fit_params 中移除 scoring（如果存在的話）
    # 因為 scoring 是 GridSearchCV 的參數，不是 Pipeline.fit() 的參數
    fit_params_clean = {k: v for k, v in fit_params.items() if k != 'scoring'}
    
    # 1) 超參數搜尋，在訓練數據上
    inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    
    # ⚠️ 注意：sklearn 的 GridSearchCV/RandomizedSearchCV 不直接支援自定義的 scoring 字符串
    # 需要創建一個自定義的 scorer 或使用 make_scorer
    # 但由於我們使用的是 PurgedKFold，可能需要自定義評分邏輯
    
    # 暫時使用 scoring 參數（如果 sklearn 支援的話）
    # 如果不支援，需要創建自定義 scorer
    from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score
    
    # 創建自定義 scorer（如果需要）
    if scoring == 'f1_macro':
        scorer = make_scorer(f1_score, average='macro', zero_division=0)
    elif scoring == 'f1_weighted':
        scorer = make_scorer(f1_score, average='weighted', zero_division=0)
    elif scoring == 'precision_macro':
        scorer = make_scorer(precision_score, average='macro', zero_division=0)
    elif scoring == 'recall_macro':
        scorer = make_scorer(recall_score, average='macro', zero_division=0)
    elif scoring == 'accuracy_macro':
        # 使用自定義的 macro_accuracy_score
        scorer = make_scorer(macro_accuracy_score, zero_division=0)
    elif scoring == 'accuracy':
        scorer = make_scorer(accuracy_score)
    elif scoring == 'neg_log_loss':
        from sklearn.metrics import log_loss
        scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
    else:
        # 預設使用 f1_macro
        scorer = make_scorer(f1_score, average='macro', zero_division=0)
    
    if rndSearchIter == 0:
        gs = GridSearchCV(
            estimator=pipe_clf,
            param_grid=param_grid,
            scoring=scorer,  # ⭐ 使用自定義 scorer
            cv=inner_cv,
            n_jobs=n_jobs
        )
    else:
        gs = RandomizedSearchCV(
            estimator=pipe_clf,
            param_distributions=param_grid,
            scoring=scorer,  # ⭐ 使用自定義 scorer
            cv=inner_cv,
            n_jobs=n_jobs,
            n_iter=rndSearchIter
        )
    
    # ⭐ 使用清理後的 fit_params（不包含 scoring）
    gs = gs.fit(feat, lbl, **fit_params_clean).best_estimator_
    
    # 2) 在全部數據上擬合驗證後的模型
    if bagging[1] is not None and bagging[1] > 0:
        gs = BaggingClassifier(
            base_estimator=MyPipeline(gs.steps),
            n_estimators=int(bagging[0]),
            max_samples=float(bagging[1]),
            max_features=float(bagging[2]),
            n_jobs=n_jobs
        )
        gs = gs.fit(feat, lbl, **fit_params_clean)
    
    return gs


In [9]:
# =====================================================
# 步驟10：PredictionPipeline - 整合步驟7、8、9
# =====================================================

import pandas as pd
import numpy as np
from typing import Optional, Dict, Any, Tuple, List
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class PredictionPipeline:
    """
    預測 Pipeline：整合模型訓練、交叉驗證、超參數調整
    
    功能:
    -----
    1. 模型訓練（步驟7）
    2. PurgedKFold 交叉驗證（步驟8）
    3. 超參數調整（步驟9）
    4. 模型預測
    5. 結果評估和可視化
    
    使用方式:
    --------
    pipeline = PredictionPipeline(
        model_type='xgboost',  # 或 'rf'
        use_hyperopt=True,     # 是否使用超參數調整
        use_cv=True            # 是否使用交叉驗證
    )
    
    # 傳入準備好的數據
    pipeline.fit(
        X=X,                    # 特徵矩陣
        y=y,                    # 標籤
        t1=t1,                  # 標籤結束時間
        sample_weight=weights   # 樣本權重（可選）
    )
    
    # 預測
    predictions, probabilities = pipeline.predict(X_test)
    """
    
    def __init__(self,
                 model_type: str = 'xgboost',
                 cv_scoring: str = 'f1_macro',  # ⭐ 新增這個參數
                 use_hyperopt: bool = True,
                 use_cv: bool = True,
                 cv_splits: int = 5,
                 pct_embargo: float = 0.01,
                 hyperopt_method: str = 'randomized',  # 'randomized' 或 'grid'
                 n_iter: int = 30,
                 bagging: Optional[List[Any]] = None,
                 n_jobs: int = -1):
        """
        初始化預測 Pipeline
        
        參數:
        -----------
        model_type : str
            模型類型：'xgboost' 或 'rf'
        use_hyperopt : bool
            是否使用超參數調整（預設 True）
        use_cv : bool
            是否使用交叉驗證（預設 True）
        cv_splits : int
            交叉驗證折數（預設 5）
        pct_embargo : float
            禁制比例（預設 0.01）
        hyperopt_method : str
            超參數調整方法：'randomized' 或 'grid'（預設 'randomized'）
        n_iter : int
            隨機搜尋迭代次數（預設 30）
        bagging : list, optional
            Bagging 參數 [n_estimators, max_samples, max_features]
        n_jobs : int
            並行作業數（預設 -1）
        """
        self.model_type = model_type.lower()
        self.cv_scoring = cv_scoring  # ⭐ 新增這行
        self.use_hyperopt = use_hyperopt
        self.use_cv = use_cv
        self.cv_splits = cv_splits
        self.pct_embargo = pct_embargo
        self.hyperopt_method = hyperopt_method
        self.n_iter = n_iter
        self.bagging = bagging if bagging is not None else [0, None, 1.]
        self.n_jobs = n_jobs
        
        # 內部變量
        self.model = None
        self.scaler = None
        self.best_params = None
        self.cv_scores = None
        self.best_cv_scores = None
        self.X_train = None
        self.y_train = None
        self.t1_train = None
        self.sample_weight_train = None
        self.feature_columns = None
        
    # 在 PredictionPipeline 類中，修改 _create_pipeline 方法
    def _create_pipeline(self, n_classes: Optional[int] = None, **model_kwargs) -> MyPipeline:
        """
        建立模型 Pipeline，會依照實際類別數調整模型目標函式。
        """
        from sklearn.preprocessing import StandardScaler

        if self.model_type == 'xgboost':
            from xgboost import XGBClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'eval_metric': 'logloss',
                    'use_label_encoder': False,
                    'objective': 'binary:logistic',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'eval_metric': 'mlogloss',
                    'use_label_encoder': False,
                    'objective': 'multi:softprob',
                    'num_class': n_classes,
                }
            default_params.update(model_kwargs)
            clf = XGBClassifier(**default_params)

        elif self.model_type in ('lightgbm', 'lgb'):
            from lightgbm import LGBMClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'verbosity': -1,
                    'force_col_wise': True,
                    'objective': 'binary',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'verbosity': -1,
                    'force_col_wise': True,
                    'objective': 'multiclass',
                    'num_class': n_classes,
                }
            default_params.update(model_kwargs)
            clf = LGBMClassifier(**default_params)

        elif self.model_type == 'catboost':
            from catboost import CatBoostClassifier
            if n_classes is None or n_classes == 2:
                default_params = {
                    'random_state': 42,
                    'verbose': False,
                    'thread_count': -1,
                    'loss_function': 'Logloss',
                }
            else:
                default_params = {
                    'random_state': 42,
                    'verbose': False,
                    'thread_count': -1,
                    'loss_function': 'MultiClass',
                }
            default_params.update(model_kwargs)
            clf = CatBoostClassifier(**default_params)

        elif self.model_type == 'rf':
            from sklearn.ensemble import RandomForestClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = RandomForestClassifier(**default_params)

        elif self.model_type == 'et':
            from sklearn.ensemble import ExtraTreesClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = ExtraTreesClassifier(**default_params)

        elif self.model_type == 'gb':
            from sklearn.ensemble import GradientBoostingClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = GradientBoostingClassifier(**default_params)

        elif self.model_type == 'histgbm':
            from sklearn.ensemble import HistGradientBoostingClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = HistGradientBoostingClassifier(**default_params)

        elif self.model_type == 'dt':
            from sklearn.tree import DecisionTreeClassifier
            default_params = {'random_state': 42, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = DecisionTreeClassifier(**default_params)

        elif self.model_type == 'lr':
            from sklearn.linear_model import LogisticRegression
            default_params = {'random_state': 42, 'max_iter': 1000, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = LogisticRegression(**default_params)

        elif self.model_type == 'svm':
            from sklearn.svm import SVC
            default_params = {'random_state': 42, 'probability': True, 'class_weight': 'balanced'}
            default_params.update(model_kwargs)
            clf = SVC(**default_params)

        elif self.model_type == 'mlp':
            from sklearn.neural_network import MLPClassifier
            default_params = {'random_state': 42, 'max_iter': 1000}
            default_params.update(model_kwargs)
            clf = MLPClassifier(**default_params)

        elif self.model_type == 'ada':
            from sklearn.ensemble import AdaBoostClassifier
            default_params = {'random_state': 42}
            default_params.update(model_kwargs)
            clf = AdaBoostClassifier(**default_params)

        elif self.model_type == 'nb':
            from sklearn.naive_bayes import GaussianNB
            clf = GaussianNB()

        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")

        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb']
        if needs_scaling:
            pipe = MyPipeline([('scaler', StandardScaler()), (self.model_type, clf)])
        else:
            pipe = MyPipeline([(self.model_type, clf)])

        return pipe
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        """
        print(f"🚀 取得 {self.model_type} 模型參數網格")
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
                return param_grid
        
        elif self.model_type == 'lightgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10, -1],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__num_leaves': [15, 31, 50, 100],
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_samples': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__num_leaves': [31, 50],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
        
        elif self.model_type == 'catboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300, 500],
                    f'{self.model_type}__depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__l2_leaf_reg': [1, 3, 5, 7],
                    f'{self.model_type}__border_count': [32, 64, 128],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__iterations': [100, 200, 300],
                    f'{self.model_type}__depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__l2_leaf_reg': [3, 5],
                }
                return param_grid
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500, 1000],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': logUniform(1e-3, 1e2),
                    f'{self.model_type}__min_samples_leaf': logUniform(1e-3, 1e1),
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                    f'{self.model_type}__bootstrap': [True, False]
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None]
                }
                return param_grid
        
        elif self.model_type == 'et':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__max_features': ['sqrt', 'log2'],
                }
                return param_grid
        
        elif self.model_type == 'gb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200],
                    f'{self.model_type}__max_depth': [3, 5],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                }
                return param_grid
        
        elif self.model_type == 'histgbm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200, 300],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__min_samples_leaf': [10, 20, 30],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__max_iter': [100, 200],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1],
                    f'{self.model_type}__max_depth': [3, 5, 7],
                }
                return param_grid
        
        elif self.model_type == 'dt':
            param_grid = {
                f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                f'{self.model_type}__min_samples_split': [2, 5, 10],
                f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                f'{self.model_type}__max_features': ['sqrt', 'log2', None],
            }
            return param_grid
        
        elif self.model_type == 'lr':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-3, 1e3),
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.01, 0.1, 1.0, 10.0],
                    f'{self.model_type}__penalty': ['l2'],  # lbfgs 只支持 l2
                    f'{self.model_type}__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],  # 支持 multinomial 的 solver
                }
                return param_grid
        
        elif self.model_type == 'svm':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__C': logUniform(1e-2, 1e2),
                    f'{self.model_type}__gamma': logUniform(1e-4, 1e-1),
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__C': [0.1, 1.0, 10.0],
                    f'{self.model_type}__gamma': ['scale', 'auto', 0.001, 0.01],
                    f'{self.model_type}__kernel': ['rbf', 'poly'],
                }
                return param_grid
        
        elif self.model_type == 'mlp':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                    f'{self.model_type}__alpha': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__hidden_layer_sizes': [(50,), (100,), (50, 50)],
                    f'{self.model_type}__alpha': [0.0001, 0.001, 0.01],
                    f'{self.model_type}__learning_rate': ['constant', 'adaptive'],
                }
                return param_grid
        
        elif self.model_type == 'ada':
            param_grid = {
                f'{self.model_type}__n_estimators': [50, 100, 200],
                f'{self.model_type}__learning_rate': [0.01, 0.1, 1.0],
            }
            return param_grid   
        
        elif self.model_type == 'knn':
            param_grid = {
                f'{self.model_type}__n_neighbors': [3, 5, 7, 10, 15],
                f'{self.model_type}__weights': ['uniform', 'distance'],
                f'{self.model_type}__metric': ['euclidean', 'manhattan', 'minkowski'],
            }
            return param_grid
        elif self.model_type == 'nb':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__var_smoothing': logUniform(1e-9, 1e-3),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
                }
                return param_grid
        elif self.model_type == 'qda':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__reg_param': logUniform(1e-3, 1.0),
                }
                return param_grid
            else:
                param_grid = {
                    f'{self.model_type}__reg_param': [0.0, 0.1, 0.5, 1.0],
                }   
                return param_grid
        else:
            # 如果模型類型沒有對應的參數網格，返回空字典
            param_grid = {}
            print(f"⚠️ 警告: 模型類型 '{self.model_type}' 沒有定義參數網格，使用空字典")
        
        return param_grid
    
    def fit(
        self,
        X: pd.DataFrame,
        y: pd.Series,
        t1: Optional[pd.Series],
        sample_weight: Optional[pd.Series] = None,
        **model_kwargs,
    ) -> "PredictionPipeline":
        """
        訓練模型（整合 PurgedKFold、超參數搜尋）
        """
        from sklearn.preprocessing import LabelEncoder

        print("=" * 60)
        print(f"🚀 PredictionPipeline 開始訓練 ({self.model_type.upper()})")
        print("=" * 60)

        # 步驟0：標籤重新編碼
        print("\n" + "=" * 60)
        print("📊 步驟0：重新編碼標籤")
        print("=" * 60)
        original_unique = sorted(y.unique())
        print(f"原始標籤唯一值: {original_unique}")
        print(f"原始標籤分布: {y.value_counts().to_dict()}")

        label_encoder = LabelEncoder()
        y_encoded = pd.Series(label_encoder.fit_transform(y), index=y.index)
        self.label_encoder_ = label_encoder

        print(f"編碼後標籤唯一值: {sorted(y_encoded.unique())}")
        print(f"編碼後標籤分布: {y_encoded.value_counts().to_dict()}")
        print(f"編碼映射: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

        n_classes = len(label_encoder.classes_)
        print(f"實際類別數: {n_classes}")

        y = y_encoded

        # 步驟1：索引對齊
        print("\n" + "=" * 60)
        print("📊 步驟1：數據對齊與驗證")
        print("=" * 60)

        common_idx = X.index.intersection(y.index)
        if t1 is not None:
            common_idx = common_idx.intersection(t1.index)
        if sample_weight is not None:
            common_idx = common_idx.intersection(sample_weight.index)

        X = X.loc[common_idx]
        y = y.loc[common_idx]
        if t1 is not None:
            t1 = t1.loc[common_idx]
        if sample_weight is not None:
            sample_weight = sample_weight.loc[common_idx]

        print(f"對齊後樣本數: {len(X):,}")
        print(f"對齊後標籤分布: {y.value_counts().to_dict()}")

        self.X_train = X
        self.y_train = y
        self.t1_train = t1
        self.sample_weight_train = sample_weight
        self.feature_columns = X.columns.tolist()

        # 步驟2：建立基準模型
        print("\n" + "=" * 60)
        print("📊 步驟2：建立基準模型")
        print("=" * 60)
        baseline_pipe = self._create_pipeline(n_classes=n_classes, **model_kwargs)

        # 步驟3：PurgedKFold 交叉驗證
        if self.use_cv:
            print("\n" + "=" * 60)
            print("📊 步驟3：執行 PurgedKFold 交叉驗證")
            print("=" * 60)

            cv = PurgedKFold(
                n_splits=self.cv_splits,
                t1=self.t1_train,
                pctEmbargo=self.pct_embargo,
            )
            scoring = self.cv_scoring
            print("交叉驗證配置:")
            print(f"  K 折數: {self.cv_splits}")
            print(f"  禁制比例: {self.pct_embargo:.2%}")
            print(f"  評分方法: {scoring}")

            self.cv_scores = cvScore(
                clf=baseline_pipe,
                X=self.X_train,
                y=self.y_train,
                sample_weight=self.sample_weight_train,
                scoring=scoring,
                cvGen=cv,
            )
            print("\n✅ 基準模型交叉驗證完成")
            print(f"   分數: {self.cv_scores}")

        # 步驟4：超參數搜尋
        if self.use_hyperopt:
            print("\n" + "=" * 60)
            print("📊 步驟4：超參數搜尋")
            print("=" * 60)

            param_grid = self._get_param_grid(
                use_randomized=(self.hyperopt_method == 'randomized')
            )
            print(f"超參數搜尋方式: {self.hyperopt_method}")
            print(f"參數空間大小: {len(param_grid)}")

            pipe_for_search = self._create_pipeline(n_classes=n_classes, **model_kwargs)
            self.model = clfHyperFit(
                feat=self.X_train,
                lbl=self.y_train,
                t1=self.t1_train,
                pipe_clf=pipe_for_search,
                param_grid=param_grid,
                cv=self.cv_splits,
                bagging=self.bagging,
                rndSearchIter=self.n_iter if self.hyperopt_method == 'randomized' else 0,
                n_jobs=self.n_jobs,
                pctEmbargo=self.pct_embargo,
                scoring=self.cv_scoring,
                sample_weight=(
                    self.sample_weight_train.values
                    if self.sample_weight_train is not None
                    else None
                ),
            )
            print("\n✅ 超參數搜尋完成")
        else:
            print("\n" + "=" * 60)
            print("📊 步驟4：使用預設超參數訓練")
            print("=" * 60)
            self.model = baseline_pipe
            self.model.fit(
                self.X_train,
                self.y_train,
                **(
                    {}
                    if self.sample_weight_train is None
                    else {f"{self.model_type}__sample_weight": self.sample_weight_train.values}
                ),
            )

        # 步驟5：建立標準化器（如有需要）
        needs_scaling = self.model_type in ['lr', 'svm', 'mlp', 'knn', 'nb']
        if needs_scaling:
            from sklearn.preprocessing import StandardScaler
            self.scaler = StandardScaler()
            self.scaler.fit(self.X_train)
        else:
            self.scaler = None

        print("\n✅ 模型訓練完成")
        return self
    
    def predict(self, X: pd.DataFrame):
        """
        預測
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
            
        Returns:
        --------
        predictions : np.ndarray
            預測標籤（編碼後的）
        probabilities : np.ndarray
            預測概率
        """
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        # 預測
        predictions_encoded = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        # 如果需要，可以將編碼後的標籤轉回原始標籤
        # 但通常我們保持編碼後的標籤，讓調用者自己處理映射
        
        return predictions_encoded, probabilities
    
    def evaluate(self,
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """
        評估模型性能
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            真實標籤
        sample_weight : pd.Series, optional
            樣本權重
            
        Returns:
        --------
        metrics : dict
            評估指標
        """
        from sklearn.metrics import (accuracy_score, f1_score, 
                                   precision_score, recall_score,
                                   roc_auc_score)
        
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        print("=" * 60)
        print("📊 評估模型性能")
        print("=" * 60)
        
        # 預測
        predictions, probabilities = self.predict(X)
        
        # 計算指標
        metrics = {
            'accuracy': accuracy_score(y, predictions, sample_weight=sample_weight),
            'precision': precision_score(y, predictions, average='weighted', 
                                        sample_weight=sample_weight, zero_division=0),
            'recall': recall_score(y, predictions, average='weighted', 
                                  sample_weight=sample_weight, zero_division=0),
            'f1': f1_score(y, predictions, average='weighted', 
                          sample_weight=sample_weight, zero_division=0),
        }
        
        # 如果是二分類，計算更多指標
        if len(np.unique(y)) == 2:
            metrics['f1_binary'] = f1_score(y, predictions, sample_weight=sample_weight, zero_division=0)
            metrics['roc_auc'] = roc_auc_score(y, probabilities[:, 1], sample_weight=sample_weight)
        
        # 打印結果
        print(f"\n評估指標:")
        for key, value in metrics.items():
            print(f"  {key}: {value:.4f}")
        
        # 混淆矩陣
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
        print(f"\n混淆矩陣:")
        print(cm)
        
        print("=" * 60)
        
        return metrics
    
    def plot_results(self,
                    X: pd.DataFrame = None,
                    y: pd.Series = None,
                    sample_weight: Optional[pd.Series] = None):
        """
        繪製結果（交叉驗證分數、預測結果等）
        
        參數:
        -----------
        X : pd.DataFrame, optional
            特徵矩陣（用於預測和評估）
        y : pd.Series, optional
            真實標籤（用於評估）
        sample_weight : pd.Series, optional
            樣本權重
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 交叉驗證分數對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.6, 
                          label='Baseline Model', color='blue', width=0.4)
            axes[0, 0].bar([i + 0.4 for i in range(len(self.best_cv_scores))], 
                          self.best_cv_scores, alpha=0.6, 
                          label='Best Model (After Tuning)', color='red', width=0.4)
            axes[0, 0].axhline(self.cv_scores.mean(), color='blue', linestyle='--', 
                             label=f'Baseline Mean: {self.cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].axhline(self.best_cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Best Mean: {self.best_cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores: Baseline vs Best Model', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].set_xticks(range(len(self.cv_scores)))
            axes[0, 0].set_xticklabels([f'Fold {i+1}' for i in range(len(self.cv_scores))])
        elif self.cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.7, color='blue')
            axes[0, 0].axhline(self.cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Mean: {self.cv_scores.mean():.4f}', linewidth=2)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
        else:
            axes[0, 0].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
        
        # 2. 分數分布對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores, self.best_cv_scores], 
                             labels=['Baseline', 'Best Model'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].scatter([2] * len(self.best_cv_scores), self.best_cv_scores, 
                             alpha=0.3, s=30, color='red')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution Comparison', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        elif self.cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores], labels=['Baseline'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
        
        # 3. 預測結果（如果有 X 和 y）
        if X is not None and y is not None:
            predictions, probabilities = self.predict(X)
            
            # 混淆矩陣
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
            sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', ax=axes[1, 0])  # 使用浮點數格式
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 0].set_ylabel('True Label', fontsize=12)
            axes[1, 0].set_xlabel('Predicted Label', fontsize=12)
            
            # 預測分布
            pred_counts = pd.Series(np.ravel(predictions)).value_counts().sort_index()
            axes[1, 1].bar(pred_counts.index, pred_counts.values, alpha=0.7)
            axes[1, 1].set_xlabel('Predicted Label', fontsize=12)
            axes[1, 1].set_ylabel('Count', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 1].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
        
        plt.tight_layout()
        plt.show()

In [10]:
# ========================================================================================================
# 🚀 MetaNewLow 全商品自動化預測流程 (二元分類 + 完整欄位 + FracDiff 全市場特徵)
# ========================================================================================================
# 適用策略：S1Strategy (新低突破)
# 標籤定義：Class 0 (Hit_SL), Class 1 (No_SL)
# 特徵來源：個別商品特徵 + 全市場日頻特徵
# ========================================================================================================

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from typing import List, Optional
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
import numpy as np

warnings.filterwarnings('ignore')

# 全局配置
TRAIN_START = '2014-11-01'
TEST_HORIZON_START_YEAR = 2019
ROLLING_YEARS = 7

ALL_COMMODITIES = ["AUS200", "NAS100", "ESP35", "FRA40", "GER30", "HKG33", "JPN225", "SPX500", "US2000"]
ALL_MODELS = ['rf', 'lightgbm', 'nb', 'et', 'dt', 'xgboost', 'catboost', 'svm', 'lr']

BASE_DIR = Path(r"C:\Users\a124a\OneDrive\桌面\策略開發\MetaLabelLearning\MetaNewLow")
RAW_DATA_DIR = BASE_DIR / "equity_prices"
FEATURE_DIR = Path(r"C:\Users\a124a\OneDrive\桌面\策略開發\MetaLabelLearning\MetaNewLow\feature_data")
RESULTS_DIR = BASE_DIR / "pred_results_binary"

RESULTS_DIR.mkdir(exist_ok=True, parents=True)

# -------------------------------------------------------------------
# S1Strategy (新低)
# -------------------------------------------------------------------
class S1Strategy:
    def __init__(self, entry_param: float = 1.0, rolling_window: int = 96):
        self.entry_param = entry_param
        self.rolling_window = rolling_window
        self.signals = None
        
    def calculate_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df['HL115'] = (
            df['High'].shift(1).rolling(self.rolling_window, min_periods=self.rolling_window).max() - 
            df['Low'].shift(1).rolling(self.rolling_window, min_periods=self.rolling_window).min()
        )
        df['突破signal價'] = (
            df['High'].shift(1).rolling(self.rolling_window).max() - 
            (self.entry_param * df['HL115'])
        )
        df['signal'] = False
        df['signal'] = (
            (df['signal'].shift(1) == False) & 
            (df['Low'] < df['突破signal價']) & 
            (df['Open'] > df['突破signal價'])
        )
        df['side'] = np.nan
        df.loc[df['signal'] == True, 'side'] = -1.0
        df['side'] = df['side'].ffill()
        self.signals = df
        return df
    
    def get_signal_events(self) -> pd.DatetimeIndex:
        if self.signals is None: return None
        return self.signals[self.signals['signal'] == True].index

# (NoiseFilter, MetaLabeling, SampleWeight, PredictionPipeline 假設已定義)

# -------------------------------------------------------------------
# 資料準備 (含全市場特徵合併)
# -------------------------------------------------------------------
def prepare_commodity_data_binary(comm):
    print(f"\n{'='*60}\n📦 {comm} (MetaNewHigh / L1)\n{'='*60}")
    
    # 1. 載入價格
    p = RAW_DATA_DIR / f"{comm}.csv"
    if not p.exists(): 
        print(f"❌ 找不到價格檔: {p}")
        return None
    try: df = pd.read_csv(p)
    except: return None
    
    d_col = next((c for c in df.columns if c.lower() in ['date','time','timestamp']), None)
    if not d_col: return None
    df[d_col] = pd.to_datetime(df[d_col])
    df = df.sort_values(by=d_col).set_index(d_col)
    
    r_map = {'bidopen':'Open', 'open':'Open', 'bidhigh':'High', 'high':'High', 
             'bidlow':'Low', 'low':'Low', 'bidclose':'Close', 'close':'Close', 'volume':'Volume'}
    df = df.rename(columns={c: r_map.get(c.lower(), c) for c in df.columns})
    df = df.dropna().loc[df.index >= TRAIN_START]
    
    # ✅ 移除 CUSUM 過濾,直接使用原始數據
    # 計算 daily volatility 用於 MetaLabeling
    close_ret = df['Close'].pct_change()
    daily_vol = close_ret.ewm(span=100).std()
    
    # Step 2: Strategy (L1)
    strategy = S1Strategy(entry_param=0.5, rolling_window=500)
    df_with_signals = strategy.calculate_indicators(df)
    tEvents = strategy.get_signal_events()
    if tEvents is None or len(tEvents) == 0:
        print("⚠️ 無策略訊號")
        return None
        
    # Step 3: Meta Labeling
    meta = MetaLabeling(ptSl=[0.25, 0.25], numPeriods=32, minRet=0.001)
    events = meta.get_events(df['Close'], tEvents, daily_vol, df_with_signals['side'])
    bins = meta.get_bins(df['Close'])
    
    # Step 5: 載入特徵 (個別 + 全市場)
    fp = FEATURE_DIR / f"{comm}_features.csv"
    if not fp.exists(): return None
    try: feats_individual = pd.read_csv(fp, index_col=0, parse_dates=True)
    except: feats_individual = pd.read_csv(fp, index_col=0, parse_dates=True, engine='python')
    feats_individual = feats_individual[~feats_individual.index.duplicated(keep='first')]
    
    fp_all = FEATURE_DIR / "fracdiff_features_all_products.csv"
    if fp_all.exists():
        try:
            feats_all = pd.read_csv(fp_all, index_col=0, parse_dates=True)
            feats_all.index = pd.to_datetime(feats_all.index)
            print(f"📥 成功載入全市場特徵: {len(feats_all)} 筆")
            feats_all_aligned = feats_all.reindex(df.index, method='ffill')
            X = pd.concat([feats_individual, feats_all_aligned], axis=1)
            X = X.loc[~X.index.duplicated(keep='first')]
        except Exception as e:
            print(f"⚠️ 載入全市場特徵失敗: {e},僅使用個別特徵")
            X = feats_individual
    else:
        print(f"⚠️ 找不到全市場特徵檔,僅使用個別特徵")
        X = feats_individual
    
    print(f"📊 原始特徵數: {len(X.columns)}")
    
    # =====================================================
    # Step 6: 數據清理
    # =====================================================
    print(f"\n{'='*60}\n🧹 數據清理...\n{'='*60}")
    
    X = X.fillna(method='ffill').fillna(method='bfill').fillna(0).replace([np.inf, -np.inf], 0)
    
    # 移除完全為常數的特徵
    constant_cols = []
    for col in X.columns:
        try:
            unique_count = int(X[col].nunique())
            if unique_count <= 1:
                constant_cols.append(col)
        except (ValueError, TypeError):
            continue
    
    if len(constant_cols) > 0:
        print(f"移除 {len(constant_cols)} 個常數特徵")
        X = X.drop(columns=constant_cols)
    
    # 移除完全為 NaN 的特徵
    nan_cols = [col for col in X.columns if X[col].isna().all()]
    if len(nan_cols) > 0:
        print(f"移除 {len(nan_cols)} 個全 NaN 特徵")
        X = X.drop(columns=nan_cols)
    
    print(f"\n✅ 清理完成")
    print(f"   最終特徵數: {len(X.columns):,}")
    
    # =====================================================
    # Step 7: 數據對齊
    # =====================================================
    print(f"\n{'='*60}\n🔗 數據對齊...\n{'='*60}")
    
    idx = bins.index.intersection(X.index)
    X, y = X.loc[idx], bins.loc[idx, 'bin']
    valid = X.dropna().index.intersection(y.dropna().index)
    X, y = X.loc[valid], y.loc[valid]
    events_valid = events.loc[valid]
    
    y_map = y.map({-1: 0, 0: 1, 1: 1})
    rev_map = {0: 'Hit_StopLoss', 1: 'No_StopLoss'}
    
    try:
        sw_calc = SampleWeight()
        sw = sw_calc.compute_sample_weights(events_valid['t1'], df['Close'].index, y_map)
    except:
        sw = pd.Series(1., index=y.index)
    
    print(f"✅ 對齊完成")
    print(f"   X: {len(X):,} 個樣本, {len(X.columns):,} 個特徵")
    print(f"   y: {len(y_map):,} 個樣本")
    
    # =====================================================
    # Step 6.5: 特徵篩選 Pipeline
    # =====================================================
    print(f"\n{'='*60}\n🔍 特徵篩選 Pipeline\n{'='*60}")
    print(f"⚠️ 只使用 test_start ({TEST_HORIZON_START_YEAR}-01-01) 之前的數據進行篩選")
    
    # 切分訓練集用於篩選
    test_start = f'{TEST_HORIZON_START_YEAR}-01-01'
    train_mask = (X.index >= TRAIN_START) & (X.index < test_start)
    X_train_filter = X.loc[train_mask].copy()
    y_train_filter = y_map.loc[train_mask].copy()
    sw_train_filter = sw.loc[train_mask].copy()
    
    print(f"\n篩選用訓練集:")
    print(f"   樣本數: {len(X_train_filter):,}")
    print(f"   特徵數: {len(X_train_filter.columns):,}")
    print(f"   時間範圍: {X_train_filter.index.min()} 至 {X_train_filter.index.max()}")
    
    # 執行篩選
    filter_pipeline = FeatureFilteringPipeline(
        corr_threshold=0.95,
        variance_threshold=0.005,
        ic_threshold=0.01,
        anova_p_threshold=0.2,
        mda_top_n=None,
        stability_n_iter=5,
        stability_threshold=0.1
    )
    
    X_train_filtered = filter_pipeline.fit_transform(
        X_train_filter,
        y_train_filter,
        sample_weight=sw_train_filter,
        use_stability=False
    )
    
    # 使用篩選後的特徵
    selected_features = filter_pipeline.selected_features
    X = X[selected_features]
    
    print(f"\n✅ 最終特徵數: {len(X.columns)} (含全市場特徵 + 篩選後)")
    
    return {
        'X': X, 'y': y_map, 'bins': bins.loc[valid], 
        'events': events_valid, 'sw': sw, 'rev_map': rev_map
    }



# ===================================================================
# 🏃‍♂️ 滾動預測 (共用邏輯)
# ===================================================================
# (此函數與 run_binary_prediction_full_format.py 中的完全相同，可直接共用)
def run_walk_forward_prediction_binary(comm, model_name, data):
    X, y, bins, events, sw, rev = data['X'], data['y'], data['bins'], data['events'], data['sw'], data['rev_map']
    
    dfs = []
    
    for yr in range(TEST_HORIZON_START_YEAR, TEST_HORIZON_START_YEAR + ROLLING_YEARS):
        tz = X.index.tz
        s_dt = pd.Timestamp(f'{yr}-01-01', tz=tz)
        e_dt = s_dt + pd.DateOffset(years=1)
        
        tr_mask = (X.index >= TRAIN_START) & (X.index < s_dt)
        te_mask = (X.index >= s_dt) & (X.index < e_dt)
        
        if tr_mask.sum() < 50 or te_mask.sum() == 0: continue
            
        X_train, y_train = X.loc[tr_mask], y.loc[tr_mask]
        t1_train, sw_train = events.loc[tr_mask, 't1'], sw.loc[tr_mask]
        X_test, y_test = X.loc[te_mask], y.loc[te_mask]
        test_events = events.loc[te_mask]
        
        try:
            # 確保 PredictionPipeline 已定義
            pipeline = PredictionPipeline(
                model_type=model_name,
                cv_scoring='roc_auc',
                use_hyperopt=True,
                use_cv=True,
                cv_splits=5,
                pct_embargo=0.01,
                hyperopt_method='grid',
                n_iter=10,
                n_jobs=-1,
            )
            
            pipeline.fit(X_train, y_train, t1_train, sw_train)
            
            preds_mapped, probs = pipeline.predict(X_test)
            preds_original = pd.Series(np.ravel(preds_mapped), index=X_test.index).map(rev)
            
            d = pd.DataFrame(index=X_test.index)
            d['t1'] = test_events['t1']
            d['side'] = test_events['side']
            d['true_label'] = y_test.map(rev)
            d['predicted_label'] = preds_original
            
            if probs is not None:
                for idx, class_name in rev.items():
                    if idx < probs.shape[1]:
                        d[f'prob_class_{idx}'] = probs[:, idx]
            
            d['prediction_confidence'] = np.max(probs, axis=1) if probs is not None else 0
            d['is_correct'] = (d['predicted_label'] == d['true_label'])
            d['model_type'] = model_name
            d['test_year'] = yr
            d['train_window_start'] = X_train.index.min()
            d['train_window_end'] = X_train.index.max()
            
            dfs.append(d)
        except Exception as e: 
            print(f"      Err {yr}: {e}")
            
    if not dfs: return
    out = pd.concat(dfs).sort_index()
    out.index.name = 'Date'
    
    out_filename = f"{comm}_pred_{model_name}_{TEST_HORIZON_START_YEAR}_{TEST_HORIZON_START_YEAR+ROLLING_YEARS-1}.csv"
    out_path = RESULTS_DIR / out_filename
    out.to_csv(out_path)
    print(f"   💾 {model_name}: {len(out)} rows saved")

# ===================================================================
# 🚀 主執行
# ===================================================================
if 'PredictionPipeline' not in globals():
    print("⚠️ 請確保 PredictionPipeline Class 已載入！")
else:
    print(f"🚀 開始執行 MetaNewLow 自動化預測...")
    for comm in ALL_COMMODITIES:
        try:
            data = prepare_commodity_data_binary(comm) # 使用 S1 準備函數
            if data is None: continue
            
            for model in ALL_MODELS:
                run_walk_forward_prediction_binary(comm, model, data)
                
        except Exception as e: 
            print(f"❌ {comm}: {e}")
            import traceback
            traceback.print_exc()

🚀 開始執行 MetaNewLow 自動化預測...

📦 AUS200 (MetaNewHigh / L1)
📥 成功載入全市場特徵: 231115 筆
📊 原始特徵數: 77

🧹 數據清理...

✅ 清理完成
   最終特徵數: 77

🔗 數據對齊...
📊 樣本權重計算配置
使用唯一性: True
使用報酬歸因: False
使用時間衰減: True
使用類別平衡: False
時間衰減類型: linear
📊 並發度統計
總時間點數: 221,764
平均並發事件數: 0.01
最大並發事件數: 6

說明:
  - 並發度衡量每個時間點有多少個標籤同時存活
  - 高並發度表示標籤之間重疊多，資訊冗餘
  - 低並發度表示標籤相對獨立
並發度分布:
count    221764.000000
mean          0.013221
std           0.134993
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           6.000000
dtype: float64
📊 唯一性統計
總事件數: 1,260
平均唯一性: 0.8153

說明:
  - 唯一性衡量每個標籤的非重疊程度
  - 唯一性 = 1 / 平均並發度
  - 唯一性越高，表示該標籤越獨立，權重應該越大
  - 唯一性越低，表示該標籤與其他標籤重疊多，權重應該越小
唯一性分布:
count    1260.000000
mean        0.815338
std         0.220004
min         0.181818
25%         0.666667
50%         1.000000
75%         1.000000
max         1.000000
dtype: float64
📊 線性時間衰減
時間衰減參數: const=1.0000, slope=0.000000
最舊樣本權重 (c): 1.0000
最新樣本權重: 1.0
平均時間衰減權重: 1.0000

說明:
  - c = 1: 無時間衰減
  - 衰減基於累積唯一性，而非時間順序
⚠

KeyboardInterrupt: 